In [ ]:
### LIBRARIES LOAD

from sentence_transformers import CrossEncoder
#from pytorch_modelsize import SizeEstimator
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import heapq
import os
import time
import sklearn.metrics
import matplotlib.pyplot as plt
import random
import codecs
import hickle as hkl
import pickle
import functools
import scipy


tqdm.pandas()
model = CrossEncoder('cross-encoder/stsb-roberta-large')  #stsb-roberta-large
#print(model.config.model_name_or_path)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#model = model.to(device)
print(device)

print(os.getcwd())

In [ ]:
### OLD REFERENCE SENTENCES FOR EACH GESTURE AND DESCRIPTION OF GESTURES. RUN THIS CELL AT THE START

reference_sentence_greet = ["hello", "hey there", "I greet you", "Greetings!", "Namaste", "good day", "goodbye", "see you"]
reference_sentence_faster = ["this is boring, go faster", "please talk faster", "continue please", "I have no time, finish the speech please"]
reference_sentence_goaway = ["let me alone", "do not disturb me", "I need some privacy", "go away", "you annoyed me"]
reference_sentence_cigarette = ["do you like to smoke?", "smoking is not healthy", "cigarette", "where are cigarettes",
                                "smoking is the cause of many diseases", "I like to smoke", "smoking"]
reference_sentence_idk = ["I don't know", "I don't understand"]
reference_sentence_no = ["I don't agree", "I don't like it", "no", "I don't want", "it's not ok", "It is not the same for me"] #never #I can't, I' wasn't
reference_sentence_yes = ["yes.", "yeah.", "I agree", "It is the same for me", "I like it", "ok", "I want it"] #right
reference_sentence_deitic = ["see there", "see that", "look at your left", "look at your right", "look over your head", "look behind"] #you have, look this thing. pay attention because sometimes this and that referes to something that is not present
reference_sentence_stop = ["slow down", "don't go further", "stop", "go slower", "stop talking", "talk slower"]
reference_sentence_adaptor = ["my arm", "my hand", "my head","my knee", "my leg", "my stomach", "my chest",
                             "my feet", "my eyes", "me", "I am"] #I have
reference_sentence_telephone = ["I have to do a call", "I want to call my friend", "telephone number", "call me"] #pay attention that call is not related to phone
reference_sentence_come = ["come to me", "come here", "come closer"]
reference_sentence_drink = ["do you want a drink?", "I want to drink", "I want some water", "can you give me a drink?",
                            "let's take a drink", "I am thirsty","let's take a drink"]
reference_sentence_exulting = ["I won!", "I am very happy!"]
reference_sentence_link = ["they are a couple", "they are lovers    ", "they are similar", "they understand each other"]
reference_sentence_apologize = ["excuse me", "please forgive me", "I am sorry"]
reference_sentence_attention = ["pay attention", "listen me", "look at here"]
reference_sentence_beg = ["please listen me", "I beg you"]
reference_sentence_careless = ["I don't care", "It's not my problem"]
reference_sentence_impossible = ["How is it possible?", "It's impossible"]
reference_sentence_walk= ["I like to walk", "do you like to walk?", "I like trekking", "I want to do some trekking",
                           "I like hiking", "I often go out for a walk"]
reference_sentence_run = ["I like to run","do you like to run?", "where you will run?", "I often run", "I want to run later"] #go,go
reference_sentence_thin = ["that person is thin", "I'm thin", "that object is thin"]
reference_sentence_later = ["let's meet later", "see you tomorrow", "next week", "next month", "next year", "do it later"]
reference_sentence_lot = ["it is very", "there is a lot", "it is too much"]
reference_sentence_time = ["what time is it?", "can you tell me the time", "It's late", "you are late"]
reference_sentence_clap = ["you did very well!", "compliments!", "congratulations!"]
reference_sentence_ita = ["What do you want?", "what are you saying?"]


all_refs = {"r_greet":reference_sentence_greet, "r_faster":reference_sentence_faster, "r_goaway":reference_sentence_goaway,
           "r_cigarette":reference_sentence_cigarette, "r_idk":reference_sentence_idk, "r_no":reference_sentence_no,
           "r_yes":reference_sentence_yes, "r_deitic":reference_sentence_deitic,"r_run":reference_sentence_run,
           "r_stop":reference_sentence_stop, "r_adaptor":reference_sentence_adaptor, "r_telephone":reference_sentence_telephone,
           "r_come":reference_sentence_come, "r_drink":reference_sentence_drink, "r_exulting":reference_sentence_exulting,
            "r_link":reference_sentence_link, "r_apologize":reference_sentence_apologize, "r_attention":reference_sentence_attention,
            "r_beg":reference_sentence_beg, "r_careless":reference_sentence_careless, "r_impossible":reference_sentence_impossible,
            "r_walk":reference_sentence_walk,"r_thin":reference_sentence_thin, "r_later":reference_sentence_later,
            "r_lot":reference_sentence_lot, "r_time":reference_sentence_time, "r_clap":reference_sentence_clap,"r_ita":reference_sentence_ita}

topic_gesture_description = {"Greeting":("r_greet", "the gesture that you use when you greet people"), "Go faster":("r_faster", "the gesture that you use when you want someone to " \
                        "talk faster (for example by keeping one or two hands ahead, palms facing you and wrist rotating them recursively)"), "Go away":("r_goaway", "the " \
                        "gesture that you use when you want someone to go away from you (for example by keeping one or two hands ahead, palms facing you and by moving the hand back and forth" \
                        "starting from the palm facing down)"), "Cigarette":("r_cigarette","the gesture that you do when you want to show smoking intentions (bring two fingers" \
                        "close to the mouth)"), "I don't know":("r_idk", "moving the shoulders up and down or move both the arms in front of you with palms facing up, used when you want to" \
                        "to show that you don't know something"), "No":("r_no", "moving the head or a finger left and right when you want to say no"), "Yes":("r_yes","moving the head up and " \
                        "down when you are agreeing for something or you want to say yes"), "Pointing something":("r_deitic","the gesture that you use for indicating something (with head, finger, etc.)"),
                        "Pointing yourself":("r_adaptor","the gesture that you use to point yourself or a part of your body"), "Run": ("r_run","moving arms aback and forth while" \
                        "maintaining an angle of 90° with the elbow to simulate a running movement"), "Walk":("r_walk","moving the index and the middle fingers back and forth to simulate" \
                        "a walking movement"), "Stop":("r_stop","gesture that you make with arms to tell someone that he as to slow down or stop to do something (for example with arms ahead," \
                        "back of the hand facing you and while moving back and forth)"), "Call":("r_telephone", "the gesture that you make to simulate a phone call, for instance by keeping" \
                        "up the thumb and pinky fingers of one hand and by bringing the hand close to an ear"), "Come to me":("r_come", "the gesture that you make to someone to tell him/her" \
                        "to get close, for instance by keeping an arm straight, palm facing down and my moving banck and forth the fingers"), "Drinking":("r_drink", "the gesture that you make" \
                        "to simulate someone drinking, for instance by keeping up the thumb and pinky fingers of one hand and by bringing the hand close to the mouth"), "Exulting":("r_exulting",
                        "the gesture that you make when you are exulting (for example by putting both your arms up and by putting your keeping your hands in the shape of a fist)"), "They are related": \
                        ("r_link","the gesture that you make to tell that 2 people or objects are related (for example by making 2 fists and by bringing them close to each other)"), "Apologize":("r_apologize",
                        "the gesture that you make when you ask someone to forgive you, for instance by tilting the upper body or by keeping arms up, palms facing up and elbows bent so that they" \
                        "form a 90° angle"), "Attention please":("r_attention","the gesture that you make when you ask attention to a person or group of people, for instance by keeping up the" \
                        " index while in front of you"), "Begging you":("r_beg","the gesture that you make when you are begging someone, for instance by keeping arms in a pray position"), "I don't care": \
                        ("r_careless","the gesture that you make when you are not interested in something (for example by bringing the back of one hand under the chin and by moving it back and forth)"), \
                        "It's impossible": ("r_impossible","the gesture that you make when you don't believe that something is possible, for instance by bending one elbow in a 90° " \
                        "angle, palm of that arm facing up, and by keeping up only the index and the thumb fingers"), "It's thin": ("r_thin","keeping only the pinkie of one hand up while maintaining the other" \
                        "indexes down and the elbow bent to say that something is thin"), "I'll do it later": ("r_later", "keeping a hand in front of you with only the index up to communicate to someone" \
                        "that somthing will happen later and not now"), "It's a lot":("r_lot","the gesture that you make when you think that something is a lot, for example by keeping a hand in front of you" \
                        "with palm facing you, and by moving it up and down"), "It's late":("r_time", "indicating the wrist with an index to say that it's late for something"), "Congratulations": ("r_clap",
                        "clapping with hands to congratulate with someone"), "What are you saying?":("r_ita", "keep one hand in front of you, palm facing you, the thumb is over the other fingers and the hand" \
                        "is moving back and forth. Used, especially in italy, to tell someone that what is he saying does not make any sense")}

In [ ]:
### READ AND SAVE A FILE OF SENTENCES CREATED BY CHATGPT

def save_sentences(data, filename="sentences.pkl",path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"): #saves data with a given filename in a path
    #if not os.path.exists(os.path.join(path,filename)):
    print(data)
    pickle.dump(data, open(os.path.join(path,filename), 'wb'))
    #else:
    #    print("the file already exists")

def read_sentences(names=[]): #It reads a text file that contains sentences generated by chat-gpt
                                  #one or more lines with the format name: sentence
    #the person pronounced, while idx is an incremental number that is incremented each time a person talks
    path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\generated_sentences_general_4.txt"
    #data = pickle.load(open(path,'rb'))
    #sentence = data.strip()
    #print(sentence)
    sentences = []
    counter=0
    with open(path, "r") as f:
        while line := f.readline():
            if line == '\n':
                continue
            else:
                #sentence_splits = line.split('.')[1:] #remove the number of the sentence
                #sentence = '.'.join(sentence_splits)
                sentence = line
                print(sentence,"\n")
                sentences.append(sentence)
                counter += 1
        #lines = [line.rstrip() for line in f]

        #data = f.read()
        #sentences = data.split('\n')
        #print(len(sentences))
        #print(data)
        #for line in f:
            #sentence = line.strip()
            #print(sentence)
           # counter +=1
            #sentence = sentence.split('.')[1:] # sentences are linke " 1. sentence \n ", we take just the sentence part
            #print(sentence)
            #sentences.append(sentence)
    print(counter)

    return sentences

all_sentences =  read_sentences()
'''
counter = 0
for i in range(len(all_sentences)):
    for j in range(len(all_sentences)):
        if all_sentences[i] == all_sentences[j]:
            #print(all_sentences[i])
            counter+=1
print(counter)
'''
#print(all_sentences, len(all_sentences), "\n", all_sentences[1])
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
#print(len(all_sentences), all_sentences)
save_sentences(all_sentences,filename="sentences_gpt_2.pkl",path=path)
#filename = "sentences_gpt.pkl"
#data = pickle.load(open(os.path.join(path,filename),'rb'))
#print(data, len(data))

In [ ]:
### READ AND SAVE A FILE OF SUBTITLES ABOUT A CONVERSATION OF TWO PEOPLE

def save_sentences(data, filename="sentences.pkl",path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"): #saves data with a given filename in a path
    if not os.path.exists(os.path.join(path,filename)):
        print(data)
        pickle.dump(data, open(os.path.join(path,filename), 'wb'))
    else:
        print("the file already exists")

def read_conversations(names=[]): #It reads a text file that contains subtitles referred to a conversation between 2 or more people. It expects that each time a person talks, we have
                                  #one or more lines with the format name: sentence
    #the person pronounced, while idx is an incremental number that is incremented each time a person talks
    path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\[English] 1.5 HOUR English Conversation Lesson [DownSub.com].txt"
    names = ['Vanessa', 'Dan']
    sentences = dict((name,[]) for name in names)
    #this dictionary has names of actors as keys, as value they have a tuple (sentence, idx) where sentence is the sentence that

    with open(path, "r", encoding='utf-8') as f:
        subs = [line.rstrip() for line in f if line.rstrip() != '']  #removes \n and "\" before apostroph
        i = 0
        turn = 1 # counter that counts the turns ( turn = turn + 1 each time a person finish to talk and the other starts)
        while i<len(subs):
            actual_name_list = [name for name in names if name+":" in subs[i]] #is the person that is currently talking
            if len(actual_name_list) == 1: # A person is talking. if the person is the same in the next rows, the sentence is composed by all the the words in these rows put together
                name = actual_name_list[0]
                sentence = subs[i][len(actual_name_list[0])+1:]    #remove "Name:"
                #print(i,len(subs)-1)
                if i < len(subs)-1: #avoid exceeding the list of subtitles
                    i = i + 1
                    next_name_list = [name for name in names if name+":" in subs[i]]
                    while len(next_name_list) == 0 and i != len(subs)-1 or (len(next_name_list)==1 and next_name_list[0] == name):  #same person is talking
                        sentence = sentence + ' ' + subs[i]
                        i = i + 1
                        next_name_list = [name for name in names if name+":" in subs[i]]
                        #print( len(next_name_list), next_name_list, name, i != len(subs), i , len(subs)-1)
                    #print("OUT!")

                sentences[name].append((sentence,turn))
                turn = turn + 1
                #print(i,len(subs)-1)
            else: # If we have for any reason more names or no names at the start of the sentence, we skip it
                i = i+1
    return sentences

all_sentences = read_conversations()
print(all_sentences)
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
save_sentences(all_sentences,path=path)
filename = "sentences.pkl"
data = pickle.load(open(os.path.join(path,filename),'rb'))
print(data)

In [ ]:
### CREATE A 300 SENTENCES VERSION OF FINAL RESULT
import pickle, random

path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
#filename_best = "new_all_processed_sentences_best.pkl" #it contains all the possible results; a results is the best words match for each topic for each possible group of words inside the sentence, for all the sentences
filename_1 = "all_similarities.pkl"
filename_2 = "all_similarities_old.pkl"
res_1 = "all_similarities_300.pkl"
res_2 = "all_similarities_300_old.pkl"
new_sentences_file_1 = "sentences_gpt_300.pkl"
new_sentences_file_2 = "sentences_gpt_300_old.pkl"
randomized_file_1 = "sentences_gpt_randomized.pkl"
randomized_file_2 = "sentences_gpt_randomized_old.pkl"
marco_file = "Marco_Limone_labelled_sentences_final.pkl"
final_res = "labelled_sentences_final_old.pkl"
mixed_results = "all_similarities_600.pkl"

if(os.path.exists(os.path.join(path,new_sentences_file_2))):    #load data already processed
    sentences = pickle.load(open(os.path.join(path,new_sentences_file_2),'rb'))
else:
    print("invalid sentences path")

if(os.path.exists(os.path.join(path,marco_file))):    #load data already processed
    marco_sentences = pickle.load(open(os.path.join(path,marco_file),'rb'))
else:
    print("invalid moving window labels path")

if(os.path.exists(os.path.join(path,res_2))):    #load data already processed
    labels_old = pickle.load(open(os.path.join(path,res_2),'rb'))
    print("loaded old similarities")
else:
    print("invalid moving window labels path")

if(os.path.exists(os.path.join(path,res_1))):    #load data already processed
    labels = pickle.load(open(os.path.join(path,res_1),'rb'))
    print("loaded similarities new")
else:
    print("invalid moving window labels path")

if(os.path.exists(os.path.join(path,randomized_file_2))):    #load data already processed
    sentences_randomized = pickle.load(open(os.path.join(path,randomized_file_2),'rb'))
else:
    print("invalid moving window labels path")

unique_sentences = list(set(sentences_randomized))
print(sentences[0])

#all_sentences_labels = labels_old + labels
#print(len(all_sentences_labels))
#pickle.dump(all_sentences_labels, open(os.path.join(path, mixed_results),'wb'))
#print(unique_sentences,len(unique_sentences))


'''  ##### take 300 random sentences
indexes = [i for i in range(len(labels))]
shuffled_indexes = random.shuffle(indexes)
new_res_list = list([labels[i] for i in indexes])[:300]
all_sentences = []
for i in range(len(new_res_list)):
    all_sentences.append(new_res_list[i]['whole sentence'])

pickle.dump(all_sentences, open(os.path.join(path,new_sentences_file),'wb'))
'''
    #print(new_res_list[i]['whole sentence'])
#pickle.dump(new_res_list , open(os.path.join(path,res_2),'wb'))

#print(all_sentences)
#print("written")

#print(len(labels), len(new_res_list))




#### take only a subset of sentences
'''
final_sentences = []
#print(len(marco_sentences))
for j,element in enumerate(marco_sentences):
    sentence = element['sentence']
    for label in labels_old:
        if label['whole sentence'] == sentence:
            final_sentences.append(element)
        else:
            continue
#print(len(final_sentences))
marco_sentences = final_sentences[30:47] + final_sentences[:13]
#print(marco_sentences)
#print(len(marco_sentences))
for j,element in enumerate(marco_sentences):
    element['sentence_index'] = j + 49*30
#print(len(marco_sentences),marco_sentences)
s = []
for element in marco_sentences:
    s.append(element['sentence'])
print(len(s),len(set(s)))
pickle.dump(marco_sentences, open(os.path.join(path, marco_file),'wb'))
pickle.dump(marco_sentences, open(os.path.join(path, final_res),'wb'))
'''
'''
#### count how many sentences of one file are the same of another file
counter = 0
for label in labels:
    result_sentence = label['whole sentence']
    for sentence in sentences:
        if sentence == result_sentence:
            #print(sentence, result_sentence)
            counter += 1
            break
print("COUNTER:",counter)
'''


In [ ]:
### NEW REFERENCE SENTENCES FOR EACH GESTURE AND DESCRIPTION OF GESTURES. RUN THIS CELL ONE TIME BEFORE ANY COMPUTATION

###STANDARD
'''
reference_sentence_greet = ["hello", "goodbye"] #['greeting']
reference_sentence_idk = ["I don't know"]#["I don't know"]
reference_sentence_no = ["no"]
reference_sentence_yes = ["yes"] #["yes"] #right
reference_sentence_deictic = ["that object", "this object", "this place", "that place"]
reference_sentence_stop = ["stop"] #["stop"]
reference_sentence_run = ["running"] #["running"] #go,go
reference_sentence_apologize = ["I apologize"] #["I apologize"]
reference_sentence_beg = ["I beg you"] #["I beg you"]
reference_sentence_time = ["It's late"] #["it's late"]
reference_sentence_clap = ["I praise you"] #["I praise you"]
reference_sentence_exulting = ["I am exulting"] #["I am exulting"]
'''

  ###ALICE
'''
reference_sentence_greet = ["See you soon","How are you? I haven’t seen you for a long time","Hi! Are you here too?","Have a good evening!" ]#['greeting']  #["hello", "goodbye"]
reference_sentence_idk = ["I don’t know where is it","I have no idea","I don’t know if I like it","I don’t remember what time is it"]#["I don't know"]
reference_sentence_no = ["No, I’m not doing all myself","We can’t keep spending money on useless items","I won’t walk all day","No, I won’t go to the club"]
reference_sentence_yes = ["All right","Yes","Okay","Yes I’m really happy"] #["yes"] #right
reference_sentence_deictic = ["We gotta get to the top of that mountain over there","Would you hand me that pen?",
                              "Take the flour in that cupboard","Look at the sea!"] #["that object", "this object", "this place", "that place"]
reference_sentence_stop = ["Stop, I don’t understand anything","Let’s just calm down","Let’s stop and take a breath","Stop, I’m exhausted."]#["stop"]
reference_sentence_run = ["The train is leaving", "I have to run","To lose weight I have to go for a run","I ran all over the shops"]#["running"] #go,go
reference_sentence_apologize = ["I’m so sorry","I didn’t think you were hurt","I’m sorry I made you wait", "I'm sorry, I can't now."]#["I apologize"]
reference_sentence_beg = ["Will you please help me with the computer exam?","Would you please take me to the party?",
                          "Would you please buy me an icecream?", "Would you like to go to Paris?"] #["I beg you"]
reference_sentence_time = ["can you hurry up and get dressed?","The bus leaves in ten minutes","Would you hurry up?","I stay there one hour"] #["it's late""]
reference_sentence_clap = ["You were good to take such a high grade","The competition was a success, congratulations really!",
                            "The cake is delicious","I’m so glad you could buy a house"] #["I praise you"]
reference_sentence_exulting = ["What a great goal we made!","I finally graduated!","I win first prize!","I finally got home"]#["I am exulting"]
'''
### ALICE RANDOM

reference_sentence_greet = ["Hi! Are you here too?"]
reference_sentence_idk = ["I don’t know if I like it"]
reference_sentence_no = ["No, I’m not doing all myself"]
reference_sentence_yes = ["Okay"]
reference_sentence_deictic = ["Take the flour in that cupboard"]
reference_sentence_stop = ["Stop, I’m exhausted."]#["stop"]
reference_sentence_run = ["I ran all over the shops"]#["running"] #go,go
reference_sentence_apologize = ["I'm sorry, I can't now."]#["I apologize"]
reference_sentence_beg = ["Will you please help me with the computer exam?"] #["I beg you"]
reference_sentence_time = ["Would you hurry up?"] #["it's late""]
reference_sentence_clap = ["congratulations really!"] #["I praise you"]
reference_sentence_exulting = ["I finally graduated!"] #["I am exulting"]

  ###CARMINE NUOVE
'''
reference_sentence_greet = ["Goodbye to everyone and thank you for listening.", "Good morning! So good to see you again!",
                            "Hey! Hello! You've finally arrived.", "Hi everyone, I'm Carmine."]#['greeting']  #["hello", "goodbye"]
reference_sentence_idk = ["I really have no idea where John is.","I don't know what I'm going to do tomorrow.",
                          "I'm speechless, I don't know what to say.", "Don’t ask me, I'm not an expert."] #["I don't know"]
reference_sentence_no = ["What you have done is not good.","I don't want anything more.","I can't believe what I saw.","I don't have what you ask for."]
reference_sentence_yes = ["Yes thank you, I will gladly accept!","It's exactly as you say.",
                          "If you insist I will come to the party.","You have answered my question correctly."] #["yes"] #right
reference_sentence_deictic = ["The building is exactly behind you.","Look at what's going on there!",
                              "Could you please get that book for me?","Go there immediately and don't move."] #["that object", "this object", "this place", "that place"]
reference_sentence_stop = ["Wait, you're talking too fast.","Stop where you are, the floor is slippery.",
                           "Quiet, listen, there's a noise upstairs.","I don't want to know about it."]#["stop"]
reference_sentence_run = ["Do you want to go for a run?", "I just got back from a good run.","I finished second in the cross-country race.",
                          "I hope to win the 100-meter flat race."] #["running"] #go,go
reference_sentence_apologize = ["It's all my fault.","I'm sorry, I guess I misunderstood.","I'm terribly sorry for the delay.",
                                "Ehy, it's okay, don't fret."]#["I apologize"]
reference_sentence_beg = ["Please, I absolutely need you now.","Let's hope for the best.",
                          "Please stop it!", "God bless you!"] #["I beg you"]
reference_sentence_time = ["You’re late, have you seen what time it is?","Hurry up, they are expecting us.",
                           "You only have 2 minutes left for your presentation.","It's time to get back to work."] #["it's late""]
reference_sentence_clap = ["Congratulations, really an interesting speech.","Best wishes to the bride and groom!",
                            "You did a really good job.","Bravo! Encore!"] #["I praise you"]
reference_sentence_exulting = ["Yay! The concert begins!","I finally made it!","My team has won the game!","I didn't expect you to come!"] #["I am exulting"]
'''
### CARMINE NUOVE RANDOM
'''
reference_sentence_greet = ["Hi everyone, I'm Carmine."]#['greeting']  #["hello", "goodbye"]
reference_sentence_idk = ["I'm speechless, I don't know what to say."] #["I don't know"]
reference_sentence_no = ["I can't believe what I saw."]
reference_sentence_yes = ["Yes thank you, I will gladly accept!"] #["yes"] #right
reference_sentence_deictic = ["Go there immediately and don't move."] #["that object", "this object", "this place", "that place"]
reference_sentence_stop = ["I don't want to know about it."]#["stop"]
reference_sentence_run = ["I finished second in the cross-country race."] #["running"] #go,go
reference_sentence_apologize = ["It's all my fault."]#["I apologize"]
reference_sentence_beg = ["God bless you!"] #["I beg you"]
reference_sentence_time = ["It's time to get back to work."] #["it's late""]
reference_sentence_clap = ["Best wishes to the bride and groom!"] #["I praise you"]
reference_sentence_exulting = ["I finally made it!"] #["I am exulting"]
'''
 ###LUCREZIA
'''
reference_sentence_greet = ["Hello!","See you soon!","Goodbye!","Come back soon!"] #['greeting]  #["hello", "goodbye"]
reference_sentence_idk = ["I have no idea...","I don't know what to say...","There's nothing I can do...","Do as you please!"]#["I don't know"]
reference_sentence_no = ["It wasn't me!","I don't know how it could've happened","I don't like it.","I don't feel like it!"]
reference_sentence_yes = ["Okay, let's go!","I really like it.","You're right","Got it, clear."] #["yes"] #right
reference_sentence_deictic = ["Who is that?","Go straight until there.", "Can you take the object on the table?","Up there is the Big Dipper constellation."] #["that object", "this object", "this place", "that place"]
reference_sentence_stop = ["Everyone stop!","Watch out, the floor is wet!","Don't move.","I'm stopping you right now: I've already done it."]#["stop"]
reference_sentence_run = ["Shall we go for a run?","Are you ready to run?","I just finished a nice treadmill run.","You should get some exercise!"]#["running"] #go,go
reference_sentence_apologize = ["I won't do it anymore!","I didn't mean to offend you.","Don't take it personally!", "It's not my fault."]#["I apologize"]
reference_sentence_beg = ["Will you come with me?","Come on, please!","Do it for me!", "Let's hope everything goes well!"] #["I beg you"]
reference_sentence_time = ["I've been waiting for you forever!","So, are you ready? We need to go!","Do you see what time it is?","How much longer until we're done?"] #["it's late""]
reference_sentence_clap = ["Well done!","I really enjoyed it!","I couldn't have done better myself!","I would've never expected that from you!"] #["I praise you"]
reference_sentence_exulting = ["We won!","Let's go celebrate!","I did it!","Woohoo, it works!"] #["I am exulting"]
'''
### LUCREZIA RANDOM
'''
reference_sentence_greet = ["Hello!"]
reference_sentence_idk = ["Do as you please!"]#["I don't know"]
reference_sentence_no = ["I don't know how it could've happened"]
reference_sentence_yes = ["Okay, let's go!"] #["yes"] #right
reference_sentence_deictic = ["Up there is the Big Dipper constellation."] #["that object", "this object", "this place", "that place"]
reference_sentence_stop = ["I'm stopping you right now: I've already done it."]#["stop"]
reference_sentence_run = ["You should get some exercise!"]#["running"] #go,go
reference_sentence_apologize = ["I didn't mean to offend you."]#["I apologize"]
reference_sentence_beg = ["Will you come with me?"] #["I beg you"]
reference_sentence_time = ["I've been waiting for you forever!"] #["it's late""]
reference_sentence_clap = ["I would've never expected that from you!"] #["I praise you"]
reference_sentence_exulting = ["We won!"] #["I am exulting"]
'''
### ANTONIO
'''
reference_sentence_greet = ['hello','how are you','goodbye','hey there']
reference_sentence_idk = ["I don't know",'maybe','who knows',"I can't help you"]
reference_sentence_no = ['no','never','I did not','not at all']  #not at all. in LARGE, not at all  in BASE
reference_sentence_yes = ['yes','sure','of course','with pleasure']
reference_sentence_deictic = ['look at','there','that place','that object']
reference_sentence_stop = ['stop','do not move','give me a break','stay away']
reference_sentence_run = ['shall we go',"let's go",'have a walk','have a run']
reference_sentence_apologize = ['sorry','I beg your pardon','apologizes',"don't be mad"]
reference_sentence_beg = ['please','do it','do me a favour','help me']
reference_sentence_time = ["it's late","it's time to",'time is running','we have to go']
reference_sentence_clap = ['congratulations','well done','bravo!','good job!'] #good job. in LARGE, good job! in BASE
reference_sentence_exulting = ['I did it!','hooray!','I won!','I am the best']
'''
'''
### ANTONIO RANDOM
reference_sentence_greet = ['hello']
reference_sentence_idk = ["I don't know"]
reference_sentence_no = ['no']
reference_sentence_yes = ['yes']
reference_sentence_deictic = ['that object']
reference_sentence_stop = ['do not move']
reference_sentence_run = ["let's go"]
reference_sentence_apologize = ['apologizes']
reference_sentence_beg = ['please']
reference_sentence_time = ['time is running']
reference_sentence_clap = ['bravo!']
reference_sentence_exulting = ['I did it!']
'''
'''
Sentences can be related to the topics represented by the words: greeting someone, praise someone,
                                        telling you don't know something, showing an object or a place,
                                        showing agreement or disagreement, begging someone, telling or asking if it's late, telling to stop or slow down,  telling or asking for apologize,
                                        telling a run story or asking someone to run with you, exulting for something."
                                        '''

all_refs = {"r_greet":reference_sentence_greet, "r_clap":reference_sentence_clap,  "r_idk":reference_sentence_idk, "r_no":reference_sentence_no,
           "r_yes":reference_sentence_yes,"r_stop":reference_sentence_stop, "r_beg":reference_sentence_beg,"r_time":reference_sentence_time,
           "r_apologize":reference_sentence_apologize, "r_deitic":reference_sentence_deictic,"r_exulting":reference_sentence_exulting,"r_run":reference_sentence_run}

topic_gesture_description = {"Greeting":("r_greet", "the gesture that you use when you greet people"), \
                             "Run": ("r_run","moving arms aback and forth while maintaining an angle of 90° with the elbow to simulate a running movement"),
                            "I apologize":("r_apologize", "the gesture that you make when you ask someone to forgive you, for instance by tilting the upper body or by keeping arms up, palms facing up and elbows bent so that they form a 90° angle"),\
                             "I am exulting":("r_exulting", "the gesture that you make when you are exulting (for example by putting both your arms up and by putting your keeping your hands in the shape of a fist)"), \
                             "I praise you": ("r_clap","clapping with hands to congratulate with someone"), \
                             "No":("r_no", "moving the head or a finger left and right when you want to say no"), \
                             "Yes":("r_yes","moving the head up and down when you are agreeing for something or you want to say yes"), \
                             "Indicate a place or object":("r_deitic","the gesture that you use for indicating a place or object (with head, finger, etc.)"),
                             "Stop":("r_stop","gesture that you make with arms to tell someone that he as to slow down or stop to do something (for example with arms ahead)"), \
                             "I don't know":("r_idk", "moving the shoulders up and down or move both the arms in front of you with palms facing up, used when you want to to show that you don't know something"), \
                             "I beg you":("r_beg","the gesture that you make when you are begging someone, for instance by keeping arms in a pray position"), \
                             "C'mon, it's late":("r_time", "indicating the wrist with an index to say that it's late for something")}

In [ ]:
### SENTENCE SIMILARITY COMPUTATION

class Subs_classification:

    # filename contains the subtitles filepath
    # refs contains the dictionary with all the reference sentences
    # model variable contains the model that will be used for the semantic similarity
    # winsize represents the number of words inside the sentence that will be used as comparison
        # with the reference sentences. Not necessary in the "fullstop_classification" mode
    # stride indicates the amount of words skipped to traslate the window. Not necessary in "fullstop_regression" mode
    # thresold represents the value over which we consider the prediction of the model "valid", i.e. when
        # reference sentence and sentence are similar enough
    # mode is a string that represents the modality that we want to use as comparison:
        # "fullstop_regression" compares a reference sentence with a sentence between two full stop
        # "window_regression" compares a reference sentence with a sentence made up by the words inside the window
        # "mixed_rgression" compares a reference sentence with a sentence made up by the words inside the window but
            # it also find relations between windows inside a sentence between two full stops. The last window does not take
            # the words after the full stop

    def __init__(self, filename = '', refs = {}, path = '', result_name = '',
                 model = CrossEncoder('cross-encoder/stsb-roberta-base'),
                 winsize = 5, stride = 2, threshold = 0.3, mode = "fullstop_regression"):

        self.filepath = filename
        self.subs = ''           #self.read_file(filename)
        self.refs = refs
        self.winsize = winsize
        self.stride = stride
        self.threshold = threshold
        self.mode = mode
        self.result_name = result_name
        self.path = path

    # used to read the "filename" file. It reads a row at time and then puts all the rows together
    def read_file(self, filename):

        with open(subtitles_path,"r",encoding='utf-8') as f:
            subs = [line.rstrip() for line in f if line.rstrip()!='']  #removes \n
            subs_final = ''
            #subs_final = [subs_final + s + ' ' for s in subs]
            for el in subs:
                subs_final = subs_final + el + ' '  #puts all the subs together
            self.subs = subs_final
            return self.subs


    def sub_classifier(self, mode='', method='', sentences = '', type = 'conversation'):

        if mode!='':
            self.mode = mode
        if method!='':
            self.method = method

        if self.mode == "fullstop_regression":
            return self._fullstop_regression(self.method)
        elif self.mode == "window_regression":
            return self._window_regression(self.method)
        elif self.mode == "mixed_regression":
            return self._mixed_regression(self.method, sentences, type)
        else:
            print("Choose a right mode and a right method")


    def _append_results(self, thresholds_dict, value, ref, sentence, start_index):

        thresholds_dict["value"].append(value)
        thresholds_dict["ref"].append(ref)
        thresholds_dict["sentence"].append(sentence)
        thresholds_dict['start_word_indexes'].append(start_index)

        return thresholds_dict


    def _best_threshold(self,local_th, all_th, sentence, start_index):

        if len(local_th["value"])!=0: #then we have at least one reference match
            max_threshold = max(local_th["value"])
            index = local_th["value"].index(max_threshold)
            all_th = self._append_results(all_th, max_threshold,
                                            local_th["ref"][index], sentence, local_th['start_word_indexes'][index])
        #else:
            #all_th = self._append_results(all_th, None,  None, sentence)
        local_th = {"value":[], "ref":[], "sentence":[], 'start_word_indexes':[]} #I don't need the local thresholds anymore


        return local_th, all_th

    def _window_regression(self, method = "best_threshold"):

        subs = self.read_file(self.filepath)    #all the subtitles
        splits = subs.split()[0:200]
        #best_th = {"value":[], "ref":[], "sentence":[]}
        all_best_thresholds = {"value":[], "ref":[], "sentence":[]}
        local_thresholds = {"value":[], "ref":[], "sentence":[]}
        all_local_th = {}

        for i in range(int(len(splits)/self.stride)):
            sentence = " ".join(splits[i*self.stride : i*self.stride + self.winsize]) #sentence that will be compared with all the ref sentences
            for key in self.refs.keys():    #each key represents a topic related to a specific gesture, each element in the key is a sentence that can generate that gesture
                for ref_sentence in self.refs[key]:   #for each reference, do a comparison with a sentence inside subtitles. If they are similar (>threshold) then append the result to local_thresholds
                    res = model.predict([sentence,ref_sentence])
                    if res > self.threshold:
                        local_thresholds = self._append_results(local_thresholds, res, ref_sentence, sentence)

                if method == "best_threshold_for_each_topic":     #for each topic, find the reference that is the most similar to the sentence (if there is anyone) and put results in all_best_thresholds
                    local_thresholds, all_best_thresholds = self._best_threshold(local_thresholds, all_best_thresholds, sentence)

            if method == "best_threshold":  #for each window, find the best reference of the best topic that is the most similar to the sentence (if there is anyone
                local_thresholds, all_best_thresholds = self._best_threshold(local_thresholds, all_best_thresholds, sentence)

        if method == "best_threshold" or method == "best_threshold_for_each_topic":
            return all_best_thresholds
        elif method == "all_thresholds":
            return local_thresholds


    def _fullstop_regression(self, method = "best_threshold"):

        subs = self.read_file(self.filepath)
        splits = subs.split(".")[0:200]
        all_best_thresholds = {"value":[], "ref":[], "sentence":[]}
        local_thresholds = {"value":[], "ref":[], "sentence":[]}

        for sentence in splits:
            for key in self.refs.keys():
                for ref_sentence in self.refs[key]:
                    res = model.predict([sentence,ref_sentence])
                    if res > self.threshold:
                        local_thresholds = self._append_results(local_thresholds, res, ref_sentence, sentence)

                if method == "best_threshold_for_each_topic":     #for each topic, find the reference that is the most similar to the sentence (if there is anyone) and put results in all_best_thresholds
                    local_thresholds, all_best_thresholds = self._best_threshold(local_thresholds, all_best_thresholds, sentence)

            if method == "best_threshold":  #for each window, find the best reference of the best topic that is the most similar to the sentence (if there is anyone
                local_thresholds, all_best_thresholds = self._best_threshold(local_thresholds, all_best_thresholds, sentence)

        if method == "best_threshold" or method == "best_threshold_for_each_topic":
            return all_best_thresholds
        elif method == "all_thresholds":
            return local_thresholds


    def _mixed_regression(self, method = "best_threshold", sentences='', type = 'sentences'):
        #TODO save the starting index of the matched words to avoid later correction - DONE
        #TODO (optional) add full stop separation in the conversation
        #subs = self.read_file(self.filepath)
        subs = sentences
        counter = 1
        #all_local_all_sentences=[]  #it contains all the local results for all possible windows for all the sentences
        all_processed_sentences = []  #it contains all the local results for all possible windows for all the sentences
        #print(subs)
        if type == 'conversation':      # In this case sentences are of type (speaker,sentence,turn), otherwise we have only a list of sentences
            conv_subs=[]
            for key in list(subs.keys()):
                speaker = (key,)
                tuples = [speaker + element for element in subs[key]]
                conv_subs.append(tuples)
                #subs[key] = [ for (sentence,turn) in subs[key]]
            #subs = conv_subs[0][0:10] + conv_subs[1][0:10] #take 10 sentences for 2 speakers
            subs = conv_subs[1][:100] + conv_subs[0][:100]
            #print(subs)


        #print(subs)
        #subs = subs[0:1] + subs[2:3] #+ subs[8:13]
        sentences_processed = 0
        all_topics_labelling_path = os.path.join(path,result_name)#"all_processed_sentences_best.pkl"
        #filename_best = "all_processed_sentences_best.pkl"
        #filename_local = "all_processed_sentences_local.pkl"
        window_max_len = 10
        #path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"

        print("loading file with all matches...")
        if(os.path.exists(all_topics_labelling_path)):    #load data already processed
            all_processed_sentences = pickle.load(open(all_topics_labelling_path,'rb'))
            print("File with all matches loaded!")
        else:
            print("No files were loaded")
        #   pickle.dump(all_processed_sentences, open(all_topics_labelling_path,'wb'))

        '''
        if(os.path.exists(os.path.join(path,filename_local))):    #load data already processed
            all_local_all_sentences = pickle.load(open(os.path.join(path,filename_local),'rb'))
        else:
            pickle.dump(all_local_all_sentences, open(os.path.join(path,filename_local),'wb'))
        print("both files loaded")
        '''
        #print(all_best_all_sentences)

        for sentence in subs:

            #all_local_th = {}
            all_topic_res = {}                                   #contains all the possible matches for all the possible winsize
            best_th_topic = {"value":[], "ref":[], "sentence":[],'start_word_indexes':[]}
            local_th_topic = {"value":[], "ref":[], "sentence":[],'start_word_indexes':[]}    #contains all the possible matches for a given winsize
            #all_locals = {"value":[], "ref":[], "sentence":[],'start_word_indexes':[]}  #contains all the possible matches for a given winsize, but differently from local_th, it  does not become empty if method is "best_threshold" or "best_threshold_for_each_topic"
            skip_flag = False
            if type == 'conversation':
                #print(sentence,len(sentence))
                speaker = sentence[0]
                turn = sentence[2]
                sentence = sentence[1]
                for i in range(len(all_processed_sentences)): #check if we already processed the sentence
                    if((sentence,speaker,turn) == all_processed_sentences[i]["whole sentence,speaker,turn"]): #we already have this data
                        skip_flag = True
                        print(f"skipped: {(sentence,speaker,turn)}", "counter",counter)
                        sentences_processed += 1
                        counter += 1
                if skip_flag == True: #if we already processed it, we go to the next sentence, otherwise we start the computations
                        continue
                else:
                    all_topic_res["whole sentence,speaker,turn"] = (sentence,speaker,turn)
                    #all_best_th["whole sentence,speaker,turn"] = (sentence,speaker,turn)
            elif type == 'sentences':
                for i in range(len(all_processed_sentences)):
                    if(sentence == all_processed_sentences[i]["whole sentence"]):
                        skip_flag = True
                        print(f"skipped: {sentence}", "counter",counter)
                        print(all_processed_sentences[i]["processing_time"])
                        sentences_processed += 1
                        counter += 1
                        continue
                if skip_flag == True:
                    continue
                else:
                    all_topic_res["whole sentence"] = sentence
                    #all_best_th["whole sentence"] = sentence
            else:
                print("Undefined element")
                return
            start_time = time.perf_counter()
            window_size = self.winsize
            sentence_splits = sentence.split()
            while window_size <= len(sentence_splits) and window_size <= window_max_len:
                #print(window_size)
                #print(self.winsize)
                #print(f"HERE:{sentence_splits}")
                #print(len(sentence_splits))
                n_windows = int((len(sentence_splits) - window_size + 1) / self.stride)
                #print(n_windows)
                for i in range(n_windows):  #use the window inside the sentence
                    sub_sentence = " ".join(sentence_splits[i*self.stride : i*self.stride + window_size]) #sentence that will be compared with all the ref sentences
                    start_index = i * self.stride
                    for key in self.refs.keys():    #for all types of references
                        for ref_sentence in self.refs[key]: #for all references
                            #print(self.winsize, sub_sentence)
                            res = model.predict([sub_sentence,ref_sentence])
                            if res > self.threshold:    #then we have a match
                                local_th_topic = self._append_results(local_th_topic, res, ref_sentence, sub_sentence, start_index) #put the result in local_th
                                #all_locals = self._append_results(all_locals, res, ref_sentence, sub_sentence) #same operation but this list do not become empty if method is "best_threshold_for_each_topic" or "best_threshold". Used for saving purposes

                        if method == "best_threshold_for_each_topic":     #for each topic, find the reference that is the most similar to the sentence (if there is anyone) and best_th_topic
                            local_th_topic, best_th_topic = self._best_threshold(local_th_topic, best_th_topic, sub_sentence, start_index)


                    if method == "best_threshold":  #for each window, find the best reference of the best topic that is the most similar to the sentence (if    there is anyone)
                        local_th_topic, best_th_topic = self._best_threshold(local_th_topic, best_th_topic, sub_sentence, start_index)
                        #print(best_th)

                #all_local_th[f"window {window_size}"] = all_locals  #save results for each window size
                all_topic_res[f"window {window_size}"] = best_th_topic
                #best_th_topic['start_word_indexes'] =
                local_th_topic = {"value":[], "ref":[], "sentence":[],'start_word_indexes':[]} #I don't need the local thresholds anymore
                best_th_topic = {"value":[], "ref":[], "sentence":[],'start_word_indexes':[]} #I don't need the best thresholds anymore
                window_size = window_size + 1
            #print(all_best_th)
            #all_local_all_sentences.append(all_local_th)
            #all_best_all_sentences.append(all_best_th)
            end_time = time.perf_counter()
            processing_time = round(end_time-start_time, 4)
            all_topic_res["processing_time"] = processing_time
            print("ELAPSED TIME:",processing_time, "counter",counter)
            counter += 1
            all_processed_sentences.append(all_topic_res)
            sentences_processed = sentences_processed + 1
            print("writing to file...")
            pickle.dump(all_processed_sentences, open(all_topics_labelling_path,'wb'))
            #pickle.dump(all_local_all_sentences, open(os.path.join(path,filename_local),'wb'))
            if type == 'conversation':
                print(f"file written {sentence, speaker, turn}, sentence processed: {sentences_processed}")
            else:
                print(f"file written {sentence}")

            #print(f"processed a sentence:{all_best_th}")
            #print(all_local_th)


        if method == "best_threshold" or method == "best_threshold_for_each_topic":
            return all_processed_sentences
        #elif method == "all_thresholds":
        #    return all_local_all_sentences


th = 0
#filename = "sentences.pkl"
filename = "sentences_gpt_300_old.pkl"
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
result_name = "all_similarities_Alice_random_large_old.pkl"
all_sentences = pickle.load(open(os.path.join(path,filename),'rb'))
#results = pickle.load(open(os.path.join(path,result_name),'rb'))
#pickle.dump(results[:133], open(os.path.join(path,result_name),'wb'))
classifier = Subs_classification(filename = filename, refs = all_refs, path = path, result_name = result_name,
                                  winsize = 1, stride = 1,threshold = th, mode = "mixed_regression")  #compute all the possible results
result = classifier.sub_classifier(mode='', method="best_threshold_for_each_topic", sentences = all_sentences , type = 'sentences')
#print([r['processing_time'] for r in results[-4:]])
#result = pickle.load(open(os.path.join(path,result_name),'rb'))
#print(result[0])
#print(result)

In [ ]:
### COMPUTATION OF BEST MATCHING WINDOW FOR EACH GESTURE

import numpy as np
import os
import pickle
#this time the computation of the average window is slightly different: instead of choosing a threshold as true and then checking all the possible windows in the text
#that respect that theshold to compute the average, this time by starting from a theshold from which we consider a result as valid:
#-we set a window_size of 1 and we check all the results with window 1 that are greater than the threshold
#-we then compute the average and the standard deviation, then avg-std
#-we repeat this operation for all the possible window sizes (currently from 1 to 10)
#-we choose, as avg window for a topic, the window which has the highest avg-std
def best_windows(results, all_refs, new_th = 0, dim_max = 10, type = 'conversation'):

    best_windows = {} #it will contain the best window for each topic, the standard deviation, the number of matches, the best and std of the values of the matches
    all_topics_data = {} #it contains all the window sizes and all the values for each topic so that we can compute sum and std at the end and put the result in the previous dict
    for topic in all_refs: #init the best windows
        best_windows[topic] = {'best_window':0, 'n_matches':0, 'avg_value':0, 'std_value':0}
        all_topics_data[topic]={}
        for i in range(1,dim_max+1):
            all_topics_data[topic][f'window {i}']=[]
    for i,res in enumerate(results):
        #if i==0:
        #    print(res,res.keys())
        if type == 'conversation':
            sentence = res['whole sentence,speaker,turn'][0]
        elif type == 'sentences':
            sentence = res['whole sentence']
        else:
            print("Undefined element")
            return

        #print(res.keys())
        #speaker = res['whole sentence,speaker,turn'][1]
        #turn = res['whole sentence,speaker,turn'][2]
        #print("HERE",int(list(res.keys())[-1][-1]))
        window_dim = int(list(res.keys())[-2].split(' ')[-1]) #the last key of res contains data of the largest window that was used for the sentence.
        #print("WINDOW DIM:",window_dim)
        #window_dim = len(sentence_splits)  TODO: CHECK IF IT IS CORRECT
        #print(window_dim)
        for j in range(1,window_dim+1):
            #print(res[f"window {j}"])
            values = res[f"window {j}"]['value']
            th_indexes = [i for i in range(len(values)) if values[i] > new_th] #returns the indexes of the values that are greater than new_th
            tmp_values = list(map(values.__getitem__,th_indexes)) #take all the values greater than new_th
            tmp_ref = list(map(res[f"window {j}"]['ref'].__getitem__,th_indexes)) #take all the references that returns a value greater than new_th
            #tmp_sentence = list(map(res[f"window {j}"]['sentence'].__getitem__,th_indexes))
            #print(th_indexes)
            for k,value in enumerate(tmp_values): #for each value that is greater than the threshold
                ref = tmp_ref[k]
                #print(ref)
                topic = [key for key in all_refs.keys() for t_ref in all_refs[key] if ref == t_ref][0] #extract the topic associated with reference
                best_windows[topic]['n_matches'] = best_windows[topic]['n_matches'] + 1 #increment the number of matches
                all_topics_data[topic][f'window {j}'].append(value)
    #print(all_topics_data['r_exulting'])

    for topic in all_refs:  #now we can compute the best window for each topic by taking the highest avg-std value
        tmp_best = 0
        tmp_win = 0
        tmp_avg = 0
        tmp_std = 0
        tmp_all_vals = []
        #print(f"The results of the topic {topic} are:",all_topics_data[topic])
        for i in range(1,dim_max+1):
            if len(all_topics_data[topic][f'window {i}'])<=10: #we don't take into account the result if it has less than 10 matches
                continue
            tmp_res = np.mean(np.array(all_topics_data[topic][f'window {i}'])) - np.std(np.array(all_topics_data[topic][f'window {i}']))
            if tmp_res > tmp_best:
                tmp_all_vals = all_topics_data[topic][f'window {i}']
                tmp_best = tmp_res
                tmp_win = i
                tmp_avg = np.mean(np.array(all_topics_data[topic][f'window {i}']))
                tmp_std = np.std(np.array(all_topics_data[topic][f'window {i}']))
        best_windows[topic]['best_window'] = tmp_win
        best_windows[topic]['avg_value'] = tmp_avg
        best_windows[topic]['std_value'] = tmp_std
        #print("ALL VALS:",tmp_all_vals)
            #average_windows[topic]['avg_value'] = round(np.mean(np.array(all_topics_data[topic]['values'])),3)
            #average_windows[topic]['std_value'] = round(np.std(np.array(all_topics_data[topic]['values'])), 3)

    return best_windows



path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
filename_old = 'all_similarities_Alice_base_old.pkl' #'all_similarities_standard_base_old.pkl' #"all_processed_sentences_best.pkl"
filename =  'all_similarities_Alice_base.pkl'
#filename_best = "all_processed_sentences_best.pkl"
#filename_local = "all_processed_sentences_local.pkl"


if(os.path.exists(os.path.join(path,filename))):    #load data already processed
    results = pickle.load(open(os.path.join(path,filename),'rb'))
else:
    print("Invalid path")

if(os.path.exists(os.path.join(path,filename_old))):    #load data already processed
    results_old = pickle.load(open(os.path.join(path,filename_old),'rb'))
else:
    print("Invalid path")

#print(results[0])

b_windows = best_windows(results+results_old, all_refs, new_th = 0.3, type = 'sentences')
for i,el in enumerate(list(b_windows.keys())):
    b_windows[el]['priority'] = np.random.randint(1000) #add a priority to the topic (now they are random, the order has yet to be decided)

filename_res = "best_windows_sentences_Alice_base_0_3.pkl"
print("writing to file... \n \n")
pickle.dump(b_windows , open(os.path.join(path,filename_res),'wb'))

for el in list(b_windows.keys()):
    print(f"{el}: {b_windows[el]}", "\n", "\n")

In [ ]:
### MULTIPLE REFERENCES. USE THIS IF YOU WANT TO ITERATE FOR MULTIPLE REFERENCES, E.G. FOR IOU AND AP COMPUTATION

reference_sentence_greet = {'Alice':["See you soon","How are you? I haven’t seen you for a long time","Hi! Are you here too?","Have a good evening!"],
                         'Alice_random':["Hi! Are you here too?"], 'Carmine_nuove':["Goodbye to everyone and thank you for listening.", "Good morning! So good to see you again!", "Hey! Hello! You've finally arrived.", "Hi everyone, I'm Carmine."],'Carmine_nuove_random':["Hi everyone, I'm Carmine."],
                         'Antonio_base':['hello','how are you','goodbye','hey there'],'Antonio_large':['hello','how are you','goodbye','hey there'],'Antonio_random':['hello'], 'Lucrezia':["Hello!","See you soon!","Goodbye!","Come back soon!"], 'Lucrezia_random':["Hello!"]}

reference_sentence_idk = {'Alice':["I don’t know where is it","I have no idea","I don’t know if I like it","I don’t remember what time is it"],
                         'Alice_random':["I don’t know if I like it"], 'Carmine_nuove':["I really have no idea where John is.","I don't know what I'm going to do tomorrow.","I'm speechless, I don't know what to say.", "Don’t ask me, I'm not an expert."],'Carmine_nuove_random':["I'm speechless, I don't know what to say."], 'Antonio_base':["I don't know",'maybe','who knows',"I can't help you"],'Antonio_large':["I don't know",'maybe','who knows',"I can't help you"],'Antonio_random':["I don't know"], 'Lucrezia':["I have no idea...","I don't know what to say...","There's nothing I can do...","Do as you please!"], 'Lucrezia_random':["Do as you please!"]}

reference_sentence_no = {'Alice':["No, I’m not doing all myself","We can’t keep spending money on useless items","I won’t walk all day","No, I won’t go to the club"],
                         'Alice_random':["No, I’m not doing all myself"], 'Carmine_nuove':["What you have done is not good.","I don't want anything more.","I can't believe what I saw.","I don't have what you ask for."],'Carmine_nuove_random':["I can't believe what I saw."], 'Antonio_base':['no','never','I did not','not at all'],'Antonio_large':['no','never','I did not','not at all.'] ,'Antonio_random':['no'], 'Lucrezia':["It wasn't me!","I don't know how it could've happened","I don't like it.","I don't feel like it!"], 'Lucrezia_random':["I don't know how it could've happened"]}

reference_sentence_yes = {'Alice':["All right","Yes","Okay","Yes I’m really happy"], 'Alice_random':["Okay"],
                          'Carmine_nuove':["Yes thank you, I will gladly accept!","It's exactly as you say.",
                          "If you insist I will come to the party.","You have answered my question correctly."],'Carmine_nuove_random':["Yes thank you, I will gladly accept!"], 'Antonio_base':['yes','sure','of course','with pleasure'],'Antonio_large':['yes','sure','of course','with pleasure'],'Antonio_random':['yes'], 'Lucrezia':["Okay, let's go!","I really like it.","You're right","Got it, clear."], 'Lucrezia_random':["Okay, let's go!"]}

reference_sentence_deictic = {'Alice':["We gotta get to the top of that mountain over there","Would you hand me that pen?",
                              "Take the flour in that cupboard","Look at the sea!"], 'Alice_random':["Take the flour in that cupboard"], 'Carmine_nuove':["The building is exactly behind you.","Look at what's going on there!",
                              "Could you please get that book for me?","Go there immediately and don't move."],'Carmine_nuove_random':["Go there immediately and don't move."], 'Antonio_base':['look at','there','that place','that object'],'Antonio_large':['look at','there','that place','that object'],'Antonio_random':['that object'], 'Lucrezia':["Who is that?","Go straight until there.", "Can you take the object on the table?","Up there is the Big Dipper constellation."], 'Lucrezia_random':["Up there is the Big Dipper constellation."]}

reference_sentence_stop = {'Alice':["Stop, I don’t understand anything","Let’s just calm down","Let’s stop and take a breath","Stop, I’m exhausted."],                                          'Alice_random':["Stop, I’m exhausted."], 'Carmine_nuove':["Wait, you're talking too fast.","Stop where you are, the floor is slippery.",                              "Quiet, listen, there's a noise upstairs.","I don't want to know about it."],'Carmine_nuove_random':["I don't want to know about it."],                              'Antonio_base':['stop','do not move','give me a break','stay away'],'Antonio_large':['stop','do not move','give me a break','stay                                    away'],'Antonio_random':['do not move'], 'Lucrezia':["Everyone stop!","Watch out, the floor is wet!","Don't move.","I'm stopping you right                           now: I've already done it."], 'Lucrezia_random':["I'm stopping you right now: I've already done it."]}

reference_sentence_run = {'Alice':["The train is leaving", "I have to run","To lose weight I have to go for a run","I ran all over the shops"],
                          'Alice_random':["I ran all over the shops"], 'Carmine_nuove':["Do you want to go for a run?", "I just got back from a good run.","I finished second in the cross-country race.","I hope to win the 100-meter flat race."],'Carmine_nuove_random':["I finished second in the cross-country race."], 'Antonio_base':['shall we go',"let's go",'have a walk','have a run'],'Antonio_large':['shall we go',"let's go",'have a walk','have a run'],'Antonio_random':["let's go"], 'Lucrezia':["Shall we go for a run?","Are you ready to run?","I just finished a nice treadmill run.","You should get some exercise!"], 'Lucrezia_random':["You should get some exercise!"]}

reference_sentence_apologize = {'Alice':["I’m so sorry","I didn’t think you were hurt","I’m sorry I made you wait", "I'm sorry, I can't now."],
                         'Alice_random':["I'm sorry, I can't now."], 'Carmine_nuove':["It's all my fault.","I'm sorry, I guess I misunderstood.","I'm terribly sorry for the delay.", "Ehy, it's okay, don't fret."],'Carmine_nuove_random':["It's all my fault."], 'Antonio_base':['sorry','I beg your pardon','apologizes',"don't be mad"],'Antonio_large':['sorry','I beg your pardon','apologizes',"don't be mad"],'Antonio_random':['apologizes'], 'Lucrezia':["I won't do it anymore!","I didn't mean to offend you.","Don't take it personally!", "It's not my fault."], 'Lucrezia_random':["I didn't mean to offend you."]}

reference_sentence_beg = {'Alice':["Will you please help me with the computer exam?","Would you please take me to the party?",
                          "Would you please buy me an icecream?", "Would you like to go to Paris?"],
                         'Alice_random':["Will you please help me with the computer exam?"], 'Carmine_nuove':["Please, I absolutely need you now.","Let's hope for the best.", "Please stop it!", "God bless you!"],'Carmine_nuove_random':["God bless you!"], 'Antonio_base':['please','do it','do me a favour','help me'],'Antonio_large':['please','do it','do me a favour','help me'],'Antonio_random':['please'], 'Lucrezia':["Will you come with me?","Come on, please!","Do it for me!", "Let's hope everything goes well!"], 'Lucrezia_random':["Will you come with me?"]}

reference_sentence_time = {'Alice':["can you hurry up and get dressed?","The bus leaves in ten minutes","Would you hurry up?","I stay there one hour"],
                         'Alice_random':["Would you hurry up?"], 'Carmine_nuove':["You’re late, have you seen what time it is?","Hurry up, they are expecting us.",
                           "You only have 2 minutes left for your presentation.","It's time to get back to work."],'Carmine_nuove_random':["It's time to get back to work."], 'Antonio_base':["it's late","it's time to",'time is running','we have to go'],'Antonio_large':["it's late","it's time to",'time is running','we have to go'],'Antonio_random':['time is running'], 'Lucrezia':["I've been waiting for you forever!","So, are you ready? We need to go!","Do you see what time it is?","How much longer until we're done?"], 'Lucrezia_random':["I've been waiting for you forever!"]}

reference_sentence_clap = {'Alice':["You were good to take such a high grade","The competition was a success, congratulations really!",
                            "The cake is delicious","I’m so glad you could buy a house"], 'Alice_random':["congratulations really!"], 'Carmine_nuove':["Congratulations, really an interesting speech.","Best wishes to the bride and groom!","You did a really good job.","Bravo! Encore!"],'Carmine_nuove_random':["Best wishes to the bride and groom!"], 'Antonio_base':['congratulations','well done','bravo!','good job!'],'Antonio_large':['congratulations','well done','bravo!','good job.'],'Antonio_random':['bravo!'], 'Lucrezia':["Well done!","I really enjoyed it!","I couldn't have done better myself!","I would've never expected that from you!"], 'Lucrezia_random':["I would've never expected that from you!"]}

reference_sentence_exulting = {'Alice':["What a great goal we made!","I finally graduated!","I win first prize!","I finally got home"],
                               'Alice_random':["I finally graduated!"], 'Carmine_nuove':["Yay! The concert begins!","I finally made it!","My team has won the game!","I didn't expect you to come!"],'Carmine_nuove_random':["I finally made it!"], 'Antonio_base':['I did it!','hooray!','I won!','I am the best'],'Antonio_large':['I did it!','hooray!','I won!','I am the best'],'Antonio_random':['I did it!'], 'Lucrezia':["We won!","Let's go celebrate!","I did it!","Woohoo, it works!"], 'Lucrezia_random':["We won!"]}

topic_gesture_description = {"Greeting":("r_greet", "the gesture that you use when you greet people"), \
                             "Run": ("r_run","moving arms aback and forth while maintaining an angle of 90° with the elbow to simulate a running movement"),
                            "I apologize":("r_apologize", "the gesture that you make when you ask someone to forgive you, for instance by tilting the upper body or by keeping arms up, palms facing up and elbows bent so that they form a 90° angle"),\
                             "I am exulting":("r_exulting", "the gesture that you make when you are exulting (for example by putting both your arms up and by putting your keeping your hands in the shape of a fist)"), \
                             "I praise you": ("r_clap","clapping with hands to congratulate with someone"), \
                             "No":("r_no", "moving the head or a finger left and right when you want to say no"), \
                             "Yes":("r_yes","moving the head up and down when you are agreeing for something or you want to say yes"), \
                             "Indicate a place or object":("r_deitic","the gesture that you use for indicating a place or object (with head, finger, etc.)"),
                             "Stop":("r_stop","gesture that you make with arms to tell someone that he as to slow down or stop to do something (for example with arms ahead)"), \
                             "I don't know":("r_idk", "moving the shoulders up and down or move both the arms in front of you with palms facing up, used when you want to to show that you don't know something"), \
                             "I beg you":("r_beg","the gesture that you make when you are begging someone, for instance by keeping arms in a pray position"), \
                             "C'mon, it's late":("r_time", "indicating the wrist with an index to say that it's late for something")}

In [ ]:
################ NEW

### COMPUTE RESULTS FOR FIXED WINDOW

#TODO SET PRIORITY DEPENDING ON SIMILARITY MATCH
#TODO DO COMPUTATIONS AGAIN AND DO NOT USE ALL_SIMILARITIES PRECIOUSLY COMPUTED

# give priority to each topic. Use the best window of each topic to take data of the sentence. If, for a topic, we have overlapping windows, chose the one that returns the best value.
# After we choose a topic given the value, the window size and the priority, all the words of that window can't be assigned to any other topic



def process_results_priority_policy_best_windows(best_windows = {} ,sentences = [], new_th = 0.2, type = 'conversation'):

    final_labels=[]
    #topics_ordered = sorted(best_windows.items(), key=lambda x: x[1]['priority'], reverse=True) #order the topics on priority values
    window_max_dim = 10
    counter = 0

    for k,sentence in enumerate(sentences):
        sentence_labels = {'sentence':'', 'topics&ref':[],  'matching_words':[], 'values':[], 'processing_time': 0,
                           'sentence_index':k, 'word_indexes':[]} #it contains a sentence labelled
        sentence_labels['sentence'] = sentence
        sentence_splits = sentence.split()
        final_word_index = len(sentence_splits) - 1 #index of the last word (starting from 0)
        current_word_index = 0

        start_time = time.perf_counter()
        while current_word_index <= final_word_index:
            #topics_best_res = {}
            top_res = 0
            top_words = ''
            top_ref = ''
            best_topic = ''
            top_word_indexes = []
            top_window = 0
            for key in all_refs.keys():    #for all types of references
                window_dim = best_windows[key]['best_window']
                if window_dim == 0:
                    continue
                if current_word_index + window_dim > final_word_index: #then we can't  compute result with this topic
                    continue
                for ref_sentence in all_refs[key]: #for all references
                    sub_sentence = ' '.join(sentence_splits[current_word_index : current_word_index + window_dim])
                    tmp_res = model.predict([sub_sentence,ref_sentence])
                    #print(window_dim,key,tmp_res, top_res)
                    if tmp_res > top_res:
                        top_words = sub_sentence
                        top_ref = ref_sentence
                        top_res = tmp_res
                        best_topic = key
                        top_window = window_dim
                        top_word_indexes = [i for i in range(current_word_index,current_word_index + window_dim)]
                #topics_best_res[key] = top_res
            #max_topic = max(topics_best_res, key=topics_best_res.get) #find the topic with highest match score
            #max_value = topics_best_res[max_topic]
            if top_res == 0:  #then we reached the end of the sentence
                break
            elif top_res >= new_th: #then we have a match
                #print("I'm here", ((best_topic, top_words,top_res, top_window)))
                sentence_labels['topics&ref'].append((best_topic,top_ref)) #save the result
                sentence_labels['matching_words'].append(top_words)
                sentence_labels['values'].append(top_res)
                sentence_labels['word_indexes'].append(top_word_indexes)
                current_word_index +=  top_window + 1
                continue
            else:
                current_word_index = current_word_index + 1
                continue
        end_time = time.perf_counter()
        processing_time = round(end_time-start_time, 4)
        print("ELAPSED TIME:", processing_time)
        counter += 1
        print(sentence_labels)
        print("SENTENCE INDEX: ", counter)
        sentence_labels["processing_time"] = processing_time
        final_labels.append(sentence_labels)
    return final_labels

path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
#filename = 'all_similarities_600.pkl'
#filename_best = "new_all_processed_sentences_best.pkl"
#filename_local = "all_processed_sentences_local.pkl"
#filename_windows = "best_windows_200_sentences.pkl"
#filename_windows = 'best_windows_sentences_Antonio_random_large_0_5.pkl'
#filename_sentences = "sentences_gpt_300_old.pkl"


from sentence_transformers import CrossEncoder
names = ["Antonio","Antonio_random","Alice","Alice_random","Carmine_nuove","Carmine_nuove_random","Lucrezia","Lucrezia_random"]
th = [0.3, 0.6, 0.9] #0.9
th_string = ["0_3", "0_6", "0_9"] #, "0_9" , "0_5"
models = ["base","large"]
types = ["","old"]


for mod in models:
    model = CrossEncoder(f'cross-encoder/stsb-roberta-large')  #stsb-roberta-large
    #( model.config.model_type)
    for name in names:
        for i,t in enumerate(th):
            filename_windows = f"best_windows_sentences_{name}_{mod}_0_3.pkl"
            b_windows = pickle.load(open(os.path.join(path,filename_windows),'rb'))
            for type in types:
                if type == "":
                    filename_sentences = "sentences_gpt_300.pkl"
                    filename_labels = f"results_priority_policy_best_windows_{name}_{mod}_{th_string[i]}.pkl"
                else:
                    filename_sentences = "sentences_gpt_300_old.pkl"
                    filename_labels = f"results_priority_policy_best_windows_{name}_{mod}_{type}_{th_string[i]}.pkl"

                if os.path.exists((os.path.join(path,filename_labels))):
                    print(f"FILE {filename_labels} already exists, SKIPPED")
                    continue
                sentences = pickle.load(open(os.path.join(path,filename_sentences),'rb'))
                if name == "Antonio" and mod == "base":
                    n_tmp = "Antonio_base"
                elif name == "Antonio" and mod == "large":
                    n_tmp = "Antonio_large"
                else:
                    n_tmp = name

                all_refs = {"r_greet":reference_sentence_greet[n_tmp], "r_clap":reference_sentence_clap[n_tmp],  "r_idk":reference_sentence_idk[n_tmp], "r_no":reference_sentence_no[n_tmp],"r_yes":reference_sentence_yes[n_tmp],"r_stop":reference_sentence_stop[n_tmp], "r_beg":reference_sentence_beg[n_tmp],"r_time":reference_sentence_time[n_tmp], "r_apologize":reference_sentence_apologize[n_tmp], "r_deitic":reference_sentence_deictic[n_tmp],"r_exulting":reference_sentence_exulting[n_tmp],"r_run":reference_sentence_run[n_tmp]}

                labels = process_results_priority_policy_best_windows(best_windows=b_windows,sentences=sentences, new_th=t, type = 'sentences')
                print("writing to file:",filename_labels)
                pickle.dump(labels, open(os.path.join(path,filename_labels),'wb'))

'''
if(os.path.exists(os.path.join(path,filename_windows))):    #load data already processed
    b_windows = pickle.load(open(os.path.join(path,filename_windows),'rb'))
else:
    print("Invalid path windows")

if(os.path.exists(os.path.join(path,filename_sentences))):    #load sentences
    sentences = pickle.load(open(os.path.join(path,filename_sentences),'rb'))
else:
    print("Invalid path sentences")

print(b_windows)
labels = process_results_priority_policy_best_windows(best_windows=b_windows,sentences=sentences, new_th=0, type = 'sentences')
filename_labels =  "results_priority_policy_best_windows_Antonio_random_large_old_0_0.pkl"
print("writing to file...")
pickle.dump(labels  , open(os.path.join(path,filename_labels),'wb'))

for label in labels:
    print(label, "\n \n \n \n")
'''

In [ ]:
### MOVING WINDOW LABELLING

# this function process the results obtained previously that contain all the possible matches between the words inside each sentence and all the references (grouped by topic, or gesture)
# by using the sentence similarity. The function must have the following characteristics:
# 1) only the results greater than a certain threhsold new_th are accepted
# 2) in case we have a match between words inside a window and more than one references related to one topic, we take only the best one related to that topi (already true in
# all_processed_sentences_best.pkl file)
# 3) in case we have more topics that have a match with words in a window, we take the topic related to the best match
# 4) the functions has to start with a window_dim equal to 1, then compares the best match of a topic with window one, with the matches found by incrementing the window size: it takes
# the best one
# 5) once we found the best window that matches to best topic, we try to see if, by incrementing n_times the window size, the result for that topic changes more than a given threshold.
# If it changes more than a given threshold at least one time, that best topic is no more considered valid
# 6) if the topic is no more considered valid, it restarts from window_dim equal to 1 but this time it starts to search considering only from second best topic to below.
# 7) repeats steps 5 and 6 until it find a match that works.
# 8) if it find a match that works, we start a new search by moving the window by 1 and comparing all the results obtained with the current best. If a better one is found, substitute the
# the best window with the one found, otherwise continue to move until the window no more contain the words of the best window.
# 9) finally, if for a word we don't have a match, the algorithm moves to the next word
# TODO add a full stop control

def process_results_best_policy(results, all_refs, new_th=0 ,window_max_dim=10, type = 'conversation'): #the maximum number of words that the window can contain

    #takes all elements of the sentence that have a value greater than threshold
    def take_elements_th(el, window_dim, new_th):
        #print(el)
        all_values = el[f"window {window_dim}"]['value']  #all the values (results of the matches between topics and parts of the sentence
        th_indexes = [i for i in range(len(all_values)) if all_values[i] > new_th] #returns the indexes of the values that are greater than new_th
        values = list(map(all_values.__getitem__,th_indexes)) #take all the values greater than new_th
        refs = list(map(el[f"window {window_dim}"]['ref'].__getitem__,th_indexes)) #take all the references that return a value greater than new_th
        matching_words = list(map(el[f"window {window_dim}"]['sentence'].__getitem__,th_indexes)) #take all the sentence parts that return a value greater than new_th
        start_word_indexes = list(map(el[f"window {window_dim}"]['start_word_indexes'].__getitem__,th_indexes))

        return values, refs, matching_words, start_word_indexes

    #n_tries indicates how many times we want to check that the threshold in maintained trough time
    # t_res contains the result referred to the best topic found in a part of the sentence
    # s_res contains all the results obtained by applying the sentence model over a sentence (only results greater tan a certain threshold are taken)
    # sentence contains the sentence that we are processing
    # max_dim is the maximum size of the window that we can apply over that sentence
    def is_coherent(n_tries, th, s_res, t_res, sentence, max_dim):
        s_splits = sentence.split()
        current_topic = t_res['topic'] #the topic found
        current_topic_value = t_res['best_match'] #the value referred to the topic found
        end_index = t_res['new_last_word_idx'] + 1 #the index of the last word of the topic found
        matching_words = t_res['matching_words'] # a sentence can have more matching_words inside, we have to pick the right ones
        start_index = t_res['old_last_word_idx'] + 1
        if t_res['new_last_word_idx'] == len(s_splits) - 1: #in this case we reached the end and we have nothing else to check (we assume that is coherent)
            return True
        #print("IS COHERENT:",matching_words, s_splits[start_index:end_index])
        #print("Sentence:",sentence)
        #print("t_res", t_res['topic'], t_res['best_match'], t_res['matching_words'],"start", start_index,"end", end_index,"len", len(matching_words),"ind", t_res['new_last_word_idx'])
        #print("COHERENCE",matching_words, s_splits[start_index:end_index])
        assert(matching_words == ' '.join(s_splits[start_index:end_index])) #check if the words we are picking are the same of the matching
        topic_ok = False

        for t in range(1,n_tries+1):
            if end_index + t > len(s_splits) - 1 or len(matching_words.split()) + t > max_dim: #check if, by increasing the window size, we reach the end of the sentence or if we can
                return True                                                                    #try to search results in a greater window
            else:
                window_dim = len(matching_words.split()) + t
                s_results = s_res[f"window {window_dim}"]
                processed_words = ' '.join(s_splits[start_index:(start_index + window_dim)])
                #print("IS COHERENT 2:",t_res['old_match_idx'])
                start_search_index = t_res['old_match_idx'][f"window {window_dim}"] #since we are assuming that the match is still not valid, start to search values from old match_indexes
                #print("IS COHERENT 3:",processed_words, s_results['matching_words'][start_search_index:])
                if not processed_words in s_results['matching_words'][start_search_index:]: #then the topic doesn't exists anymore, so fur sure we have a drop in probabilities
                    return False
                else:
                    match_index = s_results['matching_words'][start_search_index:].index(processed_words)
                    n_possible_topics = 0
                    while s_results['matching_words'][start_search_index:][match_index:][n_possible_topics] == processed_words: #count how many topics match with processed words
                        n_possible_topics = n_possible_topics + 1
                        if n_possible_topics >= len(s_results['matching_words'][start_search_index:][match_index:]):
                            break
                    all_match_values = s_results['values'][start_search_index:][match_index : match_index + n_possible_topics]
                    for value in all_match_values:
                        tmp_ref = s_results['refs'][start_search_index:][s_results['values'][start_search_index:].index(value)] #extract the reference associated with the value
                        tmp_topic = [key for key in all_refs.keys() for ref in all_refs[key] if tmp_ref == ref][0] #extract the topic associated with reference
                        if tmp_topic == current_topic:
                            if current_topic_value - value < th: #then the topic does not change so mach by incrementing the window size, check for another n_try
                                topic_ok = True
                                break
                        else: #check the other topic
                            continue
                    if topic_ok:
                        continue
                    else:
                        return False
        return True #if we reached this point than everything is ok. If the function returns False, than we have to avoid the topic

    #this function checks if the topic found is for sure the best one: since we didn't try to search results that starts after a given word, we are not sure for that
    #the paramters are the same as the previous function except for n_tries
    #the function returns a dictionary which tells if the current topic is the best one; if it is the best one that dictionary contain that topic otherwise
    #it will contain the best topic (i.e. the topic that returns greater results). The dictionary contains also the skipped words in case we the best topic changes by sliding the window
    def is_real_best(s_res, t_res, sentence, max_dim):
        s_splits = sentence.split()
        max_th_drop = 0.2 #for coherence check
        current_topic = t_res['topic'] #the topic found
        current_topic_value = t_res['best_match'] #the value referred to the topic found
        end_index = t_res['new_last_word_idx'] + 1  #the index of the last word of the topic found
        matching_words = t_res['matching_words'] # a sentence can have more matching_words inside, we have to pick the right ones
        #matching_words_indexes = t_res['matching_words_indexes']
        start_index = t_res['old_last_word_idx'] + 1
        skipped_words = ''
        end = False
        #tmp_new_match_idx = t_res['old_match_idx'].copy()
        if t_res['new_last_word_idx'] == len(s_splits) - 1: #in this case we reached the end and we have nothing else to check (we assume that is the best topic)
            return {"is_best": True, "topic_result": t_res, "skipped_words":skipped_words}
        assert(matching_words == ' '.join(s_splits[start_index:end_index])) #check if the words we are picking are the same of the matching
        tmp_best_res = t_res.copy()
        #processed_words = ' '.join(s_splits[start_index:(start_index + 1 + window_dim)])
        if len(matching_words.split()) == 1: #in this case we have nothing to check since in the next iteration we will search for other words that will not contains the current one
            return {"is_best": True, "topic_result": t_res, "skipped_words":skipped_words}
        else:
            topics_to_avoid = {'topic & dim': []}
            tmp_topics_to_avoid =  topics_to_avoid.copy()
            for t in range(1,len(matching_words.split())-1):
                '''
                w_d = 1 #window dimension ############################### this block is used to start to search results from the right index
                while t_res['old_last_word_idx'] + w_d <= len(s_splits) - 1 and w_d <= max_dim:    #until we reach the end of the sentence
                    s_results = s_res[f"window {w_d}"]
                    if len(s_results['start_word_indexes']) != 0:
                        tmp = [i for i in range(len(s_results['start_word_indexes'])) if s_results['start_word_indexes'][i] > t_res['old_last_word_idx'] + w_d - 1]
                        if len(tmp) > 0: #it means that we have at least one value that will possibly match
                            tmp_idx_start_match_idx[f"window {w_d}"] = tmp[0]
                        else:
                            tmp_idx_start_match_idx[f"window {w_d}"] = len(s_results['matching_words']) #we can avoid to search again
                        w_d = w_d + 1
                '''
                while True:  #try to search if there exists a topic that is coherent (does not change by th if we increase the window) and that is better than the current one
                    next_result = find_best_topic(s_res, max_dim, t_res['old_match_idx'], start_index + t, sentence, topics_to_avoid = topics_to_avoid)
                    #print(next_result)
                    if next_result['new_last_word_idx'] != next_result['old_last_word_idx']: ## then we have a new match
                        #then this topic associated to these words is not coherent, check the next word
                        if not is_coherent(n_tries = 1, th = max_th_drop, s_res = s_res, t_res = next_result, sentence = sentence, max_dim = max_dim): #skip this topic and try to search the next
                            topics_to_avoid['topic & dim'].append((next_result['topic'],next_result['best_window'])) #######
                            continue
                        if next_result['best_match'] > current_topic_value:
                            tmp_best_res = next_result.copy()
                            skipped_words = s_splits[start_index : start_index + t]
                            topics_to_avoid['topic & dim'] = []
                        break #in any case we stop the topic search
                    if topics_to_avoid != tmp_topics_to_avoid: #then we have a new topic to avoid and we can search if there are som other topics better than the current one
                        tmp_topics_to_avoid = topics_to_avoid.copy()
                        continue
                    else:
                        break
                    #if next_result['end']: #we reached the end and we didn't find any result
                    #    break

        return {"is_best": False, "topic_result": tmp_best_res, "skipped_words":skipped_words}

    #this function finds the best topic starting from a given word. The best topic is the topic that, for a certain window_dim that contains some of the words of the sentence,
    #returns the best results. If for a given word we can't find any match, the function returns some default values, otherwise it retuns the best topic, the associated
    #reference and value, the words of the sentence that return the best topic, the window_dim that contains those words, the old and new indexes that tell us where to start
    #at the next iteration for both the sentence and the results related to that sentence. If this function is called and we fund that the returned result is for real the best one (for
    #some define policy) then we update all the indexes, otherwise we maintain the old ones and we restart teh search by not considering the topics that do not respect the policy.
    def find_best_topic(s_res, max_dim, idx_last_matches, idx_last_word, #starting from idx_last_word+1, it will find the window that returns the best topic value
                        sentence, topics_to_avoid ={}): #topics_to_avoid indicates which topics we can't search for (e.g. if we found that a topic does not respect the                                                 #policy). and the referred window_dim on which we can't search for that topic
        s_splits = sentence.split()
        best_match = 0 #initialize the results in case we do not find anything
        best_window = 1
        ref = ''
        topic = ''
        matching_words = ''
        matching_words_indexes = None
        tmp_best_value_match = 0

        tmp_idx_last_matches = idx_last_matches.copy()
        #print(idx_last_word)
        #print(idx_last_matches)
        tmp_idx_last_word = idx_last_word
        window_dim = 1
        end = False #this becomes true when we reached the end of the sentence, i.e. if we are in the last word

        if idx_last_word + window_dim == len(s_splits):
            end = True

        #print(s_res)
        #print(max_dim)
        while idx_last_word + window_dim <= len(s_splits) - 1:    #until we reach the end of the sentence
            #for window in range(1,max_dim + 1):
            s_results = s_res[f"window {window_dim}"]
            #print(idx_last_matches)
            start_search_index = idx_last_matches[f"window {window_dim}"] #we take the data inside s_res from this index since the previous ones were already used for search purposes
            processed_words = ' '.join(s_splits[(idx_last_word+1):(idx_last_word + 1 + window_dim)]) #we start with window = 1
            processed_words_indexes = [i for i in range((idx_last_word+1),(idx_last_word + 1 + window_dim))]
            #print(processed_words, "    idx last match", idx_last_matches[f"window {window_dim}"], "    len results",len(s_results['matching_words']))
            #print("window_dim:",window_dim)
            #print("processed words:",processed_words, "  window_dim:", window_dim, "  dim_max:",max_dim, "  start_search_index:",start_search_index)
            #print("RESULTS:", s_results['matching_words'][start_search_index:], "window_dim:", window_dim, processed_words)
            if idx_last_matches[f"window {window_dim}"] > (len(s_results['matching_words']) - 1): #it means that we have no more results for that window_dim, check with another window
                #print("continuing...", "   window_dim",window_dim, "  idx", idx_last_matches[f"window {window_dim}"])
                if max_dim > window_dim:
                    #print("HERE",max_dim,window_dim)
                    window_dim = window_dim + 1
                    continue
                else:
                    break
            #print(processed_words, s_results['matching_words'][start_search_index:])
            if processed_words in s_results['matching_words'][start_search_index:]: #then we have at least one match
                index = s_results['matching_words'][start_search_index:].index(processed_words) # takes the index of the first match between a topic
                                                                                                 # and processed words starting from the last index where we had a match previously
                #print("index",index)
                n_possible_topics = 0
                #print("HERE:", s_results['matching_words'][start_search_index:][index:], processed_words)
                while s_results['matching_words'][start_search_index:][index:][n_possible_topics] == processed_words: #count how many topics match with processed words
                    n_possible_topics = n_possible_topics + 1
                    #print(n_possible_topics)
                    if n_possible_topics >= len(s_results['matching_words'][start_search_index:][index:]): #avoid taking elements that are outside the array
                        break
                # these values are associated with the topics that match the # processed words
                all_match_values = s_results['values'][start_search_index:][index : index + n_possible_topics]
                ordered_values = sorted(all_match_values, reverse=True) #order the values that match from the highest to the lowest
                #print(ordered_values,tmp_best_value_match)

                #tmp_idx_last_matches[f"window {window_dim}"] = start_search_index + index + len(ordered_values) - 1
                #print("inside fun indx matches:",idx_last_matches)
                #print("topics to avoid",topics_to_avoid,"len ordered values", len(ordered_values))
                #if len(ordered_values) == len(topics_to_avoid): #then we increment the window_size and try with other topics
                #    skip_dims.append(window_dim) #we skip until this dimension next iteration
                #    print("topics before breaking:",topics_to_avoid)
                #    print("breaking... Skipping_dim:",skip_dims)
                #    break
                for i in range(len(ordered_values)):
                    tmp_match = ordered_values[i]
                    #print(tmp_match)
                    #print("match:",tmp_best_match,"  window_dim:",window_dim, "  start_search_index", start_search_index)
                    tmp_ref = s_results['refs'][start_search_index:][s_results['values'][start_search_index:].index(tmp_match)] #reference associated to best result
                    #print(tmp_ref)
                    tmp_topic = [key for key in all_refs.keys() for ref in all_refs[key] if tmp_ref == ref][0] #extract the topic associated with reference
                    #print("FINAL CHECK:",(tmp_topic,window_dim), "FINAL CHECK 2:",topics_to_avoid['topic & dim'])
                    if (tmp_topic,window_dim) in topics_to_avoid['topic & dim']: #try with the next ordered value because this one referred to window_dim does not respect he policy
                        #print("SKIPPED TOPIC")
                        continue
                    else:
                        if tmp_match > tmp_best_value_match: #then save the current best
                            #print("OKOKOK")
                            tmp_idx_last_word = idx_last_word + window_dim #Index of the last processed word.
                            tmp_best_value_match = tmp_match
                            matching_words = processed_words
                            matching_words_indexes = processed_words_indexes
                            best_window = window_dim
                            best_match = tmp_best_value_match
                            ref = tmp_ref
                            topic = tmp_topic
                            break # stop the search for this window_dim, try with a bigger window

            if max_dim > window_dim:
                window_dim = window_dim + 1
            else: #we dindn't find any result even with the maximum window size, so we didn't find any match for a word and we can try with the next one (in the main)
                break

        # save the new indexes from which to start the search for each window. These are temporary since it may happen that the best
        # topic found does not respect the policy (in this case indexes must not be updated).
        # We search for the first match that has a "last_word_index" greater than tmp_idx_last_word ([tmp_idx_last_word + 1][0). Since start index starts from
        # idx_last_word + 1, here we have to compensate by subtracting 1
        window_dim = 1
        while idx_last_word + window_dim <= len(s_splits) - 1 and window_dim <= max_dim:    #until we reach the end of the sentence
            s_results = s_res[f"window {window_dim}"]
            if len(s_results['start_word_indexes']) != 0:
                #print("IDX LAST WORD:",tmp_idx_last_word)
                tmp = [i for i in range(len(s_results['start_word_indexes'])) if s_results['start_word_indexes'][i] > tmp_idx_last_word]#[0] # - 1
                #print(f"TMP INDEXES DIM :{window_dim}, RES:", tmp_idx_last_matches[f"window {window_dim}"])
                if len(tmp) > 0: #it means that we have at least one value that will possibly match
                    tmp_idx_last_matches[f"window {window_dim}"] = tmp[0]
                else:
                    tmp_idx_last_matches[f"window {window_dim}"] = len(s_results['matching_words']) #we can avoid to search again
            window_dim = window_dim + 1

        #print(best_match,tmp_idx_last_word,idx_last_word )
        return {'best_match':best_match, 'best_window':best_window, 'reference':ref, 'topic':topic, 'matching_words':matching_words,
                'matching_words_indexes': matching_words_indexes, 'new_match_idx':tmp_idx_last_matches, 'old_match_idx':idx_last_matches,
                'new_last_word_idx':tmp_idx_last_word, 'old_last_word_idx':idx_last_word, 'end': end, 'topics_to_avoid': topics_to_avoid}

    all_labels = [] #it will contain all the labels for all the sentences
    max_th_drop = 0.2  #if we find that the best topic x does not change by this amount for n_tries times, then this is the best topic. Otherwise we can try by taking the second best topic and by repeating the check
    #all_res = []
    for ind,el in enumerate(results):
        sentence_labels = {'sentence':'', 'topics&ref':[],  'matching_words':[], 'values':[], 'priority_levels':[],
                           'sentence_index':ind, 'word_indexes':[], "skipped_words":[], 'processing_t_1':0, 'processing_t_2':0} #it contains a sentence labelled
        #sentence_labels = {'sentence':'', 'topics&ref':[],  'matching_words':[], 'values':[], "skipped_words":[]} #it contains a sentence labelled
        if type == 'conversation':
            sentence = el['whole sentence,speaker,turn'][0]
            speaker = el['whole sentence,speaker,turn'][1]
            turn = el['whole sentence,speaker,turn'][2]
        elif type == 'sentences':
            sentence = el['whole sentence']
        else:
            print('Undefined type')
            return

        start = time.perf_counter()
        sentence_labels['sentence'] = sentence
        sentence_splits = sentence.split()
        final_word_index = len(sentence_splits) - 1 #index of the last word (starting from 0)
        last_processed_word_idx = -1
        processed_words = sentence_splits[0]
        sentence_results = {}
        if len(sentence_splits) < window_max_dim:
            window_max_d = len(sentence_splits)
        else:
            window_max_d = window_max_dim

        last_match_indexes = {}
        #for w in sentence_results.keys():
        #    print(sentence_results[w],"\n \n \n")
        for i in range(1,window_max_d + 1): #take all the data related to the matches greater than a threshold for a given sentence
            values, refs, matching_words, start_word_indexes = take_elements_th(el, i, new_th)
            sentence_results[f"window {i}"] = {'matching_words':[], 'refs':[], 'values':[]}
            sentence_results[f"window {i}"]['values'] = values
            sentence_results[f"window {i}"]['matching_words'] = matching_words
            sentence_results[f"window {i}"]['refs'] = refs
            sentence_results[f"window {i}"]['start_word_indexes'] = start_word_indexes
            last_match_indexes[f"window {i}"] = 0 #this dict will contain, for each window_dim, the index of the first element from which we will start the search in that window
                                                  #every time we use data, we update this indexes to avoid searching data that is no more useful

        topics_to_avoid = {'topic & dim': []}

        counter = 0
        '''
        print(sentence,"\n \n \n")
        for w in sentence_results.keys():
            print(sentence_results[w],"\n \n \n")
        '''
        while True and counter<10:
            #print("counter:",counter)
            #print("topics_before:",topics_to_avoid)
            #print("last match idx before:",last_match_indexes)
            result_dict = find_best_topic(sentence_results, window_max_d, last_match_indexes, last_processed_word_idx, sentence, topics_to_avoid)
            #print("last_idx:",result_dict['old_last_word_idx'],'new_idx:',result_dict['new_last_word_idx'], result_dict['topic'])
            #print(last_processed_word_idx)
            #print(result_dict)
            if result_dict['new_last_word_idx'] != result_dict['old_last_word_idx']: #then we found a match that respects the policy starting from a word
                #print(result_dict, "\n \n \n")
                #print(is_coherent(n_tries = 1, th = max_th_drop, s_res = sentence_results, t_res = result_dict, sentence = sentence, max_dim = window_max_d))
                if is_coherent(n_tries = 1, th = max_th_drop, s_res = sentence_results, t_res = result_dict, sentence = sentence, max_dim = window_max_d):
                    #print("OK:",result_dict,"\n \n \n")
                    #print("LAST WORD IDX",last_processed_word_idx)
                    check = is_real_best(s_res = sentence_results, t_res = result_dict, sentence = sentence, max_dim = window_max_d)
                    if check["is_best"]: #then the current topic is for real the best one
                        sentence_labels['topics&ref'].append((result_dict['topic'],result_dict['reference']))
                        sentence_labels['matching_words'].append(result_dict['matching_words'])
                        sentence_labels['word_indexes'].append(result_dict['matching_words_indexes'])
                        sentence_labels['values'].append(result_dict['best_match'])
                        last_processed_word_idx = result_dict['new_last_word_idx']
                        last_match_indexes = result_dict['new_match_idx'].copy()
                    else:
                        new_topic = check["topic_result"]
                        sentence_labels['topics&ref'].append((new_topic['topic'],new_topic['reference']))
                        sentence_labels['matching_words'].append(new_topic['matching_words'])
                        sentence_labels['word_indexes'].append(new_topic['matching_words_indexes'])
                        sentence_labels['values'].append(new_topic['best_match'])
                        sentence_labels['skipped_words'].append(check['skipped_words']) #TODO manage the skipped words
                        last_processed_word_idx = new_topic['new_last_word_idx']
                        last_match_indexes = new_topic['new_match_idx'].copy()
                        #print("HERERHRHERHE"
                    topics_to_avoid['topic & dim'] = [] #in botch cases we found the best result
                    #print("LAST WORD IDX",last_processed_word_idx)
                else:
                    #print("PROBLEM:",result_dict,"\n ")
                    #print("last match idx after:",last_match_indexes)
                    topics_to_avoid['topic & dim'].append((result_dict['topic'],result_dict['best_window']))
                    #print("NEW TOPICS TO AVOID:",topics_to_avoid, "\n \n \n")
                    #print("topics_to_append:",result_dict['topic'])
                    #print("topics_before:",topics_to_avoid)

                    #print("topics_after:",topics_to_avoid)

            #in this case we didn't find any match, go to the next word
            else:
                topics_to_avoid['topic & dim'] = []
                last_processed_word_idx = last_processed_word_idx + 1

            if result_dict['end'] == True: #we reached the end
                #print("ending...")
                break
            counter = counter + 1
        end = time.perf_counter()
        processing_t_2 = end-start
        sentence_labels['processing_t_1'] = el["processing_time"]
        sentence_labels['processing_t_2'] = round(processing_t_2,4)
        print("processing time t1",sentence_labels['processing_t_1'],'processing time t2',processing_t_2, "SENTENCE_IDNEX",ind)

        all_labels.append(sentence_labels)

    return all_labels


path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
#filename_old = 'all_similarities_Alice_base_old.pkl' #'all_similarities_standard_base_old.pkl' #"all_processed_sentences_best.pkl"
#filename = 'all_similarities_Alice_random_base_old.pkl' #'all_similarities_standard_base_old.pkl' #"all_processed_sentences_best.pkl"
#filename = "new_all_processed_sentences_best.pkl"
#filename = 'all_similarities_600.pkl'
name = 'Antonio_random' # Alice Carmine_nuove Carmine_nuove_random Antonio Antonio_random Lucrezia Lucrezia_random
th = [0.3, 0.5, 0.6, 0.9]
th_string = ["0_3", "0_5", "0_6", "0_9"]
models = ["base","large"]
types = ["","old"]
for i,t in enumerate(th):
    for model in models:
        for type in types:
            if type == "":
                filename = f'all_similarities_{name}_{model}.pkl'
            else:
                filename = f'all_similarities_{name}_{model}_{type}.pkl'
            results = pickle.load(open(os.path.join(path,filename),'rb'))
            labels = process_results_best_policy(results, all_refs, new_th=t, type = 'sentences')
            if type == "":
                filename_labels = f"results_moving_windows_{name}_{model}_{th_string[i]}.pkl"
            else:
                filename_labels = f"results_moving_windows_{name}_{model}_{type}_{th_string[i]}.pkl"
            print("writing to file...")
            pickle.dump(labels  , open(os.path.join(path,filename_labels),'wb'))
'''
if(os.path.exists(os.path.join(path,filename))):    #load data already processed
    results = pickle.load(open(os.path.join(path,filename),'rb'))
else:
    print("Invalid path")
'''




#filename_labels = "results_moving_window_policy.pkl"


#for el in labels:
#    print(el,"\n \n \n")
#print(labels)

In [ ]:
#### RANDOM LABELING

def process_results_random(new_th=0.2):

    def gen_avg(expected_avg=5, n=10, min=1, max=10): #generates a sequence of n random numbers that have minimum min, maximum max, and average expected_avg
        values = [random.randint(min, max)  for x in range(n)]
        average = sum(values)/n
        new_values = [round(x - average + expected_avg) for x in values]
        #new_average = sum(new_values)/n
        return new_values

    def count_gt_instances(): #counts the labelled words in the corpus, all the words in topic, all the labels and for each label counts how many times they appear and how many words they label
        topic_count =  {topic_name: {'topic_instances':0, 'topic_words':0} for topic_name in topic_names}
        total_words = 0 #total words in corpus
        total_labels = 0 #total label instances
        total_labelled_words = 0 #counts how many words were labelled
        for gt_result in gt_labels:
            sentence = gt_result['sentence']
            total_words += len(sentence.split())
            for gt_label in gt_result['labels,word_indexes']:
                gt_topic = topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
                topic_count[gt_topic]['topic_instances'] += 1
                topic_count[gt_topic]['topic_words'] += len(gt_label[1])
                total_labels += 1
                total_labelled_words += len(gt_label[1])
        print("TOPIC COUNT:",topic_count)
        print("total_words",total_words)
        print("total_labels",total_labels)
        print("total_labelled_words",total_labelled_words)

        return topic_count, total_words, total_labels, total_labelled_words


    def label_probabilities(topic_count, total_words, total_labels, total_labelled_words): #computes the probabilities for each topic and assigns to each word of corpus a topic
                                                                                           #TODO it does not consider full stops when it labels so it may happen that the window falls in the middle of two sentences.
        probs_dictionary = {topic_name: {'average_window_size':0, 'instance_probability':0} for topic_name in topic_names}
        probs_instance_list = [] #compute the instance probability depending on how many times a topic appears in a corpus
        topic_names_list= [] #only for debugging, to be sure that topic probabilities appear in the correct order
        probs_window = {} #compute the window probability depending on the average_window_size


        for topic_name in topic_names: #compute probabilities related to each topic
            if topic_count[topic_name]['topic_instances'] != 0:
                probs_dictionary[topic_name]['average_window_size'] = topic_count[topic_name]['topic_words'] / topic_count[topic_name]['topic_instances']
                probs_dictionary[topic_name]['instance_probability'] = topic_count[topic_name]['topic_instances'] / total_labels #prob to have a label of some topic among all the labels
                topic_names_list.append(topic_name)
                probs_instance_list.append(probs_dictionary[topic_name]['instance_probability'])
                probs_window[topic_name] = (gen_avg(expected_avg=probs_dictionary[topic_name]['average_window_size'],
                                                n=total_labels, min = 1, max = window_max_dim))

        labels_list = np.random.choice(topic_names_list,total_labels,p=probs_instance_list) #the list that shows the order on which the labels will appear accordingly to instance probs
        labelled_words = []
        prob_to_label_word = total_labels / total_words
        i = 0
        label_index = 0 #to select a label in labels_list
        label_indexes = {topic_name: 0 for topic_name in topic_names} #it takes trace of the chosen window for each topic

        while i < total_words:
            is_labelled = random.random() < prob_to_label_word
            if is_labelled and label_index < len(labels_list):
                topic = labels_list[label_index]
                topic_window = probs_window[topic][label_indexes[topic]] #choose the window size for the topic
                for j in range(topic_window):
                    labelled_words.append(topic)
                label_index += 1
                #print("label_index", label_index, "Word",i,"total_words", total_words)
                label_indexes[topic] += 1
                i += topic_window
            else:
                labelled_words.append('NONE')
                i += 1
        return probs_dictionary, labelled_words


    def label_sentences(labelled_words):

        all_labelled_sentences = []
        label_index = 0
        for k, gt_result in enumerate(gt_labels):
            sentence_labels = {'sentence':'', 'topics&ref':[],  'matching_words':[], 'values':[],
                               'sentence_index':k, 'word_indexes':[],'processing_time':[]} #it contains a sentence labelled
            sentence = gt_result['sentence']

            start = time.perf_counter()
            sentence_labels['sentence'] = sentence
            sentence_splits = sentence.split()
            final_word_index = len(sentence_splits) - 1 #index of the last word (starting from 0)
            current_word_index = 0

            while current_word_index <= final_word_index:
                chosen_topic = labelled_words[label_index]
                if chosen_topic == "NONE":
                    current_word_index += 1
                    label_index += 1
                    continue
                else:
                    tmp_topic = chosen_topic
                    tmp_matching_words = []
                    tmp_word_indexes = []
                    #print("Chosen topic:",chosen_topic)
                    while tmp_topic == chosen_topic:
                        tmp_matching_words.append(sentence_splits[current_word_index])
                        tmp_word_indexes.append(current_word_index)
                        current_word_index += 1
                        label_index += 1
                        chosen_topic = labelled_words[label_index]
                        if current_word_index > final_word_index:
                            break
                    ref_index = random.randint(0, len(all_refs[tmp_topic])-1) #choose a reference randomly for the given topic
                    #print(ref_index, len(all_refs[tmp_topic]), tmp_topic)
                    sentence_labels['matching_words'].append(' '.join(tmp_matching_words))
                    sentence_labels['values'].append(random.randint(new_th*10, 10)/10) #generage a random value between new_th and 1
                    sentence_labels['topics&ref'].append((tmp_topic,all_refs[tmp_topic][ref_index]))
                    sentence_labels['word_indexes'].append(tmp_word_indexes)
            end = time.perf_counter()
            processing_time = end-start
            sentence_labels['proceessing_time'] = round(processing_time,4)
            print("processing time",sentence_labels["proceessing_time"],"SENTENCE_INDEX",k)
            all_labelled_sentences.append(sentence_labels)

        return all_labelled_sentences

    gt_labels_names = topic_gesture_description.keys()
    window_max_dim = 10
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    topic_count, total_words, total_labels, total_labelled_words = count_gt_instances()
    #print(total_labels, total_labelled_words, total_words)
    topic_probs, labels = label_probabilities(topic_count, total_words, total_labels, total_labelled_words)
    labelled_sentences = label_sentences(labels)

    return labelled_sentences


path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
filename_labelled_sentences = "labelled_sentences_final_old.pkl" #it contains the ground truth labels
#filename = 'all_similarities.pkl'
filename_random = "results_random_final_0_9.pkl"

gt_labels = []
if(os.path.exists(os.path.join(path,filename_labelled_sentences))):    #load data already processed
    all_real_labels = pickle.load(open(os.path.join(path,filename_labelled_sentences),'rb'))
    for label in all_real_labels:
        gt_labels.append(label)
else:
    print("invalid real labels path")

random_labels = process_results_random(new_th=0.9)
'''
if(os.path.exists(os.path.join(path,filename_random))):    #load data already processed
    print("File already exists...")
    random_labels = []
    all_random_labels = pickle.load(open(os.path.join(path,filename_random),'rb'))
    for label in all_random_labels:
        print(label['sentence'])
        random_labels.append(label)
    #print(random_labels[2])
    print(len(random_labels))
else:
    random_labels = process_results_random(new_th=0.9)
    print(random_labels)
    print("Saving file...")
    pickle.dump(random_labels , open(os.path.join(path,filename_random),'wb'))
'''

In [ ]:
#### HALF CORPUS LABELLING
#TODO It is still not completed, it is not necessary. This function gives 50% probability to each word to have a label with a given win_size

def process_results_half_corpus(sentences = all_sentences, new_th=0.2):

    def count_gt_instances(): #counts the labelled words in the corpus, all the words in topic, all the labels and for each label counts how many times they appear and how many words they label
        topic_count =  {topic_name: {'topic_instances':0, 'topic_words':0} for topic_name in topic_names}
        total_words = 0 #total words in corpus
        total_labels = 0 #total label instances
        total_labelled_words = 0 #counts how many words were labelled
        for gt_result in gt_labels:
            sentence = gt_result['sentence']
            total_words += len(sentence.split())
            for gt_label in gt_result['labels,word_indexes']:
                gt_topic = topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
                topic_count[gt_topic]['topic_instances'] += 1
                topic_count[gt_topic]['topic_words'] += len(gt_label[1])
                total_labels += 1
                total_labelled_words += len(gt_label[1])

        return topic_count, total_words, total_labels, total_labelled_words

    def label_sentences(topic_count):
        topics_average_window = {topic_name: 0 for topic_name in topic_names}
        for topic_name in topic_names: #compute probabilities related to each topic
            if topic_count[topic_name]['topic_instances'] != 0:
                topics_average_window[topic_name] = round(topic_count[topic_name]['topic_words'] / topic_count[topic_name]['topic_instances'])

        all_labelled_sentences = []
        for k, gt_result in enumerate(gt_labels):
            sentence_labels = {'sentence':'', 'topics&ref':[],  'matching_words':[], 'values':[],
                               'sentence_index':k, 'word_indexes':[]} #it contains a sentence labelled
            sentence = gt_result['sentence']
            sentence_labels['sentence'] = sentence
            sentence_splits = sentence.split()
            for topic in topic_names:



        all_labelled_sentences = []
        label_index = 0



            sentence_labels['sentence'] = sentence
            sentence_splits = sentence.split()
            final_word_index = len(sentence_splits) - 1 #index of the last word (starting from 0)
            current_word_index = 0

            while current_word_index <= final_word_index:
                chosen_topic = labelled_words[label_index]
                if chosen_topic == "NONE":
                    current_word_index += 1
                    label_index += 1
                    continue
                else:
                    tmp_topic = chosen_topic
                    tmp_matching_words = []
                    tmp_word_indexes = []
                    #print("Chosen topic:",chosen_topic)
                    while tmp_topic == chosen_topic:
                        tmp_matching_words.append(sentence_splits[current_word_index])
                        tmp_word_indexes.append(current_word_index)
                        current_word_index += 1
                        label_index += 1
                        chosen_topic = labelled_words[label_index]
                        if current_word_index > final_word_index:
                            break
                    ref_index = random.randint(1, len(all_refs[tmp_topic])-1) #choose a reference randomly for the given topic
                    sentence_labels['matching_words'].append(' '.join(tmp_matching_words))
                    sentence_labels['values'].append(random.randint(new_th*10, 10)/10) #generage a random value between new_th and 1
                    sentence_labels['topics&ref'].append((tmp_topic,all_refs[tmp_topic][ref_index]))
                    sentence_labels['word_indexes'].append(tmp_word_indexes)
            all_labelled_sentences.append(sentence_labels)

        return all_labelled_sentences

    gt_labels_names = topic_gesture_description.keys()
    window_max_dim = 10
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    topic_count, total_words, total_labels, total_labelled_words = count_gt_instances()
    #print(total_labels, total_labelled_words, total_words)
    topic_probs, labels = label_probabilities(topic_count, total_words, total_labels, total_labelled_words)
    labelled_sentences = label_sentences(labels)

    return labelled_sentences


filename = "sentences.pkl"

path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks"
all_sentences = pickle.load(open(os.path.join(path,filename),'rb'))
#print(all_sentences)
'''
conv_subs=[]
for key in list(all_sentences.keys()):  #take sentences
    speaker = (key,)
    tuples = [speaker + element for element in all_sentences[key]]
    conv_subs.append(tuples)
all_sentences = conv_subs[1][:100] + conv_subs[0][:100] #process first 200 sentences

print(all_sentences)
'''

gt_labels = []
if(os.path.exists(os.path.join(path,filename_labelled_sentences))):    #load data already processed
    all_real_labels = pickle.load(open(os.path.join(path,filename_labelled_sentences),'rb'))
else:
    print("invalid real labels path")

filename_half_corpus = "processed_sentences_half_corpus.pkl"

if(os.path.exists(os.path.join(path,filename_half_corpus))):    #load data already processed
    print("File already exists...")
    half_corpus = pickle.load(open(os.path.join(path,filename_random),'rb'))
    print(half_corpus[198])
else:
    half_corpus_labels = process_results_half_corpus(sentences = all_sentences, new_th=0.2)
    print(half_corpus_labels[197])
    print("Saving file...")
    pickle.dump(half_corpus_labels , open(os.path.join(path,filename_half_corpus),'wb'))

In [ ]:
### RANDOMIZE SENTENCES ORDER FOR LABELLING

import os
import pickle
import random
path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
filename_sentences = "sentences_gpt_300_old.pkl"
filename_sentences_randomized = "sentences_gpt_randomized_old.pkl"

if(os.path.exists(os.path.join(path,filename_sentences))):    #load data already processed
    all_sentences = pickle.load(open(os.path.join(path,filename_sentences),'rb'))
else:
    print("Invalid path")

n_sentences = len(all_sentences)
print(all_sentences)
#print([i for i in range(n_sentences)])
n_shuffles = 5 #how many times we want to shuffle the whole set of sentences (if sentences are 500, then we will have a set of 500 * 3 sentences randomly ordered)
new_sentences_list = []
for i in range(n_shuffles):
    indexes = [i for i in range(n_sentences)]
    shuffled_indexes = random.shuffle(indexes)
    new_sentences_list.extend([all_sentences[i] for i in indexes])
pickle.dump(new_sentences_list,  open(os.path.join(path,filename_sentences_randomized),'wb'))
print(len(new_sentences_list))


In [ ]:
import tkinter as tk

root = tk.Tk()
root.mainloop()

In [ ]:
import tkinter as tk
import os
import pickle
import ctypes
import textwrap
from tkinter.font import Font
import math
#import screeninfo


class WordLabelingGUI:
    def __init__(self, sentence, labels, path, sentence_index, local_path):

        self.path = path
        self.local_path = local_path
        self.sentence_index = sentence_index
        self.sentence = sentence
        self.labels = labels
        self.completed_labels = []
        self.completed_labels_local = []
        self.word_buttons = []
        self.label_buttons = []
        self.n_words = len(self.sentence.split())
        self.n_labels = len(labels)
        self.n_words_inline = 10
        self.n_labels_inline = 7
        self.height_increment = 60
        self.height_increment_rows = 30
        self.actual_height = self.height_increment_rows
        n_word_rows = math.ceil(self.n_words/self.n_words_inline)
        n_label_rows = math.ceil(self.n_labels/self.n_labels_inline)

        #self.word_button_spaces_2 = [1/self.n_words_inline if i < n_word_rows-1 else 1/(self.n_labels - self.n_labels_inline * i) for i in range(n_word_rows)]
        self.root = tk.Tk()
        #tmp_objective_start = tk.Label(self.root, font=("Helvetica",13,"bold"), fg = "black", text="Sentence that you have to label to label:  " )
        #objective_width = tmp_objective_start.winfo_reqwidth()
        #tmp_objective_end = tk.Label(self.root, font=("Helvetica",13,"bold"), fg = "darkblue", text = sentence, wraplength = self.window_width-5-objective_width)
        #tmp_objective_height = tmp_objective_end.winfo_reqheight()
        self.window_height = int((n_word_rows + n_label_rows + 2.5) * self.height_increment_rows + 8.5 * self.height_increment)
        self.window_width = int(self.window_height * 1.5)
        self.word_button_spaces = [self.window_width / (self.n_words_inline + 1) if i < n_word_rows-1 else
                                   self.window_width / ((self.n_words - self.n_words_inline * i) + 1)  for i in range(n_word_rows)]
        self.label_button_spaces = [self.window_width / (self.n_labels_inline + 1) if i < n_label_rows-1 else
                                   self.window_width / ((self.n_labels - self.n_labels_inline * i) + 1)  for i in range(n_label_rows)]
        #scaling_factor = self.root.tk.call("tk", "scaling")
        #self.root.tk.call("tk", "scaling", scaling_factor)
        self.root.resizable(False, False)
        scaling = self.root.tk.call('tk', 'scaling')
        #screen_width = int(self.root.winfo_screenwidth() * scaling)
        #screen_height = int(self.root.winfo_screenheight() * scaling)
        #print(screen_height,screen_width)
        screen_width = 1920
        screen_height = 1080
        #screen_info = self.root.winfo_screenmmwidth(screen=1), self.root.winfo_screenmmheight(screen=1)
        self.root.update_idletasks()
        #screen_width  = self.root.winfo_screenwidth()
        #screen_height = self.root.winfo_screenheight()
        #print(screen_width, screen_height)
        self.center_x = int(screen_width/2) #- self.window_width/2)
        self.center_y = int(screen_height/2)# - self.window_height/2)
        if self.window_height < 400:
            self.window_height = 400
            self.window_width = int(self.window_height * 1.5)
        self.root.geometry(f'{self.window_width}x{self.window_height}+{self.center_x}+{self.center_y-35}')
        #print(self.root.winfo_width())
        self.word_label = tk.Label(self.root, font=("Helvetica",16,"bold"), text="Select words that you want to label:")
        self.word_label.place(relwidth=1, y = self.actual_height)
        self.all_sentence_labels = []
        self.final_result = {"sentence":self.sentence, "labels,word_indexes":[], "sentence_index":self.sentence_index}
        null_image = tk.PhotoImage(width=0, height=0)
        self.words_font = Font(family="Helvetica", size=13, weight="bold")
        self.height_exceeded = False

        y = self.actual_height
        index = -1
        #self.textbox_words_height = y + (n_word_rows+2) * self.height_increment_rows
        #self.textbox_label_height = int(self.textbox_words_height + 1.5 * self.height_increment_rows)

        for i,word in enumerate(self.sentence.split()):
            button_name = (word,i)
            button = tk.Button(self.root, text=word, command=lambda b=button_name: self.choose_word(b), font=("Arial", 10, "bold"), fg = 'black', borderwidth=3)
            button_copy = tk.Button(self.root, text=word, image=null_image, command=lambda b=button_name: self.choose_word(b), compound="center", font=("Arial", 10, "bold"), fg = 'black', borderwidth=3)
            #button_create = tk.Tk()
            #button_create.withdraw()
            button_width = button_copy.winfo_reqwidth()
            #button_create.destroy()
            #print("button_width:",button_width)
            if i % self.n_words_inline == 0:
                if i == 0:
                    self.actual_height += self.height_increment_rows/2
                index +=  1
                self.actual_height += self.height_increment_rows
                y = self.actual_height
            button.place(x = self.word_button_spaces[index] * (i+1-index*self.n_words_inline) - button_width/2, y=y)
            self.word_buttons.append(button)
            #self.root.update()
            #self.start_word_button += button.winfo_x()
            #print(button.winfo_y(),button.winfo_width())

        self.actual_height += self.height_increment
        self.label_label = tk.Label(self.root, font=("Helvetica",16,"bold"), text="Select a label:")
        self.label_label.place(relwidth=1, y = self.actual_height)
        #self.actual_height += self.height_increment
        y = self.actual_height
        index = -1

        for i,label in enumerate(self.labels):
            button = tk.Button(self.root, text=label, command=lambda l=label: self.choose_label(l), font=("Arial", 10, "bold"), fg = 'blue', borderwidth=3)
            button_copy = tk.Button(self.root, text=label, image=null_image, command=lambda l=label: self.choose_label(l), compound="center", font=("Arial", 10,"bold"), fg = 'blue', borderwidth=3)
            #button_create = tk.Tk()
            #button_create.withdraw()
            button_width = button_copy.winfo_reqwidth()
            #button_create.destroy()
            #print("button_width:",button_width)
            if i % self.n_labels_inline == 0:
                if i == 0:
                    self.actual_height += self.height_increment_rows/2
                self.actual_height += self.height_increment_rows
                index +=  1
                y = self.actual_height
            button.place(x = self.label_button_spaces[index] * (i+1-index*self.n_labels_inline) - button_width/2, y=y)
            #self.start_label_button += 100
            self.label_buttons.append(button)
            #button.pack(side='left',ipadx=30,ipady=30)



        self.actual_height += self.height_increment
        final_button_space = self.window_width / 5


        self.reset_button = tk.Button(self.root, text="Reset word \n selection", command=self.reset, font=("Arial", 12, "bold"), fg = 'red', borderwidth=3)
        button_copy =  tk.Button(self.root, image=null_image, compound="center", text="Reset word \n selection", command=self.restart, padx=20, pady=10,font=("Arial", 12, "bold"), fg = 'red', borderwidth=3)
        button_width = button_copy.winfo_reqwidth()
        button_height = button_copy.winfo_reqheight()
        self.reset_button.place(x = final_button_space - button_width/2, y = self.actual_height, width=button_width,height=button_height)


        self.submit_button = tk.Button(self.root, text="Submit", command=self.submit_labels, font=("Arial", 12, "bold"), fg = 'red', borderwidth=3)
        self.submit_button.place(x = final_button_space * 2 - button_copy.winfo_reqwidth()/2, y = self.actual_height, width=button_width,height=button_height)

        self.restart_button = tk.Button(self.root, text="Restart", command=self.restart, font=("Arial", 12, "bold"), fg = 'red', borderwidth=3)
        self.restart_button.place(x = final_button_space * 3 - button_width/2, y = self.actual_height, width=button_width,height=button_height)

        self.finish_button = tk.Button(self.root, text="Finish", command=self.finish, font=("Arial", 12, "bold"), fg = 'red', borderwidth=3)
        self.finish_button.place(x = final_button_space * 4 - button_width/2, y = self.actual_height, width=button_width,height=button_height)

        self.selected_words = []
        self.selected_indexes = []
        self.selected_label = ""

        self.objective_start = tk.Label(self.root, font=("Helvetica",13,"bold"), fg = "black", text="Sentence that you have to label:  " )
        objective_width = self.objective_start.winfo_reqwidth()
        self.objective_end = tk.Label(self.root, font=("Helvetica",13,"bold"), fg = "darkblue", text = sentence, wraplength = self.window_width-5-objective_width)
        objective_height = self.objective_end.winfo_reqheight()
        self.actual_height = self.actual_height +self.height_increment * 1.5 #self.height_increment_rows/2
        self.textbox_words_height = int(self.actual_height + objective_height + self.height_increment_rows/2)
        self.textbox_label_height = int(self.textbox_words_height + 1.0 * self.height_increment_rows)
        self.textbox_final_height = int(self.textbox_label_height + 1.0 * self.height_increment_rows)
        self.last_word_index=-1 #index used to indicate the word from which we can start the selection
        self.word_labelled_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Here there will be shown the selected words")
        self.word_labelled_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
        self.label_chosen_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Here there will be shown the selected label")
        self.label_chosen_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
        self.final_labels_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="The words that you already labelled are:")
        self.final_width = self.final_labels_start.winfo_reqwidth()
        self.final_labels_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
        self.objective_start.place(x=5, y = self.actual_height)
        self.objective_end.place(x=6+objective_width, y=self.actual_height)
        self.word_labelled_start.place(x=5, y = self.textbox_words_height)
        self.label_chosen_start.place(x=5, y = self.textbox_label_height)
        self.final_labels_start.place(x=5, y = self.textbox_final_height)


    def choose_word(self, button_name):
        #self.root.update()
        button_index = button_name[1]
        word = button_name[0]
        #print(self.selected_words, self.selected_words[-1][1])
        if len(self.selected_indexes)>0:
            if button_index - self.selected_indexes[-1] > 1:    #if the word that the user choses in not subsequent to the one that he choses before,
                                                                 # put all buttons in the normal state and clear the selected_buttons list
                for button in self.word_buttons[self.last_word_index+1:]:
                    button.config(state='normal')
                    self.word_labelled_start.destroy()
                    self.word_labelled_end.destroy()
                    self.word_labelled_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="You didn't select successive words! Please repeat the selection...")
                    self.word_labelled_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
                    self.word_labelled_start.place(x=5, y = self.textbox_words_height)
                self.selected_words=[]
                self.selected_indexes=[]
                return
            if len(self.selected_indexes) > 10:
                for button in self.word_buttons[self.last_word_index+1:]:
                    button.config(state='normal')
                    self.word_labelled_start.destroy()
                    self.word_labelled_end.destroy()
                    self.word_labelled_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="You can't label more than 10 words at a time! Please repeat the selection...")
                    self.word_labelled_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
                    self.word_labelled_start.place(x=5, y = self.textbox_words_height)
                self.selected_words=[]
                self.selected_indexes=[]
                return

        self.word_labelled_start.destroy()
        self.word_labelled_end.destroy()
        self.selected_words.append(word)
        self.selected_indexes.append(button_index)
        self.word_labelled_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Selected words:   ")
        self.word_labelled_end = tk.Label(self.root, font=("Helvetica",13,"bold"), fg="darkblue", text=" ".join(self.selected_words))
        self.word_labelled_start.place(x=5, y = self.textbox_words_height)
        word_labelled_width = self.word_labelled_start.winfo_reqwidth()
        self.word_labelled_end.place(x=6+word_labelled_width, y = self.textbox_words_height)
        for i,button in enumerate(self.word_buttons):
            if i<=button_index:
            #if button['text'] in self.selected_words:
                button.config(state='disabled')

    def reset(self):
        for button in self.word_buttons[self.last_word_index+1:]:
            button.config(state='normal')
            self.word_labelled_start.destroy()
            self.word_labelled_end.destroy()
            self.word_labelled_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Resetting word selection...")
            self.word_labelled_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
            self.word_labelled_start.place(x=5, y = self.textbox_words_height)
        self.selected_words=[]
        self.selected_indexes=[]

    def restart(self):
        self.word_labelled_start.destroy()
        self.word_labelled_end.destroy()
        self.final_labels_start.destroy()
        self.final_labels_end.destroy()
        self.label_chosen_start.destroy()
        self.label_chosen_end.destroy()
        self.all_sentence_labels = []
        self.final_result["labels,word_indexes"] = []
        self.selected_words=[]
        self.selected_indexes=[]
        self.last_word_index=-1
        self.selected_label = ""
        self.final_labels_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="All the labelled words were removed")
        self.final_labels_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
        self.final_labels_start.place(x=5, y = self.textbox_final_height)
        self.word_labelled_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Restart the selection of words")
        self.word_labelled_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
        self.word_labelled_start.place(x=5, y = self.textbox_words_height)
        self.label_chosen_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Restart the selection of the label")
        self.label_chosen_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
        self.label_chosen_start.place(x=5, y = self.textbox_label_height)
        for button in self.word_buttons:
            button.config(state='normal')
        for button in self.label_buttons:
            button.config(state='normal')

    def finish(self):
        if(os.path.exists(self.path)):    #load data already processed
            self.completed_labels = pickle.load(open(self.path,'rb'))
        if(os.path.exists(self.local_path)):    #load data already processed
            self.completed_labels_local = pickle.load(open(self.local_path,'rb'))
            #print(self.completed_labels)
        self.final_result["labels,word_indexes"] = self.all_sentence_labels
        print(self.final_result)
        self.completed_labels.append(self.final_result)
        self.completed_labels_local.append(self.final_result)
        #pickle.dump(self.completed_labels, open(self.path,'wb'))
        #pickle.dump(self.completed_labels_local, open(self.local_path,'wb'))
        self.root.destroy()


    def choose_label(self, label):
        self.selected_label = label
        self.label_chosen_start.destroy()
        self.label_chosen_end.destroy()
        self.label_chosen_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Label:  ")
        label_width = self.label_chosen_start.winfo_reqwidth()
        self.label_chosen_end = tk.Label(self.root, font=("Helvetica",13,"bold"), fg = "darkblue", text=label)
        self.label_chosen_start.place(x=5, y = self.textbox_label_height)
        self.label_chosen_end.place(x=6+label_width, y = self.textbox_label_height)
        for button in self.label_buttons:
            if button['text'] == self.selected_label:
                button.config(state='disabled')
            else:
                button.config(state='normal')

    def submit_labels(self):
        if len(self.selected_words) > 0 and self.selected_label != "":
            self.final_labels_start.destroy()
            self.final_labels_end.destroy()
            self.word_labelled_start.destroy()
            self.word_labelled_end.destroy()
            self.label_chosen_start.destroy()
            self.label_chosen_end.destroy()
            all_words = ' '.join([word for word in self.selected_words])
            #print(all_words)
            self.last_word_index = self.selected_indexes[-1]
            label_dict = {all_words:self.selected_label}
            self.all_sentence_labels.append((label_dict, self.selected_indexes))
            #print(self.all_sentence_labels)
            self.final_labels_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="The words that you already labelled are:")
            self.final_labels_start.place(x=5, y = self.textbox_final_height)
            if self.height_exceeded == False:
                self.tmp_sentence_labels = [el[0] for el in self.all_sentence_labels]
            else:
                self.tmp_sentence_labels.append(self.all_sentence_labels[-1][0])
            self.final_labels_end = tk.Label(self.root, text=' '.join(f"{self.tmp_sentence_labels}"),
                                             wraplength=self.window_width - 5 - self.final_width, justify="left", fg = "darkblue", font=("Helvetica",13,"bold"))
            final_labels_height = self.final_labels_end.winfo_reqheight()
            if self.window_height - (self.textbox_final_height + final_labels_height + self.height_increment_rows) < 0:
                self.height_exceeded = True
                self.tmp_sentence_labels.pop(0)
                self.final_labels_end = tk.Label(self.root, text= "..." + ' '.join(f"{self.tmp_sentence_labels}"),
                                             wraplength=self.window_width - 5 - self.final_width, justify="left", fg = "darkblue", font=("Helvetica",13,"bold"))
            self.final_labels_end.place(x=6+self.final_width, y = self.textbox_final_height)
            self.word_labelled_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Here there be shown the words that you are selecting for the labelling")
            self.word_labelled_end = tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
            self.label_chosen_start = tk.Label(self.root, font=("Helvetica",13,"bold"), text="Here there be shown the label selected for the words")
            self.label_chosen_end= tk.Label(self.root, font=("Helvetica",13,"bold"), text="")
            self.word_labelled_start.place(x=5, y = self.textbox_words_height)
            self.label_chosen_start.place(x=5, y = self.textbox_label_height)
            #label_dict = {word: self.selected_label for word in self.selected_words}
            #print(label_dict)
            self.selected_words = []
            self.selected_indexes = []
            self.selected_label = ""
            for button in self.label_buttons:
                button.config(state='normal')
            #for button in self.word_buttons:
            #    if button['text'] not in self.selected_words:
            #        button.config(state='normal')

def continueGUI(n_experiment):

    def on_yes_click():
        answer.set("Yes")
        root.destroy()

    def on_no_click():
        answer.set("No")
        root.destroy()

    def finish_click():
        root.destroy()


    root = tk.Tk()
    root.title(f"SENTENCE NUMBER:  {n_experiment}  of  30")
    #root.geometry("300x100")
    window_width = 400
    window_height = 100
    screen_width = 1920
    screen_height = 1080
    root.update_idletasks()
    center_x = int(2560 + screen_width/2 - window_width/2)
    center_y = int(screen_height/2 - window_height/2)
    root.geometry(f'{window_width}x{window_height}+{center_x}+{center_y-35}')

    if n_experiment == 30:
        question_label = tk.Label(root, text="You finished the experiment!", fg = "black", font=("Helvetica",13,"bold"))
        question_label.pack(pady=10)
        button_frame = tk.Frame(root)
        button_frame.pack()
        finish_button = tk.Button(button_frame, text="OK", width=10, command=finish_click, font=("Arial", 12, "bold"), fg = 'red',  borderwidth=3)
        finish_button.pack(side="left", padx=5)
        root.mainloop()
        return True



    question_label = tk.Label(root, text="Do you want to continue?", fg = "black", font=("Helvetica",14,"bold"))
    question_label.pack(pady=10)

    button_frame = tk.Frame(root)
    button_frame.pack()

    answer = tk.StringVar()

    yes_button = tk.Button(button_frame, text="Yes", width=10, command=on_yes_click, font=("Arial", 12, "bold"), fg = 'red',  borderwidth=3)
    yes_button.pack(side="left", padx=5)

    no_button = tk.Button(button_frame, text="No", width=10, command=on_no_click, font=("Arial", 12, "bold"), fg = 'red',  borderwidth=3)
    no_button.pack(side="left", padx=5)

    root.mainloop()
    print("Answer:", answer.get())
    return answer.get()


if __name__ == "__main__":
    path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
    #filename_labelled_sentences = "labelled_sentences_final.pkl" #sentences already labelled
    filename_labelled_sentences = "labelled_sentences_final.pkl" # "labelled_sentences_final.pkl"
    #filename_best = "new_all_processed_sentences_best.pkl"
    filename_sentences_to_label =  "sentences_gpt_randomized.pkl" #"sentences_gpt_randomized.pkl"
    #ctypes.windll.shcore.SetProcessDpiAwareness(2) # option for high resolution screens
    #ctypes.windll.user32.SetProcessDPIAware()

    if(os.path.exists(os.path.join(path,filename_sentences_to_label))):    #load data already processed
        all_sentences_to_label = pickle.load(open(os.path.join(path,filename_sentences_to_label),'rb'))
    else:
        print("Invalid path")

    #n_people = 30 #number of people that will participate to the experiment
    n_sentences = len(all_sentences_to_label)
    n_sentences_for_each_person = math.floor(30)
    experiment_number = 20   #####  13
    labeller_name = "Test"
    start_index = experiment_number * n_sentences_for_each_person
    end_index = (experiment_number + 1) * n_sentences_for_each_person
    sentences_to_label = all_sentences_to_label[start_index:end_index]

    #sentences_to_label = []
    sentences_already_labelled = []
    if(os.path.exists(os.path.join(path,filename_labelled_sentences))):    #load data already processed
        all_labels =  pickle.load(open(os.path.join(path,filename_labelled_sentences),'rb'))
        #print(all_labels)
        for label in all_labels:
            #print(label)
            sentences_already_labelled.append(label['sentence_index'])
    #print(sentences_already_labelled[3:])
    #print(all_labels)
    #for el in sentences_to_label:
        #print(el)
        #print(sentence_labels.keys())
        #sentence = el['whole sentence,speaker,turn'][0]
        #sentences_to_label.append(sentence) ####sentence
        #print("here")
        #speaker = el['whole sentence,speaker,turn'][1]
        #turn = el['whole sentence,speaker,turn'][2]
    labels = topic_gesture_description.keys()
    #finish_flag = False

    for ind,sentence in enumerate(sentences_to_label):
        #print(sentence,ind)
        index = n_sentences_for_each_person * experiment_number + ind
        print(index)
        if index in sentences_already_labelled:
            continue
        gui = WordLabelingGUI(sentence, labels, os.path.join(path,filename_labelled_sentences),index,
                              os.path.join(path,labeller_name+"_"+filename_labelled_sentences))
        #print("here")
        gui.root.mainloop()
        gui_2 = continueGUI(ind + 1)
        if gui_2 == True:
            break
        if gui_2 == "Yes":
            continue
        else:
            continue
    print("Experiment finished")

In [ ]:
### VISUALIZE SOME LABELS

path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
filename_labelled_sentences = "labelled_sentences_final.pkl" #it contains the ground truth labels
labeller_name = "Claudio_Del_Gaizo"
filename_local_labelled_sentences = labeller_name + "_" + filename_labelled_sentences
if(os.path.exists(os.path.join(path,filename_labelled_sentences))):    #load data already processed
    all_real_labels = pickle.load(open(os.path.join(path,filename_labelled_sentences),'rb'))
else:
    print("invalid real labels path")
#for i in range(len(all_real_labels)):
#    print(all_real_labels[i]['sentence'], "\n \n")
#print(all_real_labels)

if(os.path.exists(os.path.join(path,filename_local_labelled_sentences))):    #load data already processed
    local_real_labels = pickle.load(open(os.path.join(path,filename_local_labelled_sentences),'rb'))
else:
    print("invalid real labels path")

print(all_real_labels)
#pickle.dump(all_real_labels[:240], open(os.path.join(path,filename_labelled_sentences),'wb'))
#pickle.dump(all_real_labels[:420],open(os.path.join(path,filename_labelled_sentences),'wb'))


In [ ]:
### EVALUATION

import os, pickle


def precision_recall_curve(y_true, pred_scores, thresholds):
    precisions = []
    recalls = []

    for threshold in thresholds:
        y_pred = [1 if score >= threshold else 0 for score in pred_scores]

        precision = sklearn.metrics.precision_score(y_true=y_true, y_pred=y_pred, pos_label=1, zero_division=0)
        recall = sklearn.metrics.recall_score(y_true=y_true, y_pred=y_pred, pos_label=1, zero_division=0)

        precisions.append(precision)
        recalls.append(recall)

    return precisions, recalls

def mean_IOU(gt_labels, pd_labels, confidence = 0.5):
    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    topics_iou = {topic_gesture_description[key][0]: {'tot_intersection':0, 'tot_union':0, 'iou':0} for key in gt_labels_names}
    total_positive_topics = {topic_name: 0 for topic_name in topic_names}
    #topics_gt_labels = {topic_gesture_description[key][0]: [] for key in gt_labels_names}
    #for gt_result,pd_result in zip(gt_labels,pd_labels): #for each sentence
    #    assert(gt_result['sentence_index']==pd_result['sentence_index'])
    for gt_result in gt_labels:
        break_flag = False
        for pd_result in pd_labels:
            if pd_result['sentence'] != gt_result['sentence']: #check that the results are referred to the same sentence before continuing
                continue
            else:
                #print(pd_result['sentence'], gt_result['sentence'])
                for topic in topic_names:
                    all_gt_indexes = []
                    all_pd_indexes = []
                    for j,gt_match in enumerate(gt_result['labels,word_indexes']):
                        gt_topic = topic_gesture_description[[el for el in gt_match[0].items()][0][1]][0] #extract the right topic from the gt label
                        if gt_topic != topic:
                            continue
                        else:
                            all_gt_indexes.append(gt_match[1])
                    for k,pd_match in enumerate(zip(pd_result['topics&ref'],pd_result['word_indexes'])):
                        pd_topic = pd_match[0][0]
                        if pd_topic != topic or pd_result['values'][k] < confidence: #the result must respect the confidence threshold
                            continue
                        else:
                            all_pd_indexes.append(pd_match[1])
                    union = len(set([j for sub in all_gt_indexes + all_pd_indexes for j in sub])) #total union in the sentence
                    alL_intersections = [set(j).intersection(k) for j in all_pd_indexes for k in all_gt_indexes if len(set(j).intersection(k))!=0] #all the possible intersections
                    intersection = len(set([i for a in alL_intersections for i in a]))  #total intersection area in the sentence
                    topics_iou[topic]['tot_intersection'] += intersection
                    topics_iou[topic]['tot_union'] += union
                    if union == 0:
                        continue
                    else:
                        topics_iou[topic]['iou'] = topics_iou[topic]['tot_intersection'] / topics_iou[topic]['tot_union']
                break

    total_considered_topics = 0
    for gt_result in gt_labels: #count how many gt labels we have for each topic. Will be considered only the topics that have at least one gt label
        for j,gt_label in enumerate(gt_result['labels,word_indexes']):
            gt_topic = topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
            total_positive_topics[gt_topic] += 1
    #for topic in topic_names:
    #    if total_positive_topics[topic] > 0:
    #        total_considered_topics += 1

    iou_sum = 0
    for topic in topic_names: #compute mean_iou
        if total_positive_topics[topic] > 0:
            total_considered_topics += 1
            #print("TOPIC NAME:", topic, "TOPIC_IOU:",topics_iou[topic]['iou'])
            iou_sum += topics_iou[topic]['iou']
    mean_IOU = iou_sum / total_considered_topics
    #print("MEAN IOU:", mean_IOU)

    return topics_iou, mean_IOU


def mean_average_precision(gt_labels, pd_labels, th_fix = 0.5):
    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    total_positive_topics = {topic_name: 0 for topic_name in topic_names}
    all_results = {topic_name: {th/20: {"Confidences":[],"TP":[], "FP":[], "Acc_TP":[], "Acc_FP":[], "Precision":[], "Recall":[]} for th in range(1,21,1)}
                   for topic_name in topic_names }
    average_precision = {topic_name:{th/20:0 for th in range(1,21,1)} for topic_name in topic_names}
    final_average_precision = {topic_name:0 for topic_name in topic_names}
    total_considered_topics = 0

    for gt_result in gt_labels:
        for j,gt_label in enumerate(gt_result['labels,word_indexes']):
            gt_topic = topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
            total_positive_topics[gt_topic] += 1
    for topic in topic_names:
        if total_positive_topics[topic] > 0:
            total_considered_topics += 1
    #print(total_considered_topics)
    for t in range(1,21,1): #compute all the info needed to compute average precision for all the possible thresholds of IOU
        th = t/20
        for topic in topic_names: #for each possible topic
            if total_positive_topics[topic] == 0: #it does not make sense the computation of Average Precision (we can't compute the Recall)
                continue
            row = all_results[topic][th] #we will create rows in the results referred to the chosen topic and th. The final table will contain
                                         #all the info needed to compute average precision for a topic with a given IOU threshold
            acc_TP = 0
            acc_FP = 0
            n_comparisons = 0
            #####for skip, (gt_result,pd_result) in enumerate(zip(gt_labels,pd_labels)): #for each sentence
            #print("LEN:", len(gt_labels))
            for num,gt_result in enumerate(gt_labels):
                #if t==1 and topic == 'r_greet':
                    #print(num)
                #break_flag = False
                for pd_result in pd_labels:
                    if pd_result['sentence'] != gt_result['sentence']: #check that the results are referred to the same sentence before continuing
                        continue
                    else:
                        #break_flag = True
                        #if t == 1 and topic == 'r_greet':
                        n_comparisons += 1
                        #print("prediction:",pd_result['sentence'])
                        #print("gt:",gt_result['sentence'])
                        #n_comparisons += 1
                        #print(n_comparisons)
                        ######assert(gt_result['sentence_index']==pd_result['sentence_index'])
                        #all_gt_indexes = []
                        #all_pd_indexes = []
                        detected_gt = {"IOU":[], "Confidence":[], 'gt_indexes':[]}
                        for k,pd_match in enumerate(zip(pd_result['topics&ref'],pd_result['word_indexes'])):
                            pd_topic = pd_match[0][0]  #extract the pd_topic from the pd_label   #do the same for the predicted label
                            if pd_topic != topic:
                                #print("SKIPPING PREDICTED:", k, pd_result)
                                continue
                            else: #right topic, check the matches with ground truths
                                #print("NOT SKIPPED PD TOPIC",skip,pd_result)
                                n_intersections = 0
                                tmp_IOU = 0
                                tmp_conf = 0
                                for j,gt_match in enumerate(gt_result['labels,word_indexes']):  #for each prediction, we take only the GT that has highest IOU
                                                                                                #the other can be False Negatives or True positives
                                    gt_topic = topic_gesture_description[[el for el in gt_match[0].items()][0][1]][0] #extract the right topic from the gt label
                                    if gt_topic != topic: #only if gt_topic and pd_topic are the same we can do the comparison
                                        continue
                                    else:
                                        intersection = len(set(pd_match[1]).intersection(gt_match[1]))
                                        if intersection != 0: #we take append to a list that takes all the possible IOUs. Only the best one IOU is TP, for the other topics we have to check if they have predictions with the others GT
                                            union = len(set(pd_match[1] + gt_match[1]))
                                            IOU = intersection/union
                                            n_intersections += 1
                                            if IOU > tmp_IOU: #this allow to take only the best IOU, the others (with other GT) are false negatives
                                                tmp_conf = pd_result['values'][k]
                                                tmp_IOU = IOU
                                                gt_index = j
                                        else:
                                            continue
                                if n_intersections == 0: #then the prediction has no intersections, so we have a false positive
                                    row["Confidences"].append(pd_result['values'][k])
                                    row["FP"].append(1)
                                    row["TP"].append(0)
                                else: #we have to do another check: for each GT, we have to check which is the predicted topic that has highest IOU
                                    detected_gt["IOU"].append(tmp_IOU) #IOU
                                    detected_gt["Confidence"].append(tmp_conf) #confidence
                                    detected_gt["gt_indexes"].append(gt_index)

                        if len(detected_gt["IOU"]) == 0: #then we have no TP, only FP which is already considered and appended as rows
                            break
                        else:
                            unique_indexes = set(detected_gt["gt_indexes"])
                            for idx in unique_indexes:
                                gt_idx = [i for i, d in enumerate(detected_gt["gt_indexes"]) if d == idx] #extract predictions related to same GT
                                IOUs = [detected_gt["IOU"][i] for i in gt_idx]
                                max_IOU_idx = detected_gt["IOU"].index(max(IOUs))
                                if max(IOUs) < th:
                                    for i in gt_idx:
                                        row["Confidences"].append(detected_gt["Confidence"][i])
                                        row["FP"].append(1)
                                        row["TP"].append(0)
                                else:
                                    for i in gt_idx:
                                        if i == max_IOU_idx: #True Positive
                                            row["Confidences"].append(detected_gt["Confidence"][i])
                                            row["FP"].append(0)
                                            row["TP"].append(1)
                                        else:
                                            row["Confidences"].append(detected_gt["Confidence"][i])
                                            row["FP"].append(1)
                                            row["TP"].append(0)
                        '''
                        if len(detected_gt["IOU"]) == 0:
                            #continue
                            break
                        else:
                            best_prediction_value = max(detected_gt["IOU"])
                            if best_prediction_value < th: #then all the predictions are false positives
                                for i in range(len(detected_gt["IOU"])):
                                    row["Confidences"].append(detected_gt["Confidence"][i])
                                    row["FP"].append(1)
                                    row["TP"].append(0)
                            else:
                                for i in range(len(detected_gt["IOU"])):
                                    if best_prediction_value == detected_gt["IOU"][i]: #True Positive
                                        row["Confidences"].append(detected_gt["Confidence"][i])
                                        row["FP"].append(0)
                                        row["TP"].append(1)
                                    else:
                                        row["Confidences"].append(detected_gt["Confidence"][i])
                                        row["FP"].append(1)
                                        row["TP"].append(0)
                            break
                        '''
            #### Sort all the elements by confidence
            sort_indexes = [i for i, x in sorted(enumerate(row["Confidences"]), key=lambda x: x[1], reverse=True)]
            #print(sort_indexes, row.keys())
            for key in row.keys():
                if key == "Recall" or key == "Precision" or key == "Acc_FP" or key == "Acc_TP":
                    continue
                row[key] = [row[key][i] for i in sort_indexes]
            for i in range(len(row["Confidences"])):
                if row["TP"][i] == 1:
                    acc_TP += 1
                else:
                    acc_FP += 1
                row["Acc_TP"].append(acc_TP)
                row["Acc_FP"].append(acc_FP)
                row["Precision"].append(acc_TP / (acc_TP + acc_FP))
                row["Recall"].append(acc_TP / total_positive_topics[topic])

    ######### COCO EVALUATOR
    mean_average_precision = 0
    #for iou in range(50,100,5):
    #iou = 30
    #th = iou/100
    th = th_fix
    for topic in topic_names:
        if total_positive_topics[topic] > 0:
            topic_results = all_results[topic][th]
            #fig = plt.figure()
            #plt.plot(topic_results["Recall"],topic_results["Precision"])
            #plt.pause(1)
            if len(topic_results["Recall"])<2:
                continue
            AP = sklearn.metrics.auc(topic_results["Recall"], topic_results["Precision"]) * 100
            #print("AP:", AP)
            average_precision[topic][th] = AP
            final_average_precision[topic] += AP
        else:
            continue
    for topic in topic_names:
        #final_average_precision[topic] = final_average_precision[topic] #/ 10 #now we have
        mean_average_precision += final_average_precision[topic]
    mean_average_precision = mean_average_precision/total_considered_topics
    print("n_comparisons:", n_comparisons)
    #print(round(mean_average_precision,2),"%")

    return average_precision, mean_average_precision

if __name__ == "__main__":

    path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
    names = ['Alice','Antonio','Carmine_nuove', 'Lucrezia']#,'Carmine_nuove', 'Lucrezia']
    names_random = ['Alice_random','Antonio_random','Carmine_nuove_random', 'Lucrezia_random']#
    model_types = ["base","large"]
    analysis_types = ["results_moving_windows","results_priority_policy_best_windows"]
    analysis_random = ["results_random_final_0_3.pkl","results_random_final_old_0_3.pkl"]
    gt_new = "labelled_sentences_final.pkl"
    gt_old = "labelled_sentences_final_old.pkl"
    processed_results = ''
    all_labels = []
    #filename_best = "new_all_processed_sentences_best.pkl" #it contains all the possible results; a results is the best words match for each topic for each possible group of words inside the sentence, for all the sentences
    #filename_best = "all_similarities_600.pkl"
    #filename_random = "processed_sentences_random.pkl" #it contains all the random labels
    #filename_labelled_sentences = "labelled_sentences_final.pkl" #it contains the ground truth labels
    #filename_labelled_sentences_old = "labelled_sentences_final_old.pkl" #it contains the ground truth labels
    #filename_labelled_sentences = "Marco_Limone_labelled_sentences_final.pkl"
    #filename_fixed_window_labels = "results_priority_policy_best_windows.pkl" #it contains the labelled sentences obtained by applying the average window approach
    #filename_moving_window_labels = "results_moving_window_policy.pkl" #it contains the labelled sentences obtained by applying the moving window approach
    AP_sum = []
    '''
    array_a = []
    array_b = []
    tmp_1 = []
    tmp_2 =[]
    tmp_val_1 = 0
    tmp_val_2 = 0
    real_labels = pickle.load(open(os.path.join(path,gt_new),'rb')) if os.path.exists(os.path.join(path,gt_new)) else print("unable to load GT")
    real_labels_old = pickle.load(open(os.path.join(path,gt_old),'rb')) if os.path.exists(os.path.join(path,gt_old)) else print("unable to load GT old")
    for name in names:
        filename = analysis_types[0]+'_'+name+'_'+model_types[0]+"_old_0_3.pkl"
        if(os.path.exists(os.path.join(path,filename))):
            tmp_1 = (pickle.load(open(os.path.join(path,filename),'rb')))
        else:
            print("unable to find the file:  ",filename)
        #filename = analysis_types[1]+'_'+name+'_'+model_types[0]+"_old_0_3.pkl"
        filename = analysis_random[1]
        if(os.path.exists(os.path.join(path,filename))):
            tmp_2 = (pickle.load(open(os.path.join(path,filename),'rb')))
        else:
            print("unable to find the file:  ",filename)
        _, tmp_val_1 = mean_average_precision(real_labels_old, tmp_1, th_fix=0.5)
        _, tmp_val_2 = mean_average_precision(real_labels_old, tmp_2, th_fix=0.5)
        array_a.append(tmp_val_1)
        array_b.append(tmp_val_2)
    res = scipy.stats.ttest_ind(array_a,array_b)
    print(res)
    '''

    for name in names_random:
        filename = analysis_types[0]+'_'+name+'_'+model_types[0]+"_old_0_3.pkl"
        #filename = analysis_random[0]
        if(os.path.exists(os.path.join(path,filename))):
            all_labels.append(pickle.load(open(os.path.join(path,filename),'rb')))
        else:
            print("unable to find the file:  ",filename)

    real_labels = pickle.load(open(os.path.join(path,gt_new),'rb')) if os.path.exists(os.path.join(path,gt_new)) else print("unable to load GT")
    real_labels_old = pickle.load(open(os.path.join(path,gt_old),'rb')) if os.path.exists(os.path.join(path,gt_old)) else print("unable to load GT old")

    all_AP = {name:[] for name in names}
    all_IOU = {name:[] for name in names}
    all_mAP = {name:0 for name in names}
    all_mIOU = {name:0 for name in names}
    exec_time = []
    for i,labels in enumerate(all_labels):
        #print(labels[0]['processing_t_1']+)
        for label in labels:
            exec_time.append(label['processing_t_1'] + label['processing_t_2'])
            #exec_time.append(label['processing_time'])
        all_AP[names[i]], all_mAP[names[i]] = mean_average_precision(real_labels_old, labels, th_fix=0.5)
        all_IOU[names[i]], all_mIOU[names[i]] = mean_IOU(real_labels_old, labels,confidence=0.5)
        #all_AP_ml, mAP_ml = mean_average_precision(all_real_labels, moving_window_labels)
        #all_IOU_ml, mIOU_ml = mean_IOU(all_real_labels, moving_window_labels)
    #print("mAP random labels:", round(mAP_rl,2),"   mIOU random labels:",round(mIOU_rl*100,2))
    #for name in names:

    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    #print(topic_names)
    AP_mean = {topic:[] for topic in topic_names}
    mAP_mean = []
    IOU_mean = {topic:[] for topic in topic_names}
    mIOU_mean = []


    for name in names:
        #print(all_AP[name])
        print(all_mAP[name])
        print(all_mIOU[name])
        mAP_mean.append(all_mAP[name])
        mIOU_mean.append(all_mIOU[name]*100)
        for topic in topic_names:
            AP_mean[topic].append(all_AP[name][topic][0.5])
            IOU_mean[topic].append(all_IOU[name][topic]['iou']*100)

    print('mAP_mean:',round(np.mean(mAP_mean),2),'mAP_std:',round(np.std(mAP_mean),2))
    print('mIOU_mean:',round(np.mean(mIOU_mean),2),'mAP_std:',round(np.std(mIOU_mean),2))
    for topic in topic_names:
        print(f'AP_mean {topic}:',round(np.mean(AP_mean[topic]),2),f'AP_std {topic}:',round(np.std(AP_mean[topic]),2))
        print(f'IOU_mean {topic}:',round(np.mean(IOU_mean[topic]),2),f'mAP_std {topic}:',round(np.std(IOU_mean[topic]),2))
    print("MEAN TIME:", round(np.mean(exec_time),2), "STD TIME:", round(np.std(exec_time),2))


    #print(np.mean(mAP_mean),np.AP_mean)
    #print(IOU_mean, mIOU_mean)
    #for topic in topic_names:
    #    print()
        #print(f" mAP {name}",round(all_mAP[name],2))
        #print(f" mIOU {name}",round(all_mIOU[name]*100,2))
    #AP_MEAN = np.mean(np.array(AP_mean))
    #print(AP_MEAN[0.5])
        #for topic_name in topic_names:
        #    print(f" AP {name} {topic_name}",round(all_AP[name][topic_name][0.3],2))
        #    print(f" IOU {name} {topic_name}",round(all_IOU[name][topic_name]['iou']*100,2))

    #print("mAP fixed window labels:", round(mAP_fl,2),"     mIOU fixed window labels:",round(mIOU_fl*100,2))
    #print("mAP moving window labels:", round(mAP_ml,2),"    mIOU moving window labels:",round(mIOU_ml*100,2))
    '''
    if(os.path.exists(os.path.join(path,filename_random))):    #load data already processed
        all_random_labels = pickle.load(open(os.path.join(path,filename_random),'rb'))
    else:
        print("Invalid path for random labels")
    '''
    '''
    if(os.path.exists(os.path.join(path,filename_best))):    #load data already processed
        result = pickle.load(open(os.path.join(path,filename_best),'rb'))
    else:
        print("Invalid path for all the results")

    if(os.path.exists(os.path.join(path,filename_labelled_sentences))):    #load data already processed
        all_real_labels = pickle.load(open(os.path.join(path,filename_labelled_sentences),'rb'))
        print("NEW",len(all_real_labels))
    else:
        print("invalid real labels path")

    if(os.path.exists(os.path.join(path,filename_labelled_sentences_old))):    #load data already processed
        all_real_labels_old = pickle.load(open(os.path.join(path,filename_labelled_sentences_old),'rb'))
        print("OLD",len(all_real_labels_old))
    else:
        print("invalid real labels path")

    if(os.path.exists(os.path.join(path,filename_fixed_window_labels))):    #load data already processed
        fixed_window_labels = pickle.load(open(os.path.join(path,filename_fixed_window_labels),'rb'))
    else:
        print("invalid fixed window labels path")

    if(os.path.exists(os.path.join(path,filename_moving_window_labels))):    #load data already processed
        moving_window_labels = pickle.load(open(os.path.join(path,filename_moving_window_labels),'rb'))
    else:
        print("invalid moving window labels path")
    '''
    #print(len(all_real_labels))

    #print(avg_labels[0],"\n",real_labels[0])
    #all_AP_rl, mAP_rl = mean_average_precision(all_real_labels, all_random_labels)
    #all_IOU_rl, mIOU_rl = mean_IOU(all_real_labels, all_random_labels)
    '''
    all_real_labels = all_real_labels + all_real_labels_old
    print(len(all_real_labels))
    all_AP_fl, mAP_fl = mean_average_precision(all_real_labels, fixed_window_labels)
    all_IOU_fl, mIOU_fl = mean_IOU(all_real_labels, fixed_window_labels)
    all_AP_ml, mAP_ml = mean_average_precision(all_real_labels, moving_window_labels)
    all_IOU_ml, mIOU_ml = mean_IOU(all_real_labels, moving_window_labels)
    #print("mAP random labels:", round(mAP_rl,2),"   mIOU random labels:",round(mIOU_rl*100,2))
    print("mAP fixed window labels:", round(mAP_fl,2),"     mIOU fixed window labels:",round(mIOU_fl*100,2))
    print("mAP moving window labels:", round(mAP_ml,2),"    mIOU moving window labels:",round(mIOU_ml*100,2))
    '''
    #print(all_real_labels[0],moving_window_labels[0])
    #print(mAP)
    #print(mIOU*100)
    #print("m_IOU",all_IOU)
    #print("mAP", all_AP)
    #print(sentences_already_labelled[3:])
    #print(avg_labels[0],"\n",all_real_labels[199])
    #for el in result:

In [ ]:
### EVALUATION BETWEEN GT AND GT

import os, pickle


def precision_recall_curve(y_true, pred_scores, thresholds):
    precisions = []
    recalls = []

    for threshold in thresholds:
        y_pred = [1 if score >= threshold else 0 for score in pred_scores]

        precision = sklearn.metrics.precision_score(y_true=y_true, y_pred=y_pred, pos_label=1, zero_division=0)
        recall = sklearn.metrics.recall_score(y_true=y_true, y_pred=y_pred, pos_label=1, zero_division=0)

        precisions.append(precision)
        recalls.append(recall)

    return precisions, recalls

def mean_IOU(gt_labels_1, gt_labels_2, confidence = 0.5):
    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    topics_iou = {topic_gesture_description[key][0]: {'tot_intersection':0, 'tot_union':0, 'iou':0} for key in gt_labels_names}
    total_positive_topics = {topic_name: 0 for topic_name in topic_names}
    #topics_gt_labels = {topic_gesture_description[key][0]: [] for key in gt_labels_names}
    #for gt_result,pd_result in zip(gt_labels,pd_labels): #for each sentence
    #    assert(gt_result['sentence_index']==pd_result['sentence_index'])
    for gt_1, gt_result in enumerate(gt_labels_1):
        break_flag = False
        for gt_2 in range(gt_1+1,len(gt_labels_2)):
            if gt_1 == len(gt_labels_1):
                break
            if gt_labels_2[gt_2]['sentence'] != gt_result['sentence']: #check that the results are referred to the same sentence before continuing
                continue
            else:
                #print(pd_result['sentence'], gt_result['sentence'])
                for topic in topic_names:
                    all_gt_indexes = []
                    all_pd_indexes = []

                    for k,gt_match_1 in enumerate(gt_result['labels,word_indexes']):
                        gt_topic_1 = topic_gesture_description[[el for el in gt_match_1[0].items()][0][1]][0]
                        if gt_topic_1 != topic: #or pd_result['values'][k] < confidence: #the result must respect the confidence threshold
                            continue
                        else:
                            all_pd_indexes.append(gt_match_1[1])

                    for j,gt_match_2 in enumerate(gt_labels_2[gt_2]['labels,word_indexes']):
                        gt_topic_2 = topic_gesture_description[[el for el in gt_match_2[0].items()][0][1]][0] #extract the right topic from the gt label
                        if gt_topic_2 != topic:
                            continue
                        else:
                            all_gt_indexes.append(gt_match_2[1])


                    union = len(set([j for sub in all_gt_indexes + all_pd_indexes for j in sub])) #total union in the sentence
                    alL_intersections = [set(j).intersection(k) for j in all_pd_indexes for k in all_gt_indexes if len(set(j).intersection(k))!=0] #all the possible intersections
                    intersection = len(set([i for a in alL_intersections for i in a]))  #total intersection area in the sentence
                    topics_iou[topic]['tot_intersection'] += intersection
                    topics_iou[topic]['tot_union'] += union
                    if union == 0:
                        continue
                    else:
                        topics_iou[topic]['iou'] = topics_iou[topic]['tot_intersection'] / topics_iou[topic]['tot_union']
                    #print(gt_result)
                    #print(gt_labels_2[gt_2])
                    #print("intersection:",intersection, "union:",union, "topic", topic, "tot_intersection",topics_iou[topic]['tot_intersection'])
                break

    total_considered_topics = 0
    for gt_result in gt_labels_1: #count how many gt labels we have for each topic. Will be considered only the topics that have at least one gt label
        for j,gt_label in enumerate(gt_result['labels,word_indexes']):
            gt_topic = topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
            total_positive_topics[gt_topic] += 1
    #for topic in topic_names:
    #    if total_positive_topics[topic] > 0:
    #        total_considered_topics += 1

    iou_sum = 0
    for topic in topic_names: #compute mean_iou
        if total_positive_topics[topic] > 0:
            total_considered_topics += 1
            #print("TOPIC NAME:", topic, "TOPIC_IOU:",topics_iou[topic]['iou'])
            iou_sum += topics_iou[topic]['iou']
    mean_IOU = iou_sum / total_considered_topics
    #print("MEAN IOU:", mean_IOU)

    return topics_iou, mean_IOU


def mean_average_precision(gt_labels_1, gt_labels_2, th_fix = 0.5):
    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    total_positive_topics = {topic_name: 0 for topic_name in topic_names}
    all_results = {topic_name: {th/20: {"Confidences":[],"TP":[], "FP":[], "Acc_TP":[], "Acc_FP":[], "Precision":[], "Recall":[]} for th in range(1,21,1)}
                   for topic_name in topic_names }
    average_precision = {topic_name:{th/20:0 for th in range(1,21,1)} for topic_name in topic_names}
    final_average_precision = {topic_name:0 for topic_name in topic_names}
    total_considered_topics = 0

    for gt_result in gt_labels_1:
        for j,gt_label in enumerate(gt_result['labels,word_indexes']):
            gt_topic = topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
            total_positive_topics[gt_topic] += 1
    for topic in topic_names:
        if total_positive_topics[topic] > 0:
            total_considered_topics += 1
    #print(total_considered_topics)
    for t in range(1,21,1): #compute all the info needed to compute average precision for all the possible thresholds of IOU
        th = t/20
        for topic in topic_names: #for each possible topic
            if total_positive_topics[topic] == 0: #it does not make sense the computation of Average Precision (we can't compute the Recall)
                continue
            row = all_results[topic][th] #we will create rows in the results referred to the chosen topic and th. The final table will contain
                                         #all the info needed to compute average precision for a topic with a given IOU threshold
            acc_TP = 0
            acc_FP = 0
            n_comparisons = 0
            #####for skip, (gt_result,pd_result) in enumerate(zip(gt_labels,pd_labels)): #for each sentence
            #print("LEN:", len(gt_labels))
            for num,gt_result in enumerate(gt_labels_1):
                #if t==1 and topic == 'r_greet':
                    #print(num)
                #break_flag = False
                for gt_2 in range(num+1,len(gt_labels_2)):
                    if num == len(gt_labels_2):
                        break
                    if gt_labels_2[gt_2]['sentence'] != gt_result['sentence']: #check that the results are referred to the same sentence before continuing
                        continue
                    else:
                        #print(gt_labels_2[gt_2])
                        #print(gt_result)
                        #break_flag = True
                        #if t == 1 and topic == 'r_greet':
                        n_comparisons += 1
                        #print("prediction:",pd_result['sentence'])
                        #print("gt:",gt_result['sentence'])
                        #n_comparisons += 1
                        #print(n_comparisons)
                        ######assert(gt_result['sentence_index']==pd_result['sentence_index'])
                        #all_gt_indexes = []
                        #all_pd_indexes = []
                        detected_gt = {"IOU":[], "Confidence":[], 'gt_indexes':[]}
                        for k,gt_match_2 in enumerate(gt_labels_2[gt_2]['labels,word_indexes']):
                            #print(gt_labels_2[gt_2]['labels,word_indexes'])
                            gt_topic_2 = topic_gesture_description[[el for el in gt_match_2[0].items()][0][1]][0]  #extract the pd_topic from the pd_label   #do the same for the predicted label
                            if gt_topic_2 != topic:
                                #print("SKIPPING PREDICTED:", k, pd_result)
                                continue
                            else: #right topic, check the matches with ground truths
                                #print("NOT SKIPPED PD TOPIC",skip,pd_result)
                                n_intersections = 0
                                tmp_IOU = 0
                                tmp_conf = 0
                                for j,gt_match_1 in enumerate(gt_result['labels,word_indexes']):  #for each prediction, we take only the GT that has highest IOU
                                                                                                #the other can be False Negatives or True positives
                                    gt_topic_1 = topic_gesture_description[[el for el in gt_match_1[0].items()][0][1]][0] #extract the right topic from the gt label
                                    if gt_topic_1 != topic: #only if gt_topic and pd_topic are the same we can do the comparison
                                        continue
                                    else:
                                        intersection = len(set(gt_match_2[1]).intersection(gt_match_1[1]))
                                        if intersection != 0: #we take append to a list that takes all the possible IOUs. Only the best one IOU is TP, for the other topics we have to check if they have predictions with the others GT
                                            union = len(set(gt_match_2[1] + gt_match_1[1]))
                                            IOU = intersection/union
                                            n_intersections += 1
                                            if IOU > tmp_IOU: #this allow to take only the best IOU, the others (with other GT) are false negatives
                                                tmp_conf = 1 #pd_result['values'][k]
                                                tmp_IOU = IOU
                                                gt_index = j
                                        else:
                                            continue
                                if n_intersections == 0: #then the prediction has no intersections, so we have a false positive
                                    row["Confidences"].append(1)
                                    row["FP"].append(1)
                                    row["TP"].append(0)
                                else: #we have to do another check: for each GT, we have to check which is the predicted topic that has highest IOU
                                    detected_gt["IOU"].append(tmp_IOU) #IOU
                                    detected_gt["Confidence"].append(tmp_conf) #confidence
                                    detected_gt["gt_indexes"].append(gt_index)
                        #print(detected_gt)
                        if len(detected_gt["IOU"]) == 0: #then we have no TP, only FP which is already considered and appended as rows
                            break
                        else:
                            unique_indexes = set(detected_gt["gt_indexes"])
                            for idx in unique_indexes:
                                gt_idx = [i for i, d in enumerate(detected_gt["gt_indexes"]) if d == idx] #extract predictions related to same GT
                                IOUs = [detected_gt["IOU"][i] for i in gt_idx]
                                max_IOU_idx = detected_gt["IOU"].index(max(IOUs))
                                if max(IOUs) < th:
                                    for i in gt_idx:
                                        row["Confidences"].append(detected_gt["Confidence"][i])
                                        row["FP"].append(1)
                                        row["TP"].append(0)
                                else:
                                    for i in gt_idx:
                                        if i == max_IOU_idx: #True Positive
                                            row["Confidences"].append(detected_gt["Confidence"][i])
                                            row["FP"].append(0)
                                            row["TP"].append(1)
                                        else:
                                            row["Confidences"].append(detected_gt["Confidence"][i])
                                            row["FP"].append(1)
                                            row["TP"].append(0)
                        '''
                        if len(detected_gt["IOU"]) == 0:
                            #continue
                            break
                        else:
                            best_prediction_value = max(detected_gt["IOU"])
                            if best_prediction_value < th: #then all the predictions are false positives
                                for i in range(len(detected_gt["IOU"])):
                                    row["Confidences"].append(detected_gt["Confidence"][i])
                                    row["FP"].append(1)
                                    row["TP"].append(0)
                            else:
                                for i in range(len(detected_gt["IOU"])):
                                    if best_prediction_value == detected_gt["IOU"][i]: #True Positive
                                        row["Confidences"].append(detected_gt["Confidence"][i])
                                        row["FP"].append(0)
                                        row["TP"].append(1)
                                    else:
                                        row["Confidences"].append(detected_gt["Confidence"][i])
                                        row["FP"].append(1)
                                        row["TP"].append(0)
                            break
                        '''
            #### Sort all the elements by confidence
            sort_indexes = [i for i, x in sorted(enumerate(row["Confidences"]), key=lambda x: x[1], reverse=True)]
            #print(sort_indexes, row.keys())
            for key in row.keys():
                if key == "Recall" or key == "Precision" or key == "Acc_FP" or key == "Acc_TP":
                    continue
                row[key] = [row[key][i] for i in sort_indexes]
            for i in range(len(row["Confidences"])):
                if row["TP"][i] == 1:
                    acc_TP += 1
                else:
                    acc_FP += 1
                row["Acc_TP"].append(acc_TP)
                row["Acc_FP"].append(acc_FP)
                row["Precision"].append(acc_TP / (acc_TP + acc_FP))
                row["Recall"].append(acc_TP / total_positive_topics[topic])

    ######### COCO EVALUATOR
    mean_average_precision = 0
    #for iou in range(50,100,5):
    #iou = 30
    #th = iou/100
    th = th_fix
    for topic in topic_names:
        if total_positive_topics[topic] > 0:
            topic_results = all_results[topic][th]
            #fig = plt.figure()
            #plt.plot(topic_results["Recall"],topic_results["Precision"])
            #plt.pause(1)
            if len(topic_results["Recall"])<2:
                continue
            AP = sklearn.metrics.auc(topic_results["Recall"], topic_results["Precision"]) * 100
            #print("AP:", AP)
            average_precision[topic][th] = AP
            final_average_precision[topic] += AP
        else:
            continue
    for topic in topic_names:
        #final_average_precision[topic] = final_average_precision[topic] #/ 10 #now we have
        mean_average_precision += final_average_precision[topic]
    mean_average_precision = mean_average_precision/total_considered_topics
    print("n_comparisons:", n_comparisons)
    #print(round(mean_average_precision,2),"%")

    return average_precision, mean_average_precision

if __name__ == "__main__":

    path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
    names = ['Alice','Antonio','Carmine_nuove', 'Lucrezia']#,'Carmine_nuove', 'Lucrezia']
    names_random = ['Alice_random','Antonio_random','Carmine_nuove_random', 'Lucrezia_random']#
    model_types = ["base","large"]
    analysis_types = ["results_moving_windows","results_priority_policy_best_windows"]
    analysis_random = ["results_random_final_0_3.pkl","results_random_final_old_0_3.pkl"]
    gt_new = "labelled_sentences_final.pkl"
    gt_old = "labelled_sentences_final_old.pkl"
    processed_results = ''
    all_labels = []
    #filename_best = "new_all_processed_sentences_best.pkl" #it contains all the possible results; a results is the best words match for each topic for each possible group of words inside the sentence, for all the sentences
    #filename_best = "all_similarities_600.pkl"
    #filename_random = "processed_sentences_random.pkl" #it contains all the random labels
    #filename_labelled_sentences = "labelled_sentences_final.pkl" #it contains the ground truth labels
    #filename_labelled_sentences_old = "labelled_sentences_final_old.pkl" #it contains the ground truth labels
    #filename_labelled_sentences = "Marco_Limone_labelled_sentences_final.pkl"
    #filename_fixed_window_labels = "results_priority_policy_best_windows.pkl" #it contains the labelled sentences obtained by applying the average window approach
    #filename_moving_window_labels = "results_moving_window_policy.pkl" #it contains the labelled sentences obtained by applying the moving window approach
    AP_sum = []
    '''
    array_a = []
    array_b = []
    tmp_1 = []
    tmp_2 =[]
    tmp_val_1 = 0
    tmp_val_2 = 0
    real_labels = pickle.load(open(os.path.join(path,gt_new),'rb')) if os.path.exists(os.path.join(path,gt_new)) else print("unable to load GT")
    real_labels_old = pickle.load(open(os.path.join(path,gt_old),'rb')) if os.path.exists(os.path.join(path,gt_old)) else print("unable to load GT old")
    for name in names:
        filename = analysis_types[0]+'_'+name+'_'+model_types[0]+"_old_0_3.pkl"
        if(os.path.exists(os.path.join(path,filename))):
            tmp_1 = (pickle.load(open(os.path.join(path,filename),'rb')))
        else:
            print("unable to find the file:  ",filename)
        #filename = analysis_types[1]+'_'+name+'_'+model_types[0]+"_old_0_3.pkl"
        filename = analysis_random[1]
        if(os.path.exists(os.path.join(path,filename))):
            tmp_2 = (pickle.load(open(os.path.join(path,filename),'rb')))
        else:
            print("unable to find the file:  ",filename)
        _, tmp_val_1 = mean_average_precision(real_labels_old, tmp_1, th_fix=0.5)
        _, tmp_val_2 = mean_average_precision(real_labels_old, tmp_2, th_fix=0.5)
        array_a.append(tmp_val_1)
        array_b.append(tmp_val_2)
    res = scipy.stats.ttest_ind(array_a,array_b)
    print(res)
    '''
    '''
    for name in names_random:
        filename = analysis_types[0]+'_'+name+'_'+model_types[0]+"_old_0_3.pkl"
        #filename = analysis_random[0]
        if(os.path.exists(os.path.join(path,filename))):
            all_labels.append(pickle.load(open(os.path.join(path,filename),'rb')))
        else:
            print("unable to find the file:  ",filename)
    '''
    real_labels = pickle.load(open(os.path.join(path,gt_new),'rb')) if os.path.exists(os.path.join(path,gt_new)) else print("unable to load GT")
    real_labels_old = pickle.load(open(os.path.join(path,gt_old),'rb')) if os.path.exists(os.path.join(path,gt_old)) else print("unable to load GT old")
    IOU, mIOU = mean_IOU(real_labels_old + real_labels, real_labels_old + real_labels,confidence=0.5)
    AP, mAP = mean_average_precision(real_labels_old + real_labels, real_labels_old + real_labels, th_fix=0.5)

    #all_AP = {name:[] for name in names}
    #all_IOU = {name:[] for name in names}
    #all_mAP = {name:0 for name in names}
    #all_mIOU = {name:0 for name in names}
    #exec_time = []
    #for i,labels in enumerate(all_labels):
        #print(labels[0]['processing_t_1']+)
        #for label in labels:
            #exec_time.append(label['processing_t_1'] + label['processing_t_2'])
            #exec_time.append(label['processing_time'])

        #all_AP_ml, mAP_ml = mean_average_precision(all_real_labels, moving_window_labels)
        #all_IOU_ml, mIOU_ml = mean_IOU(all_real_labels, moving_window_labels)
    #print("mAP random labels:", round(mAP_rl,2),"   mIOU random labels:",round(mIOU_rl*100,2))
    #for name in names:

    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    #print(topic_names)
    AP_mean = {topic:[] for topic in topic_names}
    mAP_mean = []
    IOU_mean = {topic:[] for topic in topic_names}
    mIOU_mean = []


    #for name in names:
        #print(all_AP[name])
    #print(mAP)
    #print(mIOU*100)
    #mAP_mean.append(all_mAP[name])
    #mIOU_mean.append(all_mIOU[name]*100)
    #for topic in topic_names:
    #    AP_mean[topic].append(AP[topic][0.5])
    #    IOU_mean[topic].append(IOU[topic]['iou']*100)

    print('mAP:',round(mAP,2)) #,'mAP_std:',round(mAP,2))
    print('mIOU:',round(mIOU*100,2))
    #print('mIOU_mean:',round(np.mean(mIOU_mean),2),'mAP_std:',round(np.std(mIOU_mean),2))
    for topic in topic_names:
        print(f'AP {topic}:',round(AP[topic][0.5],2))#,f'AP_std {topic}:',round(np.std(AP_mean[topic]),2))
        print(f'IOU {topic}:',round(IOU[topic]['iou']*100,2), "tot intersection:",round(IOU[topic]['tot_intersection'],2), "tot union",round(IOU[topic]['tot_union'],2))
        #print(f'IOU_mean {topic}:',round(np.mean(IOU_mean[topic]),2),f'mAP_std {topic}:',round(np.std(IOU_mean[topic]),2))
    #print("MEAN TIME:", round(np.mean(exec_time),2), "STD TIME:", round(np.std(exec_time),2))

In [ ]:
### JUST FOR TESTS


path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
result_name = "all_similarities_Antonio_base_old.pkl"
result = pickle.load(open(os.path.join(path,result_name),'rb'))
print(result[1], list(result[2].keys())[1:-1])
tmp = result[0]
tmp_ind = result[0]['window 1']['start_word_indexes'].index(1)
tmp_ind_2 = result[0]['window 1']['start_word_indexes'].index(2)
print(result[0]['window 1']['value'][tmp_ind:tmp_ind_2])
a=[1,2,3,1,2,1,1,3,4,3,2]
f=[-1,0,1,2,3,4,5,6,7,8,9]
b=[i for i in range(5)]
print(b)
if 3 not in a:
    print('here')
#print(a.index(4))
b = set(a)
for x in b:
    c = [i for i, d in enumerate(a) if d == x]
    print("b",b,c,[f[i] for i in c])

a = [[1,2,3],[4,5],[7,8,9,10], [11,12,13,14,15]]
b = [[2,3], [6,7,8], [9,10], [12,13], [16]]

alL_intersections = [set(j).intersection(k) for j in a for k in b if len(set(j).intersection(k))!=0] #all the possible interesections
intersection = len(set([i for a in alL_intersections for i in a]))  #total intersection area in the sentence
print(alL_intersections, intersection)

In [ ]:
### COMPUTE SOME STATISTICS ON PROCESSED SENTENCES

path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
#filename_best = "new_all_processed_sentences_best.pkl" #it contains all the possible results; a results is the best words match for each topic for each possible group of words inside the sentence, for all the sentences
#filename_best = "all_similarities.pkl"
#filename_random = "processed_sentences_random_300.pkl" #it contains all the random labels
#filename_labelled_sentences = "labelled_sentences_final.pkl" #it contains the ground truth labels
filename_fixed_window_labels = "results_priority_policy_best_windows.pkl" #it contains the labelled sentences obtained by applying the average window approach
filename_moving_window_labels = "results_moving_window_policy.pkl" #it contains the labelled sentences obtained by applying the moving window approach
#filename_random_2 = "processed_sentences_random_300_2.pkl" #it contains all the random labels
#filename_labelled_sentences = "labelled_sentences_final.pkl" #it contains the ground truth labels
filename_fixed_window_labels_old = "results_priority_policy_best_windows_old.pkl" #it contains the labelled sentences obtained by applying the average window approach
filename_moving_window_labels_old = "results_moving_window_policy_old.pkl" #it contains the labelled sentences obtained by applying the moving window approach



if(os.path.exists(os.path.join(path,filename_moving_window_labels))):    #load data already processed
    moving_window_labels = pickle.load(open(os.path.join(path,filename_moving_window_labels),'rb'))
else:
    print("invalid moving window labels path")


if(os.path.exists(os.path.join(path,filename_fixed_window_labels))):    #load data already processed
    fixed_window_labels = pickle.load(open(os.path.join(path,filename_fixed_window_labels),'rb'))
else:
    print("invalid fixed window labels path")

'''
if(os.path.exists(os.path.join(path,filename_random))):    #load data already processed
    all_random_labels = pickle.load(open(os.path.join(path,filename_random),'rb'))
else:
    print("Invalid path for random labels")
'''


if(os.path.exists(os.path.join(path,filename_moving_window_labels_old))):    #load data already processed
    moving_window_labels_old = pickle.load(open(os.path.join(path,filename_moving_window_labels_old),'rb'))
else:
    print("invalid moving window labels path")


if(os.path.exists(os.path.join(path,filename_fixed_window_labels_old))):    #load data already processed
    fixed_window_labels_old = pickle.load(open(os.path.join(path,filename_fixed_window_labels_old),'rb'))
else:
    print("invalid fixed window labels path")

'''
if(os.path.exists(os.path.join(path,filename_random_2))):    #load data already processed
    all_random_labels_2 = pickle.load(open(os.path.join(path,filename_random_2),'rb'))
else:
    print("Invalid path for random labels")
'''



def count_gt_instances(pd_results): #counts the labelled words in the corpus, all the words in topic, all the labels and for each label counts how many times they appear and how many words they label
    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    topic_count =  {topic_name: {'topic_instances':0, 'topic_words':0} for topic_name in topic_names}
    total_words = 0 #total words in corpus
    total_labels = 0 #total label instances
    total_labelled_words = 0 #counts how many words were labelled
    for pd_result in pd_results:
        sentence = pd_result['sentence']
        total_words += len(sentence.split())
        for pd_label in zip(pd_result['topics&ref'],pd_result['word_indexes']):
            pd_topic = pd_label[0][0] #topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
            topic_count[pd_topic]['topic_instances'] += 1
            topic_count[pd_topic]['topic_words'] += len(pd_label[1])
            total_labels += 1
            total_labelled_words += len(pd_label[1])

    return topic_count, total_words, total_labels, total_labelled_words

topic_count_f, total_words_f, total_labels_f, total_labelled_words_f = count_gt_instances(fixed_window_labels)
topic_count_m, total_words_m, total_labels_m, total_labelled_words_m = count_gt_instances(moving_window_labels)
#topic_count_r, total_words_r, total_labels_r, total_labelled_words_r = count_gt_instances(all_random_labels)
topic_count_f2, total_words_f2, total_labels_f2, total_labelled_words_f2 = count_gt_instances(fixed_window_labels_old)
topic_count_m2, total_words_m2, total_labels_m2, total_labelled_words_m2 = count_gt_instances(moving_window_labels_old)
#topic_count_r2, total_words_r2, total_labels_r2, total_labelled_words_r2 = count_gt_instances(all_random_labels_2)
print(topic_count_f, "\n\n", topic_count_m,"\n\n", total_labels_f, "\n\n", total_labels_m, "\n\n",total_labelled_words_m, "\n\n\n")
print(topic_count_f2, "\n\n", topic_count_m2, "\n\n", total_labels_f2, "\n\n", total_labels_m2, "\n\n", total_labelled_words_m2)


In [ ]:
### EVALUATION - MODIFIED VERSION THAT CONSIDERS AS TRUE ALL THE MATCHES

import os, pickle, copy

def take_elements_topic(res, topic):
    refs = all_refs[topic]
    #print(refs)
    res_copy = copy.deepcopy(res) #res.copy()
    #print(res_copy)
    #for res in result:
    max_window_dim = int(list(res.keys())[-2].split(' ')[-1])
    #print(max_window_dim)
    for window in range(1,max_window_dim+1):
        all_res_refs = res[f"window {window}"]['ref']
        #print(all_res_refs)
        topic_indexes = [i for i in range(len(all_res_refs)) for r in refs if all_res_refs[i] == r] #returns the indexes of the matches that are related to a topic
        res_copy[f"window {window}"]['value'] = list(map(res_copy[f"window {window}"]['value'].__getitem__,topic_indexes))
        res_copy[f"window {window}"]['ref'] = list(map(res_copy[f"window {window}"]['ref'].__getitem__,topic_indexes))
        res_copy[f"window {window}"]['sentence'] = list(map(res_copy[f"window {window}"]['sentence'].__getitem__,topic_indexes))
        res_copy[f"window {window}"]['start_word_indexes'] = list(map(res_copy[f"window {window}"]['start_word_indexes'].__getitem__,topic_indexes))
    #print(res)
    return res_copy

def take_best_match(best_match, pd_result, values, window, th):
    dic_window = f'window {window}'
    #print(pd_result)
    #print(dic_window)
    #print(values)
    th_indexes = [i for i in range(len(values)) if values[i] > th] #returns the indexes of the values that are greater than th
    th_values = list(map(values.__getitem__,th_indexes)) #take all the values greater than th
    if len(th_values)==0:
        return best_match
    #th_ref = list(map(pd_result[dic_window]['ref'].__getitem__,th_indexes)) #take all the references that returns a value greater than th
    #th_matching_words = list(map(pd_result[dic_window]['sentence'].__getitem__,th_indexes)) #take all the sentence parts that return a value greater than new_th
    #th_topics = [key for key in all_refs.keys() for ref in all_refs[key] if tmp_ref == ref][0]
    #th_start_indexes =  list(map(pd_result[dic_window]['start_word_indexes'].__getitem__,th_indexes))
    max_val_index = pd_result[dic_window]['value'].index(max(th_values))
    max_val = pd_result[dic_window]['value'][max_val_index]
    #print("WINDOW:",window,"MAX VAL INDEX", max_val_index)
    #print("MAX VAL",max_val, " best_match['value']", best_match['value'])
    #print("PD RESULT", pd_result[dic_window])
    if max_val > best_match['value']:
        best_match['value'] = max_val
        best_match['ref'] = pd_result[dic_window]['ref'][max_val_index]
        best_match['matching_words'] = pd_result[dic_window]['sentence'][max_val_index]
        best_match['start_word_indexes'] = pd_result[dic_window]['start_word_indexes'][max_val_index]
        best_match['window'] = window
    return best_match

def extract_values(start_index,pd_result,window):  ## extract all the values related to the references of some topic (if a topic has 4 refs then I will have 4 vals)
    #print(window)
    window = f'window {window}'
    if start_index not in pd_result[window]['start_word_indexes']:
        return False
    else:
        values = []
        #print(start_index,type(start_index))
        valid_index = pd_result[window]['start_word_indexes'].index(start_index)
        i = valid_index
        while pd_result[window]['start_word_indexes'][i] == start_index:
            values.append(pd_result[window]['value'][i])
            i += 1
            if i >= len(pd_result[window]['start_word_indexes']):
                break

    '''
    elif start_index in pd_result[window]['start_word_indexes'] and start_index + 1  not in pd_result[window]['start_word_indexes']:
        valid_index = pd_result[window]['start_word_indexes'].index(start_index)
        values = pd_result[window]['value'][valid_index:]
    else:
        valid_index = pd_result[window]['start_word_indexes'].index(start_index)
        end_index = pd_result[window]['start_word_indexes'].index(start_index + 1)
        values = pd_result[window]['value'][valid_index:end_index]
    '''

    return values


def precision_recall_curve(y_true, pred_scores, thresholds):
    precisions = []
    recalls = []

    for threshold in thresholds:
        y_pred = [1 if score >= threshold else 0 for score in pred_scores]

        precision = sklearn.metrics.precision_score(y_true=y_true, y_pred=y_pred, pos_label=1, zero_division=0)
        recall = sklearn.metrics.recall_score(y_true=y_true, y_pred=y_pred, pos_label=1, zero_division=0)

        precisions.append(precision)
        recalls.append(recall)

    return precisions, recalls

def mean_IOU(gt_labels, pd_labels, best_windows, confidence = 0.5, type = 'moving_window'):
    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    topics_iou = {topic_gesture_description[key][0]: {'tot_intersection':0, 'tot_union':0, 'iou':0} for key in gt_labels_names}
    total_positive_topics = {topic_name: 0 for topic_name in topic_names}
    #topics_gt_labels = {topic_gesture_description[key][0]: [] for key in gt_labels_names}
    #for gt_result,pd_result in zip(gt_labels,pd_labels): #for each sentence
    #    assert(gt_result['sentence_index']==pd_result['sentence_index'])
    for topic in topic_names:
        for gt_result in gt_labels:
            for pd_result in pd_labels:
                if pd_result['whole sentence'] != gt_result['sentence']: #check that the results are referred to the same sentence before continuing
                    continue
                else:
                    #print(pd_result['sentence'], gt_result['sentence'])
                    all_gt_indexes = []
                    all_pd_indexes = []
                    start_index = 0 #we can take the results from the word in this index to avoid window overlapping
                    pd_result_copy = take_elements_topic(pd_result, topic) ##all the results referred to some topic
                    new_pd_result = {'topics&ref':[],'word_indexes':[],'values':[]}
                    #n_comparisons += 1 #count how many GT-PD comparisons we made for debugging

                    while start_index < len(pd_result['whole sentence'].split()):
                        best_match = {'value':0,'ref':'','matching_words':'','start_word_indexes':0,'window':1} #it contains the data related to the topic with highest confidence
                        if type == 'moving_window':
                            for window in list(pd_result_copy.keys())[1:-1]:
                                window = int(window.split(' ')[-1])
                                values = extract_values(start_index,pd_result_copy,window)
                                if values is False:
                                    break
                                else:
                                    best_match = take_best_match(best_match,pd_result_copy,values,window,confidence)

                        elif type == 'fixed_window':
                            window = best_windows[topic]['best_window']
                            if window > len(pd_result_copy['whole sentence'].split()):
                                break
                            values = extract_values(start_index,pd_result_copy,window)
                            if values is False:
                                break
                            else:
                                best_match = take_best_match(best_match,pd_result_copy,values,window,confidence)
                        else:
                            print("wrong type")
                            return
                        swi = best_match['start_word_indexes']
                        new_pd_result['topics&ref'].append((topic, best_match['ref']))
                        new_pd_result['word_indexes'].append([i for i in range(swi,swi + best_match['window'])])
                        new_pd_result['values'].append(best_match['value'])
                        start_index += best_match['window']
                    pd_result_copy.clear()
                    for j,gt_match in enumerate(gt_result['labels,word_indexes']):
                        gt_topic = topic_gesture_description[[el for el in gt_match[0].items()][0][1]][0] #extract the right topic from the gt label
                        if gt_topic != topic:
                            continue
                        else:
                            all_gt_indexes.append(gt_match[1])
                    for k,pd_match in enumerate(zip(new_pd_result['topics&ref'],new_pd_result['word_indexes'])):
                        pd_topic = pd_match[0][0]
                        if pd_topic != topic or new_pd_result['values'][k] < confidence: #the result must respect the confidence threshold
                            continue
                        else:
                            all_pd_indexes.append(pd_match[1])

                    union = len(set([j for sub in all_gt_indexes + all_pd_indexes for j in sub])) #total union in the sentence
                    alL_intersections = [set(j).intersection(k) for j in all_pd_indexes for k in all_gt_indexes if len(set(j).intersection(k))!=0] #all the possible intersections
                    intersection = len(set([i for a in alL_intersections for i in a]))  #total intersection area in the sentence
                    topics_iou[topic]['tot_intersection'] += intersection
                    topics_iou[topic]['tot_union'] += union
                    if union == 0:
                        continue
                    else:
                        topics_iou[topic]['iou'] = topics_iou[topic]['tot_intersection'] / topics_iou[topic]['tot_union']
                break

    total_considered_topics = 0
    for gt_result in gt_labels: #count how many gt labels we have for each topic. Will be considered only the topics that have at least one gt label
        for j,gt_label in enumerate(gt_result['labels,word_indexes']):
            gt_topic = topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
            total_positive_topics[gt_topic] += 1
    #for topic in topic_names:
    #    if total_positive_topics[topic] > 0:
    #        total_considered_topics += 1

    iou_sum = 0
    for topic in topic_names: #compute mean_iou
        if total_positive_topics[topic] > 0:
            total_considered_topics += 1
            #print("TOPIC NAME:", topic, "TOPIC_IOU:",topics_iou[topic]['iou'])
            iou_sum += topics_iou[topic]['iou']
    mean_IOU = iou_sum / total_considered_topics
    #print("MEAN IOU:", mean_IOU)

    return topics_iou, mean_IOU


def mean_average_precision(gt_labels, pd_labels, best_windows, type = 'moving_window'):

    gt_labels_names = topic_gesture_description.keys()
    topic_names = [topic_gesture_description[name][0] for name in gt_labels_names]
    total_positive_topics = {topic_name: 0 for topic_name in topic_names} #counts all the positive topics
    all_results = {topic_name: {th/20: {"Confidences":[],"TP":[], "FP":[], "Acc_TP":[], "Acc_FP":[], "Precision":[], "Recall":[]} for th in range(1,21,1)}
                   for topic_name in topic_names }
    average_precision = {topic_name:{th/20:0 for th in range(1,21,1)} for topic_name in topic_names}
    final_average_precision = {topic_name:0 for topic_name in topic_names}
    total_considered_topics = 0

    for gt_result in gt_labels:
        for j,gt_label in enumerate(gt_result['labels,word_indexes']):
            gt_topic = topic_gesture_description[[el for el in gt_label[0].items()][0][1]][0] #extract the right topic from the gt label
            total_positive_topics[gt_topic] += 1
    for topic in topic_names:
        if total_positive_topics[topic] > 0:
            total_considered_topics += 1
    print("total_topics:",total_considered_topics)
    for t in range(1,21,1): #compute all the info needed to compute average precision for all the possible thresholds of IOU
        th = t/20
        for topic in topic_names: #for each possible topic
            print(topic,t)
            if total_positive_topics[topic] == 0: #it does not make sense the computation of Average Precision (we can't compute the Recall)
                continue
            row = all_results[topic][th] #we will create rows in the results referred to the chosen topic and th. The final table will contain
                                         #all the info needed to compute average precision for a topic with a given IOU threshold
            acc_TP = 0
            acc_FP = 0
            n_comparisons = 0
            #####for skip, (gt_result,pd_result) in enumerate(zip(gt_labels,pd_labels)): #for each sentence
            #print("LEN:", len(gt_labels))
            for num,gt_result in enumerate(gt_labels):
                #if t==1 and topic == 'r_greet':
                    #print(num)
                #break_flag = False
                for pd_result in pd_labels:
                    if pd_result['whole sentence'] != gt_result['sentence']: #check that the results are referred to the same sentence before continuing
                        continue
                    else:
                        #print(pd_result)
                        #print(pd_result['whole sentence'], gt_result['sentence'])
                        start_index = 0 #we can take the results from the word in this index to avoid window overlapping
                        pd_result_copy = take_elements_topic(pd_result, topic) ##all the results referred to some topic
                        #print(pd_result_copy)
                        #print(pd_result)
                        new_pd_result = {'sentence': pd_result['whole sentence'], 'topics&ref':[],'word_indexes':[],'values':[]}
                        n_comparisons += 1 #count how many GT-PD comparisons we made for debugging

                        while start_index < len(pd_result_copy['whole sentence'].split()):
                            best_match = {'value':0,'ref':'','matching_words':'','start_word_indexes':0,'window':1} #it contains the data related to the topic with highest confidence
                            if type == 'moving_window':
                                for window in list(pd_result_copy.keys())[1:-1]:
                                    window = int(window.split(' ')[-1])
                                    values = extract_values(start_index,pd_result_copy,window)
                                    if values is False:
                                        continue
                                    else:
                                        #print("VALUES:",values)
                                        best_match = take_best_match(best_match,pd_result_copy,values,window,0)
                                        #print("BEST MATCH:",best_match)

                            elif type == 'fixed_window':
                                window = best_windows[topic]['best_window']
                                #print("window",window, "len", len(pd_result['whole sentence'].split()))
                                if window > len(pd_result_copy['whole sentence'].split()):
                                    break
                                values = extract_values(start_index,pd_result_copy,window)
                                #print(values)
                                if values is False:
                                    break
                                else:
                                    best_match = take_best_match(best_match,pd_result_copy,values,window,0)
                            else:
                                print("wrong type")
                                return
                            #print(best_match)
                            start_index += best_match['window']
                            if best_match['value']==0:
                                continue
                            else:
                                swi = best_match['start_word_indexes']
                                new_pd_result['topics&ref'].append((topic, best_match['ref']))
                                new_pd_result['word_indexes'].append([i for i in range(swi,swi + best_match['window'])])
                                new_pd_result['values'].append(best_match['value'])
                            #print("NEW PD RESULT:",new_pd_result)

                        #print(new_pd_result)
                        #print("PD_COPY!!!!", pd_result_copy['whole sentence'], len(pd_result_copy['whole sentence'].split()))
                        #print(pd_result_copy[f"window {best_match['window']}"] if window<=len(pd_result_copy['whole sentence'].split()) else '')
                        pd_result_copy.clear()
                        #print(new_pd_result)
                        detected_gt = {"IOU":[], "Confidence":[], 'gt_indexes':[]}
                        for k,pd_match in enumerate(zip(new_pd_result['topics&ref'],new_pd_result['word_indexes'])):
                            #print("I'm here")
                            #print(new_pd_result)
                            #print(gt_result)
                            #for k,pd_label_value in enumerate(tmp_values): #enumerate(zip(pd_result['topics&ref'],pd_result['word_indexes'])):
                            pd_topic = pd_match[0][0]  #  #extract the pd_topic from the pd_label   #do the same for the predicted label
                            if pd_topic != topic:
                                #print("SKIPPING PREDICTED:", k, pd_result)
                                continue
                            else: #right topic, check the matches with ground truths
                            #print("NOT SKIPPED PD TOPIC",skip,pd_result)
                                n_intersections = 0
                                tmp_IOU = 0
                                tmp_conf = 0
                                for j,gt_match in enumerate(gt_result['labels,word_indexes']):
                                    gt_topic = topic_gesture_description[[el for el in gt_match[0].items()][0][1]][0] #extract the right topic from the gt label
                                    if gt_topic != topic: #only if gt_topic and pd_topic are the same we can do the comparison
                                        continue
                                    else:
                                        intersection = len(set(pd_match[1]).intersection(gt_match[1]))
                                        if intersection != 0: #we append to a list that takes all the possible IOUs. Only the best one IOU is TP, for the other topics we have to check if they have predictions with the others GT
                                            union = len(set(pd_match[1] + gt_match[1]))
                                            IOU = intersection/union
                                            n_intersections += 1
                                            if IOU > tmp_IOU: #this allow to take only the best IOU between a PD and many GT, the others will be false negatives
                                                tmp_conf = new_pd_result['values'][k]
                                                tmp_IOU = IOU
                                                gt_index = j
                                        else:
                                            continue
                                if n_intersections == 0: #then the prediction has no intersections, so we have a false positive
                                    row["Confidences"].append(new_pd_result['values'][k])
                                    row["FP"].append(1)
                                    row["TP"].append(0)
                                    #print("False Positive!!",gt_result['labels,word_indexes'],new_pd_result['topics&ref'])
                                else: #we have to do another check: for each GT, we have to check which is the predicted topic that has highest IOU
                                      #with the current check we take only the GT that has highest IOU with a PD (if a GT has not a PD, then we will take into account later during the recall computation since this will be a FN)
                                    detected_gt["IOU"].append(tmp_IOU) #IOU
                                    detected_gt["Confidence"].append(tmp_conf) #confidence
                                    detected_gt["gt_indexes"].append(gt_index) #confidence
                                    #print("Appending some result")
                        #print(detected_gt)
                        if len(detected_gt["IOU"]) == 0: #then we have no TP, only FP which is already considered and appended as rows
                            #print("break")
                            break
                        else:
                            unique_indexes = set(detected_gt["gt_indexes"])
                            for idx in unique_indexes:
                                gt_idx = [i for i, d in enumerate(detected_gt["gt_indexes"]) if d == idx] #extract predictions related to same GT
                                IOUs = [detected_gt["IOU"][i] for i in gt_idx]
                                max_IOU_idx = detected_gt["IOU"].index(max(IOUs)) #take the best prediction related to a GT
                                if max(IOUs) < th:
                                    for i in gt_idx:
                                        row["Confidences"].append(detected_gt["Confidence"][i])
                                        row["FP"].append(1)
                                        row["TP"].append(0)
                                else:
                                    for i in gt_idx:
                                        if i == max_IOU_idx: #True Positive
                                            #print(detected_gt)
                                            #print(pd_result_copy)
                                            #print(gt_result)
                                            row["Confidences"].append(detected_gt["Confidence"][i])
                                            row["FP"].append(0)
                                            row["TP"].append(1)
                                        else:
                                            row["Confidences"].append(detected_gt["Confidence"][i])
                                            row["FP"].append(1)
                                            row["TP"].append(0)

            #### Sort all the elements by confidence for each topic
            sort_indexes = [i for i, x in sorted(enumerate(row["Confidences"]), key=lambda x: x[1], reverse=True)]
            #print(sort_indexes, row.keys())
            for key in row.keys():
                if key == "Recall" or key == "Precision" or key == "Acc_FP" or key == "Acc_TP":
                    continue
                row[key] = [row[key][i] for i in sort_indexes] #we order only Confidences, FP and TP
            for i in range(len(row["Confidences"])): #now compute Acc_TP, Acc_FP, Precisions and Recalls for each row
                if row["TP"][i] == 1:
                    acc_TP += 1
                else:
                    acc_FP += 1
                row["Acc_TP"].append(acc_TP)
                row["Acc_FP"].append(acc_FP)
                row["Precision"].append(acc_TP / (acc_TP + acc_FP))
                row["Recall"].append(acc_TP / total_positive_topics[topic])

    ######### COCO EVALUATOR
    mean_average_precision = 0
    #for iou in range(50,100,5): #### for each IOU, compute AP
    iou = 50
    th = iou/100
    for topic in topic_names:
        if total_positive_topics[topic] > 0:
            topic_results = all_results[topic][th]
            #fig = plt.figure()
            #plt.plot(topic_results["Recall"],topic_results["Precision"])
            #plt.pause(1)
            if len(topic_results["Recall"])<2:  #we need at least 2 points to find a curve and compute AUC
                continue
            AP = sklearn.metrics.auc(topic_results["Recall"], topic_results["Precision"]) * 100
            #print("AP:", AP)
            average_precision[topic][th] = AP
            final_average_precision[topic] += AP
        else:
            continue
    for topic in topic_names:
        final_average_precision[topic] = final_average_precision[topic] #/ 10 #now we have
        mean_average_precision += final_average_precision[topic]
    mean_average_precision = mean_average_precision/total_considered_topics
    print("n_comparisons:", n_comparisons)
    #print(round(mean_average_precision,2),"%")

    return average_precision, mean_average_precision

if __name__ == "__main__":

    path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
    #filename_best = "new_all_processed_sentences_best.pkl" #it contains all the possible results; a results is the best words match for each topic for each possible group of words inside the sentence, for all the sentences
    filename_similarities = "all_similarities_300.pkl"#"all_similarities_standard_base.pkl"
    filename_similarities_old = "all_similarities_300_old.pkl"
    #filename_random = "processed_sentences_random.pkl" #it contains all the random labels
    filename_labelled_sentences = "labelled_sentences_final.pkl" #it contains the ground truth labels
    filename_labelled_sentences_old = "labelled_sentences_final_old.pkl" #it contains the ground truth labels
    filename_windows = 'best_windows_sentences_standard_large_0_3.pkl'#'best_windows_sentences_standard_base_0_3.pkl'


    if(os.path.exists(os.path.join(path,filename_windows))):    #load data already processed
        b_windows = pickle.load(open(os.path.join(path,filename_windows),'rb'))
    else:
        print("Invalid path windows")

    if(os.path.exists(os.path.join(path,filename_similarities))):    #load data already processed
        result = pickle.load(open(os.path.join(path,filename_similarities),'rb'))
    else:
        print("Invalid path for all the results")

    if(os.path.exists(os.path.join(path,filename_similarities_old))):    #load data already processed
        result_old = pickle.load(open(os.path.join(path,filename_similarities_old),'rb'))
    else:
        print("Invalid path for all the results")

    if(os.path.exists(os.path.join(path,filename_labelled_sentences))):    #load data already processed
        all_real_labels = pickle.load(open(os.path.join(path,filename_labelled_sentences),'rb'))
        print("NEW",len(all_real_labels))
    else:
        print("invalid real labels path")

    if(os.path.exists(os.path.join(path,filename_labelled_sentences_old))):    #load data already processed
        all_real_labels_old = pickle.load(open(os.path.join(path,filename_labelled_sentences_old),'rb'))
        print("OLD",len(all_real_labels_old))
    else:
        print("invalid real labels path")

    #print(len(all_real_labels))

    #print(avg_labels[0],"\n",real_labels[0])
    #all_AP_rl, mAP_rl = mean_average_precision(all_real_labels, all_random_labels)
    #all_IOU_rl, mIOU_rl = mean_IOU(all_real_labels, all_random_labels)
    all_real_labels = all_real_labels + all_real_labels_old
    print(len(all_real_labels))
    #print(result[0])
    #print(b_windows)
    all_AP_ml, mAP_ml = mean_average_precision(all_real_labels, result, b_windows, type = 'moving_window')
    all_IOU_ml, mIOU_ml = mean_IOU(all_real_labels, result, b_windows, type = 'moving_window')
    all_AP_fl, mAP_fl = mean_average_precision(all_real_labels, result, b_windows, type = 'fixed_window')
    all_IOU_fl, mIOU_fl = mean_IOU(all_real_labels, result, b_windows, type = 'fixed_window')

    #print("mAP random labels:", round(mAP_rl,2),"   mIOU random labels:",round(mIOU_rl*100,2))
    print("mAP fixed window labels:", round(mAP_fl,2),"     mIOU fixed window labels:",round(mIOU_fl*100,2))
    print("mAP moving window labels:", round(mAP_ml,2),"    mIOU moving window labels:",round(mIOU_ml*100,2))
    #print(all_real_labels[0],moving_window_labels[0])
    #print(mAP)
    #print(mIOU*100)
    #print("m_IOU",all_IOU)
    #print("mAP", all_AP)
    #print(sentences_already_labelled[3:])
    #print(avg_labels[0],"\n",all_real_labels[199])
    #for el in result:

In [ ]:
############ OLD. DO NOT CONSIDER

### COMPUTE RESULTS FOR FIXED WINDOW

# give priority to each topic. Use the best window of each topic to take data of the sentence. If, for a topic, we have overlapping windows, chose the one that returns the best value.
# After we choose a topic given the value, the window size and the priority, all the words of that window can't be assigned to any other topic



def process_results_priority_policy_best_windows(best_windows = b_windows ,result = results, new_th = 0.2, type = 'conversation'):

    def take_elements_th(el, window_dim, new_th):
        all_values = el[f"window {window_dim}"]['value']  #all the values (results of the matches between topics and parts of the sentence
        th_indexes = [i for i in range(len(all_values)) if all_values[i] > new_th] #returns the indexes of the values that are greater than new_th
        values = list(map(all_values.__getitem__,th_indexes)) #take all the values greater than new_th
        refs = list(map(el[f"window {window_dim}"]['ref'].__getitem__,th_indexes)) #take all the references that return a value greater than new_th
        matching_words = list(map(el[f"window {window_dim}"]['sentence'].__getitem__,th_indexes)) #take all the sentence parts that return a value greater than new_th
        start_word_indexes = list(map(el[f"window {window_dim}"]['start_word_indexes'].__getitem__,th_indexes))

        return values, refs, matching_words, start_word_indexes

    final_labels=[]
    topics_ordered = sorted(best_windows.items(), key=lambda x: x[1]['priority'], reverse=True) #order the topics on priority values
    window_max_dim = 10
    for k,el in enumerate(result):
        sentence_labels = {'sentence':'', 'topics&ref':[],  'matching_words':[], 'values':[], 'priority_levels':[],
                           'sentence_index':k, 'word_indexes':[]} #it contains a sentence labelled
        #print(sentence_labels.keys())
        if type == 'conversation':
            sentence = el['whole sentence,speaker,turn'][0]
            speaker = el['whole sentence,speaker,turn'][1]
            turn = el['whole sentence,speaker,turn'][2]
        elif type == 'sentences':
            sentence = el['whole sentence']
        else:
            print("Undefined element")
            return

        sentence_labels['sentence'] = sentence
        sentence_splits = sentence.split()
        final_word_index = len(sentence_splits) - 1 #index of the last word (starting from 0)
        current_word_index = 0
        #idx_last_word  = current_word_index
        break_flag = False #if true,  we found the right label to a set of words in the sentence, so we can continue

        topic_index = 0
        #last_processed_word_idx = -1
        #processed_words = sentence_splits[0]

        if len(sentence_splits) < window_max_dim:
            window_max_d = len(sentence_splits)
        else:
            window_max_d = window_max_dim

        sentence_results = {}
        last_match_indexes = {}
        for i in range(1,window_max_d + 1): #take all the data related to the matches greater than a threshold for a given sentence
            values, refs, matching_words, start_word_indexes = take_elements_th(el, i, new_th)
            sentence_results[f"window {i}"] = {'matching_words':[], 'refs':[], 'values':[]}
            sentence_results[f"window {i}"]['values'] = values
            sentence_results[f"window {i}"]['matching_words'] = matching_words
            sentence_results[f"window {i}"]['refs'] = refs
            sentence_results[f"window {i}"]['start_word_indexes'] = start_word_indexes
            last_match_indexes[f"window {i}"] = 0 #this dict will contain, for each window_dim, the index of the first element from which we will start the search in

        while topic_index < len(topics_ordered) and current_word_index <= final_word_index:
            topic_name = topics_ordered[topic_index][0]
            window_dim = topics_ordered[topic_index][1]['best_window']
            #start_search_index = last_match_indexes[f"window {window_dim}"] #we take the data from this index since the previous ones were already used
            #print(processed_words, "    idx last match", idx_last_matches[f"window {window_dim}"], "    len results",len(s_results['matching_words']))
            #print("window_dim:",window_dim)
            #print("processed words:",processed_words, "  window_dim:", window_dim, "  dim_max:",max_dim, "  start_search_index:",start_search_index)
            #print("RESULTS:", s_results['matching_words'][start_search_index:], "window_dim:", window_dim, processed_words)
            if window_dim == 0 or f"window {window_dim}" not in sentence_results.keys(): #window dim is 0 if we didn't find any match during avg window computation
                topic_index +=  1
                continue
            s_results = sentence_results[f"window {window_dim}"]
            words_in_window = ' '.join(sentence_splits[current_word_index : current_word_index + window_dim])
            #it means that we have no more results for that window or that with that we reach the end of the sentence
            if last_match_indexes[f"window {window_dim}"] > (len(s_results['matching_words']) - 1) or current_word_index + window_dim > final_word_index:
                topic_index +=  1
                continue

            start_search_index = last_match_indexes[f"window {window_dim}"]
            if words_in_window in s_results['matching_words'][start_search_index:]: #then we have a match with a topic and the words in the window
                #win_results = el[f"window {window_dim}"] #all the results that we have for a given window dimension
                #values = win_results['value']   #all the values referred to the topics that return result greater than a threshold
                index = s_results['matching_words'][start_search_index:].index(words_in_window)  #since we have at least one topic > threshold, we take the first index of the first topic that match
                n_possible_topics = 0
                #print("HERE:", s_results['matching_words'][start_search_index:][index:], processed_words)
                while s_results['matching_words'][start_search_index:][index:][n_possible_topics] == words_in_window: #count how many topics match withe processed words
                    n_possible_topics = n_possible_topics + 1
                    #print(n_possible_topics)
                    if n_possible_topics >= len(s_results['matching_words'][start_search_index:][index:]): #avoid taking elements that are outside the array
                        break
                # these values are associated with the topics that match the # processed words
                all_match_values = s_results['values'][start_search_index:][index : index + n_possible_topics]
                all_indexes = [i for i in range(index, index + n_possible_topics)]
                #tmp_match = ordered_values[i]
                    #print(tmp_match)
                    #print("match:",tmp_best_match,"  window_dim:",window_dim, "  start_search_index", start_search_index)
                #all_best_values = tmp_values[index : index + n_possible_topics] #all the values of the topics that match with the words in the window
                for value,ind in zip(all_match_values,all_indexes): #check if the current topic has a match. If no, skip it and go to the next one by following the priorities
                    tmp_ref = s_results['refs'][start_search_index:][s_results['values'][start_search_index:].index(value)] #reference associated to best result
                    tmp_topic = [key for key in all_refs.keys() for ref in all_refs[key] if tmp_ref == ref][0] #extract the topic associated with reference
                    #ref = tmp_ref[ind] #reference associated to the result
                    #tmp_topic = [key for key in all_refs.keys() for tmp_ref in all_refs[key] if ref == tmp_ref][0] #extract the topic associated with reference
                    if topic_name != tmp_topic: #check if another topic is greater than new_th
                        continue
                    else:
                        w_dim=1
                        last_word_idx = current_word_index + window_dim
                        while current_word_index + w_dim <= len(sentence_splits) - 1 and w_dim <= window_max_d:    #update search indexes
                            s_res = sentence_results[f"window {w_dim}"]
                            if len(s_res['start_word_indexes']) != 0:  #then
                                #print("IDX LAST WORD:",tmp_idx_last_word)
                                tmp = [i for i in range(len(s_res['start_word_indexes'])) if s_res['start_word_indexes'][i] > last_word_idx]
                                #print(f"TMP INDEXES DIM :{window_dim}, RES:", tmp_idx_last_matches[f"window {window_dim}"])
                                if len(tmp) > 0: #it means that we have at least one value that will possibly match
                                    last_match_indexes[f"window {w_dim}"] = tmp[0]
                                else:
                                    last_match_indexes[f"window {w_dim}"] = len(s_res['matching_words']) #we can avoid to search again
                            w_dim = w_dim + 1
                        #print(sentence_labels.keys())
                        sentence_labels['topics&ref'].append((topic_name,tmp_ref)) #save the result
                        sentence_labels['matching_words'].append(words_in_window)
                        sentence_labels['values'].append(value)
                        sentence_labels['priority_levels'].append(topic_index+1)
                        sentence_labels['word_indexes'].append([i for i in range(current_word_index,current_word_index + window_dim)])
                        current_word_index = current_word_index + window_dim + 1
                        break_flag = True
                        break #break the loop and restart with another topic
                topic_index = topic_index + 1 #restart from next topic
                if break_flag == True:
                    topic_index = 0 #we found the topic, restart the topic search from the current_word_index and reset the break_flag
                    break_flag = False
            else:
                topic_index = topic_index + 1
                if topic_index >= len(topics_ordered):
                    current_word_index = current_word_index + 1
                    topic_index = 0
                continue #try with another topic. Changing topic means that we change the size of window so we take different words to compare. If no topics match, go to the next word
        final_labels.append(sentence_labels)
    return final_labels

path = r"C:\Users\ariel\Documents\PHD\GitHub\semantic_similarity\notebooks\final_results"
filename = 'all_similarities_600.pkl'
#filename_best = "new_all_processed_sentences_best.pkl"
#filename_local = "all_processed_sentences_local.pkl"
#filename_windows = "best_windows_200_sentences.pkl"
filename_windows = 'best_windows_sentences.pkl'

if(os.path.exists(os.path.join(path,filename))):    #load data already processed
    results = pickle.load(open(os.path.join(path,filename),'rb'))
else:
    print("Invalid path best")

if(os.path.exists(os.path.join(path,filename_windows))):    #load data already processed
    b_windows = pickle.load(open(os.path.join(path,filename_windows),'rb'))
else:
    print("Invalid path windows")

labels = process_results_priority_policy_best_windows(best_windows=b_windows,result=results, new_th=.3, type = 'sentences')
filename_labels =  "results_priority_policy_best_windows.pkl"
print("writing to file...")
pickle.dump(labels  , open(os.path.join(path,filename_labels),'wb'))

for label in labels:
    print(label, "\n \n \n \n")